# Парсинг данных с сайта АСВ

## Парсинг данных с главной страницы

Так как сайт динамический, чтобы учесть обновления буду использовать библиотеку selenium.

По моим наблюдениям, стандартные разделы каждой страницы: Общая информация о банке, страховые выплаты, конкурсное производство/ликвидация с подразделами "Выплачено кредиторам", "Сведения о формировании конкурсной массы", "Расходы", "Юридическая работа". Однако на некоторых страницах присутствует только общая информация о банке, конкурсное производство не отображается, потому что оно завершено или была совершена ликвидация банка.

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [ ]:
"""
Общая логика кода: 
Сначала я запускаю chrome.driver - программу, которая имитирует действия пользователя на сайте и обеспечивает переходы по ссылкам. На странице действует "ленивая загрузка" - 
такой способ загрузки страницы, что пока пользователь не прокрутит до конца страницу она не загрузит блок. Это сложность для selenium, потому что он не видит такие незагруженные разделы, 
поэтмоу в код добавляем условие прокрутки страницы до конца.

Эта программа работает примерно 1,5 минуты
"""

chrome_options = Options()
driver = webdriver.Chrome(options=chrome_options) # Инициализация WebDriver

base_url = "https://www.asv.org.ru/banks?category=insurance-event" # URL главной страницы

try:
    driver.get(base_url)

    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.catalog-item__body'))) # Ожидание 20 секунд загрузки элементов

    # Прокрутка страницы до конца
    bank_data = []
    last_count = 0
    same_count_iterations = 0  # Счетчик итераций без изменений
    max_same_count_iterations = 10 # Максимальное число итераций без изменений

    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)

        bank_elements = driver.find_elements(By.CSS_SELECTOR, 'div.catalog-item__body') # Находим все элементы, содержащие информацию о банке
        current_count = len(bank_elements)

        print(f"Найдено {current_count} банка(ов) на текущей итерации") # На 18.06.2025 на странице 553 банка, нам надо убедиться, что мы нашли все из них

        if current_count == last_count:
            same_count_iterations += 1
            print(f"Количество элементов не изменилось ({same_count_iterations}/{max_same_count_iterations})")

            if same_count_iterations >= max_same_count_iterations:
                print("Достигнут конец списка банков") # Если количество не меняется 10 раз подряд, значит, достигли конца
                break  

        else:
            same_count_iterations = 0 # Сбрасываем счетчик, если количество изменилось
            print("Количество элементов увеличилось, продолжаем прокрутку")


        last_count = current_count

    # Перебираем элементы и извлекаем нужную информацию
    for element in bank_elements:
        try:
            title_element = element.find_element(By.CSS_SELECTOR, 'div.catalog-item__title') # Название банка
            bank_name = title_element.text.strip()
        except:
            bank_name = "Название не найдено"

        try:
            status_element = element.find_element(By.CSS_SELECTOR, 'div.catalog-item__status') # Извлекаем информацию о лицензии
            license_info = status_element.text.strip()
        except:
            license_info = "Информация о лицензии отсутствует"

        try:
            try:
                footer_element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, 'div.catalog-item__footer.ng-star-inserted')) # Извлекаем информацию о состоянии конкурсного производства
                )
            except Exception as e:
                footer_element = None
                print(f"Элемент footer не найден")

            if footer_element:
                try:
                    span_elements = footer_element.find_elements(By.TAG_NAME, 'span')
                    footer_info = " ".join([span.text.strip() for span in span_elements])
                except Exception as e:
                    footer_info = f"Ошибка при извлечении текста из span"
            else:
                footer_info = "Элемент footer отсутствует"
        except:
            footer_info = "Информация о футере отсутствует"

        bank_info = {
            'Название банка': bank_name,
            'Информация о лицензии': license_info,
            'Информация о футере': footer_info
        }
        bank_data.append(bank_info)

    for bank in bank_data:
        print(f"Название банка: {bank.get('Название банка', 'Не найдено')}")
        print(f"Информация о лицензии: {bank.get('Информация о лицензии', 'Не найдено')}")
        print(f"Информация о футере: {bank.get('Информация о футере', 'Не найдено')}")
        print("-" * 30)


except Exception as e:
    print(f"Произошла общая ошибка")

finally:
    driver.quit()

Найдено 104 банка(ов) на текущей итерации
Количество элементов увеличилось, продолжаем прокрутку
Найдено 117 банка(ов) на текущей итерации
Количество элементов увеличилось, продолжаем прокрутку
Найдено 145 банка(ов) на текущей итерации
Количество элементов увеличилось, продолжаем прокрутку
Найдено 145 банка(ов) на текущей итерации
Количество элементов не изменилось (1/10)
Найдено 146 банка(ов) на текущей итерации
Количество элементов увеличилось, продолжаем прокрутку
Найдено 178 банка(ов) на текущей итерации
Количество элементов увеличилось, продолжаем прокрутку
Найдено 213 банка(ов) на текущей итерации
Количество элементов увеличилось, продолжаем прокрутку
Найдено 221 банка(ов) на текущей итерации
Количество элементов увеличилось, продолжаем прокрутку
Найдено 309 банка(ов) на текущей итерации
Количество элементов увеличилось, продолжаем прокрутку
Найдено 309 банка(ов) на текущей итерации
Количество элементов не изменилось (1/10)
Найдено 370 банка(ов) на текущей итерации
Количество эле

Получили датафрейм для данных только с главной страницы.

In [4]:
import pandas as pd
df_main_page = pd.DataFrame(bank_data)
df_main_page

,Название банка,Информация о лицензии,Информация о футере
0,«1Банк» АКБ (ПАО),ЛИЦЕНЗИЯ ОТОЗВАНА 17 МАРТА 2016,Идет конкурсное производство
1,«Ав­то­град­банк» (АО),ЛИЦЕНЗИЯ ОТОЗВАНА 17 ИЮНЯ 2024,Идет конкурсное производство
2,«Агро­ин­ком­банк» (ПАО),ЛИЦЕНЗИЯ ОТОЗВАНА 2 НОЯБРЯ 2015,Идет конкурсное производство
3,«Агро­со­юз» КБ (ООО),ЛИЦЕНЗИЯ ОТОЗВАНА 7 НОЯБРЯ 2018,Идет конкурсное производство
4,«Агро­хим­банк» (ОАО),ЛИЦЕНЗИЯ ОТОЗВАНА 30 ДЕКАБРЯ 2008,Идет конкурсное производство
...,...,...,...
548,«Юник­банк» АКБ (ЗАО),ЛИЦЕНЗИЯ ОТОЗВАНА 23 АПРЕЛЯ 2009,Идет конкурсное производство
549,«Юни­кор­банк» КБ (ОАО),ЛИЦЕНЗИЯ ОТОЗВАНА 25 ИЮЛЯ 2014,Идет конкурсное производство
550,«Юнит­банк» КБ (ЗАО),ЛИЦЕНЗИЯ ОТОЗВАНА 16 ОКТЯБРЯ 2008,Идет конкурсное производство
551,«ЮРБ» (ООО),ЛИЦЕНЗИЯ ОТОЗВАНА 20 ИЮЛЯ 2018,Идет конкурсное производство


## Парсинг ссылок на банки с главной страницы

Для того, чтобы спарсить данные со всех страниц банков, которые находятся на главной странице, надо спарсить с главной страницы все ссылки на банки.

In [9]:
"""Эта программа работает примерно 30 минут"""

chrome_options = Options()
driver = webdriver.Chrome(options=chrome_options)

base_url = "https://www.asv.org.ru/banks?category=insurance-event"

try:
    driver.get(base_url)

   
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.catalog-item__body')))  # Ожидание загрузки элементов

    # Прокрутка страницы до конца и получение ссылок на страницы банков
    bank_links = []
    last_count = 0
    same_count_iterations = 0
    max_same_count_iterations = 10 # Количество итераций, в течение которых будем пытаться найти ссылку, если по истечении 10 раз ничего не изменилось, код прерывается и считает, что данные готовы

    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
        bank_elements = driver.find_elements(By.CSS_SELECTOR, 'div.catalog-item__body') # Находим все элементы, содержащие информацию о банке
        current_count = len(bank_elements)

        print(f"Найдено {current_count} элементов на текущей итерации") # Проверяем все ли ссылки мы нашли

        if current_count == last_count:
            same_count_iterations += 1
            print(f"Количество элементов не изменилось ({same_count_iterations}/{max_same_count_iterations})")

            if same_count_iterations >= max_same_count_iterations:
                print("Достигнут конец списка банков")
                break

        else:
            same_count_iterations = 0
            print("Количество элементов увеличилось, продолжаем прокрутку")

        last_count = current_count

        try: 
            link_elements = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a.catalog-item'))) # Находим все ссылки
            for link_element in link_elements:
                link = link_element.get_attribute('href')  # Извлекаем ссылку
                if link and link not in bank_links:
                    bank_links.append(link)  # Добавляем ссылку в список, если ее там еще нет

        except Exception as e:
            print()

    print(f"Найдено {len(bank_links)} ссылок на страницы банков")
    print("Найденные ссылки:")
    for link in bank_links:
        print(link)

    all_bank_data = []
    for link in bank_links:
        print(f"Переходим по ссылке: {link}")
        driver.get(link)
        time.sleep(2)

        try:
            link_element = driver.find_element(By.CSS_SELECTOR,'body > app-root > app-layout-main > div > div.layout-container.body.pt-sm-4.pt-md-0.pb-8 > div > app-page-banks > app-block-page-organization-list > app-wrapper-page > div > div.page-wrapper__body > div > app-organization-list > div.row.mt-3.mt-lg-5.ng-star-inserted > div > div:nth-child(1) > div.catalog-list.catalog-list_grid > app-catalog-list-item-card > a') # Находим тег <a> внутри элемента
            info = link_element.get_attribute('href')
            all_bank_data.append(info)
        except:
            print()
    print(f"Найдено {len(all_bank_data)} ссылок на страницы банков")

except Exception as e:
    print(f"Произошла общая ошибка")

finally:
    driver.quit()

Найдено 104 элементов на текущей итерации
Количество элементов увеличилось, продолжаем прокрутку
Найдено 129 элементов на текущей итерации
Количество элементов увеличилось, продолжаем прокрутку
Найдено 146 элементов на текущей итерации
Количество элементов увеличилось, продолжаем прокрутку
Найдено 152 элементов на текущей итерации
Количество элементов увеличилось, продолжаем прокрутку
Найдено 213 элементов на текущей итерации
Количество элементов увеличилось, продолжаем прокрутку
Найдено 221 элементов на текущей итерации
Количество элементов увеличилось, продолжаем прокрутку
Найдено 327 элементов на текущей итерации
Количество элементов увеличилось, продолжаем прокрутку
Найдено 470 элементов на текущей итерации
Количество элементов увеличилось, продолжаем прокрутку
Найдено 497 элементов на текущей итерации
Количество элементов увеличилось, продолжаем прокрутку
Найдено 524 элементов на текущей итерации
Количество элементов увеличилось, продолжаем прокрутку
Найдено 531 элементов на текущ

Итоговый список всех ссылок на банки

In [11]:
len(bank_links)

553

In [10]:
bank_links

['https://www.asv.org.ru/banks/1bank-akb-pao',
 'https://www.asv.org.ru/banks/ao-avtogradbank',
 'https://www.asv.org.ru/banks/agroinkombank-pao',
 'https://www.asv.org.ru/banks/agrosoyuz-kb-ooo',
 'https://www.asv.org.ru/banks/agrokhimbank-oao',
 'https://www.asv.org.ru/banks/adam-interneshnl-akb-ao',
 'https://www.asv.org.ru/banks/admiralteyskiy-kb-ooo',
 'https://www.asv.org.ru/banks/azimut-akb-pao',
 'https://www.asv.org.ru/banks/ayvi-bank-ao',
 'https://www.asv.org.ru/banks/aymanibank-kb-ooo',
 'https://www.asv.org.ru/banks/ak-bank-ao',
 'https://www.asv.org.ru/banks/aksonbank-kb-ooo',
 'https://www.asv.org.ru/banks/akb-aktiv-bank-pao',
 'https://www.asv.org.ru/banks/aktsent-akb-pao',
 'https://www.asv.org.ru/banks/aktsiya-akb-oao',
 'https://www.asv.org.ru/banks/aldanzolotobank-ab-ao',
 'https://www.asv.org.ru/banks/altaybiznes-bank-akb-ao',
 'https://www.asv.org.ru/banks/ooo-kb-alba-alyans',
 'https://www.asv.org.ru/banks/alta-bank-kb-zao',
 'https://www.asv.org.ru/banks/alyans-

In [43]:
bank_links_1 = ['https://www.asv.org.ru/banks/1bank-akb-pao',
 'https://www.asv.org.ru/banks/ao-avtogradbank',
 'https://www.asv.org.ru/banks/agroinkombank-pao',
 'https://www.asv.org.ru/banks/agrosoyuz-kb-ooo',
 'https://www.asv.org.ru/banks/agrokhimbank-oao',
 'https://www.asv.org.ru/banks/adam-interneshnl-akb-ao',
 'https://www.asv.org.ru/banks/admiralteyskiy-kb-ooo',
 'https://www.asv.org.ru/banks/azimut-akb-pao',
 'https://www.asv.org.ru/banks/ayvi-bank-ao',
 'https://www.asv.org.ru/banks/aymanibank-kb-ooo',
 'https://www.asv.org.ru/banks/ak-bank-ao',
 'https://www.asv.org.ru/banks/aksonbank-kb-ooo',
 'https://www.asv.org.ru/banks/akb-aktiv-bank-pao',
 'https://www.asv.org.ru/banks/aktsent-akb-pao',
 'https://www.asv.org.ru/banks/aktsiya-akb-oao',
 'https://www.asv.org.ru/banks/aldanzolotobank-ab-ao',
 'https://www.asv.org.ru/banks/altaybiznes-bank-akb-ao',
 'https://www.asv.org.ru/banks/ooo-kb-alba-alyans',
 'https://www.asv.org.ru/banks/alta-bank-kb-zao',
 'https://www.asv.org.ru/banks/alyans-bank-kb-ooo',
 'https://www.asv.org.ru/banks/amb-bank-pao',
 'https://www.asv.org.ru/banks/amt-bank-ooo',
 'https://www.asv.org.ru/banks/andzhibank-akb-oao',
 'https://www.asv.org.ru/banks/anelik-ru-kb-ooo',
 'https://www.asv.org.ru/banks/ankor-bank-ao',
 'https://www.asv.org.ru/banks/antalbank-kb-ooo',
 'https://www.asv.org.ru/banks/antares-kb-ooo',
 'https://www.asv.org.ru/banks/arvesta-akgb-zao',
 'https://www.asv.org.ru/banks/arksbank-ao',
 'https://www.asv.org.ru/banks/arsenal-kb-ooo',
 'https://www.asv.org.ru/banks/art-bank-kb-ooo',
 'https://www.asv.org.ru/banks/askaniya-trast-kb-ooo',
 'https://www.asv.org.ru/banks/askold-oao',
 'https://www.asv.org.ru/banks/aspekt-ab-ao',
 'https://www.asv.org.ru/banks/assotsiatsiya-kb-ao',
 'https://www.asv.org.ru/banks/astrakhanprombank-oao',
 'https://www.asv.org.ru/banks/auerbank-kb-ooo',
 'https://www.asv.org.ru/banks/af-bank-oao',
 'https://www.asv.org.ru/banks/baykalbank-pao',
 'https://www.asv.org.ru/banks/baltika-akb-pao',
 'https://www.asv.org.ru/banks/baltkredobank-kb-zao',
 'https://www.asv.org.ru/banks/ooo-bank-bkf',
 'https://www.asv.org.ru/banks/bank-bft-kb-ooo',
 'https://www.asv.org.ru/banks/bank-vefk-ural-zao',
 'https://www.asv.org.ru/banks/bank-voronezh-ao',
 'https://www.asv.org.ru/banks/bank-vysokikh-tekhnologiy-kb-ooo',
 'https://www.asv.org.ru/banks/bank-gorod-ao',
 'https://www.asv.org.ru/banks/bank-ekaterininskiy-pao',
 'https://www.asv.org.ru/banks/bank-imperiya-ooo',
 'https://www.asv.org.ru/banks/bank-innovatsiy-i-razvitiya-ao',
 'https://www.asv.org.ru/banks/bank-premer-kredit-pao',
 'https://www.asv.org.ru/banks/bank-raschetov-i-sberezheniy-kb-ooo',
 'https://www.asv.org.ru/banks/bank-razvitiya-biznesa-ooo',
 'https://www.asv.org.ru/banks/bank-rossiyskiy-kredit-oao',
 'https://www.asv.org.ru/banks/bank-fininvest-ooo',
 'https://www.asv.org.ru/banks/bank-ekonomicheskiy-soyuz-ao',
 'https://www.asv.org.ru/banks/bank-t-oao',
 'https://www.asv.org.ru/banks/bank24-ru-oao',
 'https://www.asv.org.ru/banks/bankirskiy-dom-akb-ao',
 'https://www.asv.org.ru/banks/basmannyy-akb-zao',
 'https://www.asv.org.ru/banks/bva-bank-zao',
 'https://www.asv.org.ru/banks/bdb-kb-ooo',
 'https://www.asv.org.ru/banks/benifit-bank-akb-zao',
 'https://www.asv.org.ru/banks/bzl-investbank-oao',
 'https://www.asv.org.ru/banks/biznesbank-ooo',
 'https://www.asv.org.ru/banks/blagovest-kb-ooo',
 'https://www.asv.org.ru/banks/bmb-kb-ooo',
 'https://www.asv.org.ru/banks/bnkv-akb-ao',
 'https://www.asv.org.ru/banks/bogorodskiy-bank-ooo',
 'https://www.asv.org.ru/banks/borskiy-kombank-ooo',
 'https://www.asv.org.ru/banks/bpf-ab-zao',
 'https://www.asv.org.ru/banks/bank-razvitiya-regiona-akb-oao',
 'https://www.asv.org.ru/banks/brt-kb-ao',
 'https://www.asv.org.ru/banks/btf-kb-ao',
 'https://www.asv.org.ru/banks/bulgar-bank-ao',
 'https://www.asv.org.ru/banks/bum-bank-ooo',
 'https://www.asv.org.ru/banks/bumerang-komsotsbank-ao',
 'https://www.asv.org.ru/banks/bfg-kredit-kb-ooo',
 'https://www.asv.org.ru/banks/vbb-kb-ooo',
 'https://www.asv.org.ru/banks/vvb-bank-pao',
 'https://www.asv.org.ru/banks/vebrr-kb-ooo',
 'https://www.asv.org.ru/banks/vega-bank-kb-ooo',
 'https://www.asv.org.ru/banks/vek-akb-ao',
 'https://www.asv.org.ru/banks/vekombank-kb-oao',
 'https://www.asv.org.ru/banks/vestinterbank-ooo',
 'https://www.asv.org.ru/banks/kb-vetb-ao',
 'https://www.asv.org.ru/banks/vzaimodeystvie-kb-ooo',
 'https://www.asv.org.ru/banks/vitas-bank-ooo',
 'https://www.asv.org.ru/banks/vityaz-kb-ooo',
 'https://www.asv.org.ru/banks/vkb-oao',
 'https://www.asv.org.ru/banks/vladprombank-ooo',
 'https://www.asv.org.ru/banks/vlbank-ao',
 'https://www.asv.org.ru/banks/vneshprombank-ooo',
 'https://www.asv.org.ru/banks/volgo-don-bank-oao',
 'https://www.asv.org.ru/banks/volgo-kamskiy-bank-oao',
 'https://www.asv.org.ru/banks/vologdabank-ao',
 'https://www.asv.org.ru/banks/vostokkreditbank-oao',
 'https://www.asv.org.ru/banks/vostsibtranskombank-ao',
 'https://www.asv.org.ru/banks/vpb-bank-ao',
 'https://www.asv.org.ru/banks/vsb-ooo',
 'https://www.asv.org.ru/banks/vyborg-bank-pao',
 'https://www.asv.org.ru/banks/vympel-kb-oao',
 'https://www.asv.org.ru/banks/velkombank-zao',
 'https://www.asv.org.ru/banks/ao-velton-bank',
 'https://www.asv.org.ru/banks/gagarinskiy-kb-ao',
 'https://www.asv.org.ru/banks/gazbank-akb-ao',
 'https://www.asv.org.ru/banks/gazinvestbank-kb-oao',
 'https://www.asv.org.ru/banks/gazstroybank-akb-ao',
 'https://www.asv.org.ru/banks/galabank-akb-zao',
 'https://www.asv.org.ru/banks/kb-garant-invest-ao',
 'https://www.asv.org.ru/banks/gelendzhik-bank-pao',
 'https://www.asv.org.ru/banks/kb-geobank-ooo',
 'https://www.asv.org.ru/banks/ooo-kb-gefest',
 'https://www.asv.org.ru/banks/grad-bank-akb-oao',
 'https://www.asv.org.ru/banks/ao-grand-invest-bank',
 'https://www.asv.org.ru/banks/grinkombank-ao',
 'https://www.asv.org.ru/banks/grinfildbank-ao',
 'https://www.asv.org.ru/banks/dagestan-kbsr-ooo',
 'https://www.asv.org.ru/banks/dagenergobank-ooo',
 'https://www.asv.org.ru/banks/daletbank-akb-ao',
 'https://www.asv.org.ru/banks/ao-danske-bank',
 'https://www.asv.org.ru/banks/derbent-kredit-akzb-oao',
 'https://www.asv.org.ru/banks/dig-bank-iab-oao',
 'https://www.asv.org.ru/banks/dil-bank-ooo',
 'https://www.asv.org.ru/banks/ao-mkb-dom-bank',
 'https://www.asv.org.ru/banks/donbank-kb-oao',
 'https://www.asv.org.ru/banks/doninvest-kb-ooo',
 'https://www.asv.org.ru/banks/donkhlebbank-pao',
 'https://www.asv.org.ru/banks/doris-bank-kb-ooo',
 'https://www.asv.org.ru/banks/evraziya-tsentr-mkb-zao',
 'https://www.asv.org.ru/banks/ooo-kb-evroaziatskiy-investitsionnyy-bank',
 'https://www.asv.org.ru/banks/evroaksis-bank-ao',
 'https://www.asv.org.ru/banks/evrokapital-alyans-kb-ooo',
 'https://www.asv.org.ru/banks/evrokommerts-kb-pao',
 'https://www.asv.org.ru/banks/evrokredit-bank-ooo',
 'https://www.asv.org.ru/banks/evromet-akb-pao',
 'https://www.asv.org.ru/banks/evropeyskiy-ekspress-kb-ooo',
 'https://www.asv.org.ru/banks/evroprominvest-kb-oao',
 'https://www.asv.org.ru/banks/evrosib-bank-oao',
 'https://www.asv.org.ru/banks/evrositibank-kb-pao',
 'https://www.asv.org.ru/banks/evrostandart-kb-ooo',
 'https://www.asv.org.ru/banks/evropeyskiy-trastovyy-bank-kb-zao',
 'https://www.asv.org.ru/banks/edinstvennyy-kb-ooo',
 'https://www.asv.org.ru/banks/evropeyskiy-industrialnyy-bank-akb-zao',
 'https://www.asv.org.ru/banks/enisey-akb-pao',
 'https://www.asv.org.ru/banks/zhilkredit-kb-ooo',
 'https://www.asv.org.ru/banks/zamoskvoretskiy-mkb-oao',
 'https://www.asv.org.ru/banks/zapadnyy-bank-oao',
 'https://www.asv.org.ru/banks/ao-zauber-bank',
 'https://www.asv.org.ru/banks/zelak-bank-kb-oao',
 'https://www.asv.org.ru/banks/zernobank-ao',
 'https://www.asv.org.ru/banks/zlatkombank-kb-ao',
 'https://www.asv.org.ru/banks/ivanovo-kb-ao',
 'https://www.asv.org.ru/banks/ideya-bank-pao',
 'https://www.asv.org.ru/banks/ikf-kb-ooo',
 'https://www.asv.org.ru/banks/imbank-aib-oao',
 'https://www.asv.org.ru/banks/invest-ekobank-kb-ooo',
 'https://www.asv.org.ru/banks/investbank-akb-oao',
 'https://www.asv.org.ru/banks/investitsionnyy-promenergobank-kb-oao',
 'https://www.asv.org.ru/banks/investitsionnyy-soyuz-kb-ooo',
 'https://www.asv.org.ru/banks/industrialnyy-kb-ooo',
 'https://www.asv.org.ru/banks/inkarobank-akb-ao',
 'https://www.asv.org.ru/banks/inkasbank-oao',
 'https://www.asv.org.ru/banks/instroybank-kb-ooo',
 'https://www.asv.org.ru/banks/integro-kb-oao',
 'https://www.asv.org.ru/banks/intelfinans-kb-ooo',
 'https://www.asv.org.ru/banks/interaktivnyy-bank-ooo',
 'https://www.asv.org.ru/banks/interkommerts-kb-ooo',
 'https://www.asv.org.ru/banks/interkoopbank-akb-ao',
 'https://www.asv.org.ru/banks/internatsionalnyy-torgovyy-bank-kb-ooo',
 'https://www.asv.org.ru/banks/ao-kb-interprombank',
 'https://www.asv.org.ru/banks/intekhbank-pao',
 'https://www.asv.org.ru/banks/intrastbank-akb-oao',
 'https://www.asv.org.ru/banks/informprogress-kb-ooo',
 'https://www.asv.org.ru/banks/ipotek-bank-zao',
 'https://www.asv.org.ru/banks/ipf-bank',
 'https://www.asv.org.ru/banks/akb-irs-ao',
 'https://www.asv.org.ru/banks/itb-bank-ao',
 'https://www.asv.org.ru/banks/kamskiy-gorizont-kb-ooo',
 'https://www.asv.org.ru/banks/kamchatka-kb-ooo',
 'https://www.asv.org.ru/banks/kamchatkomagroprombank-pao',
 'https://www.asv.org.ru/banks/kanskiy-kb-ooo',
 'https://www.asv.org.ru/banks/kapital-kredit-kb-ooo',
 'https://www.asv.org.ru/banks/kapitalbank-akb-pao',
 'https://www.asv.org.ru/banks/kauri-kb-oao',
 'https://www.asv.org.ru/banks/kbr-bank-kb-ooo',
 'https://www.asv.org.ru/banks/kbts-kb-ooo',
 'https://www.asv.org.ru/banks/ooo-kbtsa',
 'https://www.asv.org.ru/banks/kemsotsinbank-ao',
 'https://www.asv.org.ru/banks/kivi-bank-ao',
 'https://www.asv.org.ru/banks/ao-bank-kkb',
 'https://www.asv.org.ru/banks/klientskiy-bank-ao',
 'https://www.asv.org.ru/banks/konstans-bank-akb-ao',
 'https://www.asv.org.ru/banks/konfidens-bank-kb-ooo',
 'https://www.asv.org.ru/banks/kor-akb-ao',
 'https://www.asv.org.ru/banks/krayniy-sever-nab-oao',
 'https://www.asv.org.ru/banks/kranbank-ao',
 'https://www.asv.org.ru/banks/kredit-ekspress-kb-ooo',
 'https://www.asv.org.ru/banks/kredit-kontakt-kb-ooo',
 'https://www.asv.org.ru/banks/kredit-moskva-bank-pao',
 'https://www.asv.org.ru/banks/kreditbank-kkb-oao',
 'https://www.asv.org.ru/banks/kreditinvest-kb-ooo',
 'https://www.asv.org.ru/banks/kreditsoyuzkombank-akb-zao',
 'https://www.asv.org.ru/banks/kredo-finans-kb-ooo',
 'https://www.asv.org.ru/banks/kredprombank-ao',
 'https://www.asv.org.ru/banks/krk-kb-oao',
 'https://www.asv.org.ru/banks/ao-krosna-bank',
 'https://www.asv.org.ru/banks/krossinvestbank-akb-oao',
 'https://www.asv.org.ru/banks/krylovskiy-akb-ao',
 'https://www.asv.org.ru/banks/akksb-ks-bank-pao',
 'https://www.asv.org.ru/banks/kubanskiy-universalnyy-bank-kb-ooo',
 'https://www.asv.org.ru/banks/bank-kuznetskiy-most-ao',
 'https://www.asv.org.ru/banks/kurganprombank-akb-oao',
 'https://www.asv.org.ru/banks/lada-kredit-kb-zao',
 'https://www.asv.org.ru/banks/laytbank-kb-ooo',
 'https://www.asv.org.ru/banks/legion-akb-ao',
 'https://www.asv.org.ru/banks/lenoblbank-ooo',
 'https://www.asv.org.ru/banks/lesbank-akb-oao',
 'https://www.asv.org.ru/banks/link-bank-akb-oao',
 'https://www.asv.org.ru/banks/lipetskoblbank-oao',
 'https://www.asv.org.ru/banks/logos-ikb-ooo',
 'https://www.asv.org.ru/banks/m-bank-zao',
 'https://www.asv.org.ru/banks/m-plyus-ooo',
 'https://www.asv.org.ru/banks/m2m-prayvet-bank-pao',
 'https://www.asv.org.ru/banks/mab-zao',
 'https://www.asv.org.ru/banks/ao-maykopbank',
 'https://www.asv.org.ru/banks/ooo-bank-mayskiy',
 'https://www.asv.org.ru/banks/maksimum-kb-oao',
 'https://www.asv.org.ru/banks/mast-bank-kb-oao',
 'https://www.asv.org.ru/banks/master-bank-kb-oao',
 'https://www.asv.org.ru/banks/master-kapital-akb-oao',
 'https://www.asv.org.ru/banks/mbsp-bank-ao',
 'https://www.asv.org.ru/banks/mbfi-bank-oao',
 'https://www.asv.org.ru/banks/mber-kb-ooo']

## Парсинг данных с каждой страницы

In [44]:
"""Эта программа работает примерно 25 минут"""

chrome_options = Options()
driver = webdriver.Chrome(options=chrome_options)

try:
    bank_links = bank_links_1
    all_bank_data_1 = [] # Парсим данные с каждой страницы банка
    for link in bank_links:
        print(f"Переходим по ссылке: {link}")
        driver.get(link)
        time.sleep(5)

        bank_info = {}  # Создаем словарь для хранения информации о банке
        try:
            try:
                ul_element = WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, 'div.foldable__body > div > app-company-block-requisites-table > ul')) # В коде страницы этот элемент располагается в ветви с названием 'ul', поэтому нам надо найти все 'ul', чтобы найти все элементы
                )
            except Exception as e:
                ul_element = None
                print()

            if ul_element:
                li_elements = ul_element.find_elements(By.TAG_NAME, "li") # Находим все элементы <li> внутри <ul>

                for li in li_elements:
                    try:
                        
                        title_element = li.find_element(By.CSS_SELECTOR, 'div.requisite-item__title > div') # Извлекаем название поля
                        title = title_element.text.strip()
                        value_element = li.find_element(By.CSS_SELECTOR, 'div.requisite-item__value > div') # Извлекаем значение поля
                        value = value_element.text.strip()

                        bank_info[title] = value

                    except Exception as e:
                        print(f"Ошибка при парсинге элемента <li>")
            else:
                print("Не удалось найти элемент ul, пропуск парсинга информации о банке")
                continue  # Перейти к следующей ссылке, если ul не найден

        except Exception as e:
            print(f"Ошибка при парсинге страницы {link}")

        all_bank_data_1.append(bank_info)  # Сохраняем информацию о банке в список

    df_1 = pd.DataFrame(all_bank_data_1)
    df_1['Ссылка на банк'] = bank_links[:len(all_bank_data_1)]
    cols = df_1.columns.tolist()
    cols = ['Ссылка на банк'] + [col for col in cols if col != 'Ссылка на банк']
    df_1 = df_1[cols]

    # df_1.to_csv('bank_data.csv', index=False, encoding='utf-8-sig')

except Exception as e:
    print(f"Произошла общая ошибка")

finally:
    driver.quit()

Переходим по ссылке: https://www.asv.org.ru/banks/1bank-akb-pao
Переходим по ссылке: https://www.asv.org.ru/banks/ao-avtogradbank
Переходим по ссылке: https://www.asv.org.ru/banks/agroinkombank-pao
Переходим по ссылке: https://www.asv.org.ru/banks/agrosoyuz-kb-ooo
Переходим по ссылке: https://www.asv.org.ru/banks/agrokhimbank-oao
Переходим по ссылке: https://www.asv.org.ru/banks/adam-interneshnl-akb-ao
Переходим по ссылке: https://www.asv.org.ru/banks/admiralteyskiy-kb-ooo
Переходим по ссылке: https://www.asv.org.ru/banks/azimut-akb-pao
Переходим по ссылке: https://www.asv.org.ru/banks/ayvi-bank-ao
Переходим по ссылке: https://www.asv.org.ru/banks/aymanibank-kb-ooo
Переходим по ссылке: https://www.asv.org.ru/banks/ak-bank-ao
Переходим по ссылке: https://www.asv.org.ru/banks/aksonbank-kb-ooo
Переходим по ссылке: https://www.asv.org.ru/banks/akb-aktiv-bank-pao
Переходим по ссылке: https://www.asv.org.ru/banks/aktsent-akb-pao
Переходим по ссылке: https://www.asv.org.ru/banks/aktsiya-akb-o

In [45]:
df_1

,Ссылка на банк,Полное фирменное наименование,Сокращенное фирменное наименование,Место нахождения,Почтовый адрес,ОГРН,Номер в Книге государственной регистрации кредитных организаций,Номер по реестру,Дата включения в реестр,Статусы,Дата исключения из реестра,Основание для исключения из реестра,,Инвестор,Примечание
0,https://www.asv.org.ru/banks/1bank-akb-pao,"Акционерный коммерческий банк ""1Банк"" (публичн...","ПАО АКБ ""1Банк""","362040, Республика Северная Осетия-Алания, г.В...","362001, г. Владикавказ, ул. Гагарина, д. 30",1021500000147,2896,985,20.10.2010,Лицензия отозвана 17 марта 2016\nУчастник ССВ,NaN,NaN,NaN,NaN,NaN
1,https://www.asv.org.ru/banks/ao-avtogradbank,Акционерное общество «Автоградбанк»,АО «Автоградбанк»,"423831, Республика Татарстан (Татарстан), г. Н...","109992, Москва, «Москва ГСП-2» – ликвидатор АО...",1021600000806,1455,5,21.09.2004,Лицензия отозвана 17 июня 2024\nУчастник ССВ,NaN,NaN,NaN,NaN,NaN
2,https://www.asv.org.ru/banks/agroinkombank-pao,Публичное акционерное общество Агроинвестицион...,ПАО АГРОИНКОМБАНК,"414000, город Астрахань, пл. Ленина/ул. Бурова...","109992, г. Москва, ГСП-2, ГК «АСВ» – конкурсны...",1023000927520,1946,22,21.09.2004,Лицензия отозвана 2 ноября 2015\nУчастник ССВ,NaN,NaN,NaN,NaN,NaN
3,https://www.asv.org.ru/banks/agrosoyuz-kb-ooo,Общество с ограниченной ответственностью Комме...,"ООО КБ ""Агросоюз""","101000, г. Москва, Уланский пер., д. 13, стр. 1","109992, г. Москва, ГСП-2, ГК «АСВ» – конкурсны...",1025600001130,1459,649,17.02.2005,Лицензия отозвана 7 ноября 2018\nУчастник ССВ,NaN,NaN,NaN,NaN,NaN
4,https://www.asv.org.ru/banks/agrokhimbank-oao,Банк развития аграрной и химической промышленн...,"ОАО ""Агрохимбанк""","119992, г. Москва, Большой Знаменский пер., д....",NaN,1027739282317,119,468,20.01.2005,Лицензия отозвана 30 декабря 2008,13.01.2016,Отзыв (аннулирование) лицензии и завершение пр...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,https://www.asv.org.ru/banks/master-bank-kb-oao,"Коммерческий банк ""Мастер-Банк"" (Открытое акци...","""Мастер-Банк"" (ОАО)","115184, г. Москва, Руновский пер., д.12","109992, г. Москва, ГСП-2, ГК «АСВ» – конкурсны...",1027739049304,2176,307,16.12.2004,Лицензия отозвана 20 ноября 2013\nУчастник ССВ,NaN,NaN,NaN,NaN,NaN
230,https://www.asv.org.ru/banks/master-kapital-ak...,"Акционерный Коммерческий Банк ""Мастер-Капитал""...","АКБ ""Мастер-Капитал"" (ОАО)","105120, г. Москва, ул.Нижняя Сыромятническая, ...","109992, г. Москва, ГСП-2, ГК «АСВ» – конкурсны...",1027739338175,3011,776,14.03.2005,Лицензия отозвана 20 февраля 2018\nУчастник ССВ,NaN,NaN,NaN,NaN,NaN
231,https://www.asv.org.ru/banks/mbsp-bank-ao,Международный банк Санкт-Петербурга (Акционерн...,Банк МБСП (АО),"194044, Санкт-Петербург, Крапивный пер., 5","199106, г. Санкт-Петербург, пр-кт Средний В.О....",1027800001547,197,331,16.12.2004,Лицензия отозвана 31 октября 2018\nУчастник ССВ,NaN,NaN,NaN,NaN,NaN
232,https://www.asv.org.ru/banks/mbfi-bank-oao,"ОТКРЫТОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ""МЕЖДУНАРОДНЫЙ Б...","ОАО БАНК ""МБФИ""","105062, г. Москва, Подсосенский пер, д. 17","109992, г. Москва, ГСП-2, ГК «АСВ» – конкурсны...",1020500000510,2073,987,28.12.2010,Лицензия отозвана 27 апреля 2016\nУчастник ССВ,NaN,NaN,NaN,NaN,NaN


In [46]:
df_1.to_csv('bank_data_1.csv', index=False, encoding='utf-8-sig')

In [48]:
len(df_1) == len(bank_links_1)

True

In [49]:
bank_links_2 = ['https://www.asv.org.ru/banks/mvkb-kb-oao',
 'https://www.asv.org.ru/banks/mgb-rnko-ooo',
 'https://www.asv.org.ru/banks/mgmb-ooo',
 'https://www.asv.org.ru/banks/mega-bank-zao',
 'https://www.asv.org.ru/banks/ooo-kb-megapolis',
 'https://www.asv.org.ru/banks/medinvestbank-ooo',
 'https://www.asv.org.ru/banks/mezhdunarodnyy-bank-razvitiya-kb-ao',
 'https://www.asv.org.ru/banks/mezhdunarodnyy-torgovo-promyshlennyy-bank-oao',
 'https://www.asv.org.ru/banks/mezhregionalnyy-investitsionnyy-bank-akb-zao',
 'https://www.asv.org.ru/banks/mezhregionalnyy-kliringovyy-bank-kb-ooo',
 'https://www.asv.org.ru/banks/kb-mezhregionalnyy-pochtovyy-bank-ooo',
 'https://www.asv.org.ru/banks/mezhtopenergobank-pao',
 'https://www.asv.org.ru/banks/mezhtrastbank-kb-ooo',
 'https://www.asv.org.ru/banks/metrobank-ao',
 'https://www.asv.org.ru/banks/metropol-kb-ooo',
 'https://www.asv.org.ru/banks/mi-bank-oao',
 'https://www.asv.org.ru/banks/migros-kb-oao',
 'https://www.asv.org.ru/banks/miko-bank-kb-ooo',
 'https://www.asv.org.ru/banks/milbank-kb-ooo',
 'https://www.asv.org.ru/banks/millenium-bank-zao',
 'https://www.asv.org.ru/banks/mira-bank-zao',
 'https://www.asv.org.ru/banks/miraf-bank-ao',
 'https://www.asv.org.ru/banks/mir-akb-ao',
 'https://www.asv.org.ru/banks/ao-kb-mikhaylovskiy-pzhsb',
 'https://www.asv.org.ru/banks/kb-mkb-pao',
 'https://www.asv.org.ru/banks/mnib-kb-ooo',
 'https://www.asv.org.ru/banks/mobilbank-oao',
 'https://www.asv.org.ru/banks/moy-bank-ooo',
 'https://www.asv.org.ru/banks/monetnyy-dom-oao',
 'https://www.asv.org.ru/banks/monolit-kb-ooo',
 'https://www.asv.org.ru/banks/moskva-kb-pao',
 'https://www.asv.org.ru/banks/moskovskiy-vekselnyy-bank-akb-ao',
 'https://www.asv.org.ru/banks/moskovskiy-kapital-kb-ooo',
 'https://www.asv.org.ru/banks/mosoblinvestbank-kb-ooo',
 'https://www.asv.org.ru/banks/mostransbank-akb-oao',
 'https://www.asv.org.ru/banks/mosuralbank-akb-ao',
 'https://www.asv.org.ru/banks/mrb-kb-ooo',
 'https://www.asv.org.ru/banks/mezhdunarodnyy-solidarnyy-bank',
 'https://www.asv.org.ru/banks/multibank-akb-oao',
 'https://www.asv.org.ru/banks/munitsipalnyy-kommercheskiy-bank-oao',
 'https://www.asv.org.ru/banks/mfbank-kb-ooo',
 'https://www.asv.org.ru/banks/mft-bank-akb-oao',
 'https://www.asv.org.ru/banks/navigator-bank-oao',
 'https://www.asv.org.ru/banks/nadezhnost-zaskb-oao',
 'https://www.asv.org.ru/banks/ao-narodnyy-bank',
 'https://www.asv.org.ru/banks/narodnyy-kredit-bank-oao',
 'https://www.asv.org.ru/banks/naftabank-kb-ooo',
 'https://www.asv.org.ru/banks/natskorpbank-ao',
 'https://www.asv.org.ru/banks/natsprombank-kb-zao',
 'https://www.asv.org.ru/banks/nash-bank-oao',
 'https://www.asv.org.ru/banks/nbb-oao',
 'https://www.asv.org.ru/banks/nezavisimyy-bank-razvitiya',
 'https://www.asv.org.ru/banks/ao-bank-nbt',
 'https://www.asv.org.ru/banks/nvkbank-ao',
 'https://www.asv.org.ru/banks/ooo-kb-nevastroyinvest',
 'https://www.asv.org.ru/banks/nevskiy-bank-pao',
 'https://www.asv.org.ru/banks/bank-neyva-ooo',
 'https://www.asv.org.ru/banks/neopolis-bank-kb-ooo',
 'https://www.asv.org.ru/banks/ao-nefteprombank',
 'https://www.asv.org.ru/banks/neftyanoy-alyans-kb-pao',
 'https://www.asv.org.ru/banks/neftyanoy-kb-ooo',
 'https://www.asv.org.ru/banks/nzbank-akb-oao',
 'https://www.asv.org.ru/banks/nkb-kb-ooo',
 'https://www.asv.org.ru/banks/novokuznetskiy-munitsipalnyy-bank-akb-oao',
 'https://www.asv.org.ru/banks/novatsiya-akb-pao',
 'https://www.asv.org.ru/banks/kb-novoe-vremya-ooo',
 'https://www.asv.org.ru/banks/novopokrovskiy-kb-ooo',
 'https://www.asv.org.ru/banks/obshchestvo-s-ogranichennoy-otvetstvennostyu-kommercheskiy-bank-novosibkoopbank',
 'https://www.asv.org.ru/banks/novyy-kreditnyy-soyuz-akb-ao',
 'https://www.asv.org.ru/banks/novyy-simvol-bank-ao',
 'https://www.asv.org.ru/banks/ao-nordea-bank',
 'https://www.asv.org.ru/banks/nota-bank-pao',
 'https://www.asv.org.ru/banks/noyabrskneftekombank',
 'https://www.asv.org.ru/banks/nst-bank-ao',
 'https://www.asv.org.ru/banks/neklis-bank-kb-ooo',
 'https://www.asv.org.ru/banks/o-k-bank-pao',
 'https://www.asv.org.ru/banks/obpi-akb-pao',
 'https://www.asv.org.ru/banks/kb-obr-ooo',
 'https://www.asv.org.ru/banks/obrazovanie-akib-ao',
 'https://www.asv.org.ru/banks/obedinennyy-gornyy-bank-mkb-ooo',
 'https://www.asv.org.ru/banks/obedinennyy-natsionalnyy-bank-ooo',
 'https://www.asv.org.ru/banks/ogni-moskvy-kb-ooo',
 'https://www.asv.org.ru/banks/odinbank-mkb-ooo',
 'https://www.asv.org.ru/banks/okean-bank-ao',
 'https://www.asv.org.ru/banks/olma-bank-ikb-ooo',
 'https://www.asv.org.ru/banks/ao-bank-onego',
 'https://www.asv.org.ru/banks/opm-bank-kb-ooo',
 'https://www.asv.org.ru/banks/ao-orbank',
 'https://www.asv.org.ru/banks/mbo-orgbank-ooo',
 'https://www.asv.org.ru/banks/orelsotsbank-oao',
 'https://www.asv.org.ru/banks/oskolbank-ooo',
 'https://www.asv.org.ru/banks/ofk-bank-pao',
 'https://www.asv.org.ru/banks/okhotnyy-ryad-kb-ooo',
 'https://www.asv.org.ru/banks/paritet-nko-ooo',
 'https://www.asv.org.ru/banks/partnerbank-kb-ooo',
 'https://www.asv.org.ru/banks/partnerkapitalbank-ao',
 'https://www.asv.org.ru/banks/pv-bank-zao',
 'https://www.asv.org.ru/banks/pervomayskiy-bank-pao',
 'https://www.asv.org.ru/banks/pervyy-depozitnyy-kb-ooo',
 'https://www.asv.org.ru/banks/pervyy-kapital-akb-zao',
 'https://www.asv.org.ru/banks/pervyy-ekspress-kb-oao',
 'https://www.asv.org.ru/banks/peresvet-akb-pao-prezhnee-naimenovanie-peresvet-akb-ao-',
 'https://www.asv.org.ru/banks/petro-aero-bank-kb-oao',
 'https://www.asv.org.ru/banks/piko-bank-oao',
 'https://www.asv.org.ru/banks/pir-bank-ooo',
 'https://www.asv.org.ru/banks/pk-bank-ooo',
 'https://www.asv.org.ru/banks/ooo-kb-platina',
 'https://www.asv.org.ru/banks/plato-bank-ooo',
 'https://www.asv.org.ru/banks/povolzhskiy-nemetskiy-bank-zao',
 'https://www.asv.org.ru/banks/povolzhskoe-obshchestvo-vzaimnogo-kredita',
 'https://www.asv.org.ru/banks/praym-finans-bank-ao',
 'https://www.asv.org.ru/banks/prb-oao',
 'https://www.asv.org.ru/banks/premer-makb-zao',
 'https://www.asv.org.ru/banks/preodolenie-kb-ooo',
 'https://www.asv.org.ru/banks/prestizhkreditbank-kb-ooo',
 'https://www.asv.org.ru/banks/privolzhskoe-obshchestvo-vzaimnogo-kredita',
 'https://www.asv.org.ru/banks/prikame-kombank-oao',
 'https://www.asv.org.ru/banks/printbank-oao',
 'https://www.asv.org.ru/banks/prioritet-bank-oao',
 'https://www.asv.org.ru/banks/prisko-kapital-bank-kb-ao',
 'https://www.asv.org.ru/banks/probank-ao',
 'https://www.asv.org.ru/banks/probiznesbank-akb-oao',
 'https://www.asv.org.ru/banks/akb-proinvestbank-pao',
 'https://www.asv.org.ru/banks/prombank-kb-oao',
 'https://www.asv.org.ru/banks/promkreditbank-kb-ooo',
 'https://www.asv.org.ru/banks/promregionbank-ooo',
 'https://www.asv.org.ru/banks/promsberbank-zao',
 'https://www.asv.org.ru/banks/promenergobank-ao',
 'https://www.asv.org.ru/banks/bank-prokhladnyy-ooo',
 'https://www.asv.org.ru/banks/psb-bank-oao',
 'https://www.asv.org.ru/banks/bank-ptb-ooo',
 'https://www.asv.org.ru/banks/puls-stolitsy-kb-ooo',
 'https://www.asv.org.ru/banks/purpe-bank-oao',
 'https://www.asv.org.ru/banks/pushkino-ab-oao',
 'https://www.asv.org.ru/banks/pao-kb-pfs-bank',
 'https://www.asv.org.ru/banks/pchrb-bank-ooo',
 'https://www.asv.org.ru/banks/radian-akb-oao',
 'https://www.asv.org.ru/banks/radiotekhbank-nkb-pao',
 'https://www.asv.org.ru/banks/razvitie-kb-ooo',
 'https://www.asv.org.ru/banks/ram-bank-ooo',
 'https://www.asv.org.ru/banks/ratibor-bank-kb-ooo',
 'https://www.asv.org.ru/banks/rbr-akb-pao',
 'https://www.asv.org.ru/banks/rbs-kb-ooo',
 'https://www.asv.org.ru/banks/regnum-bank-ooo',
 'https://www.asv.org.ru/banks/rezerv-bank-ao',
 'https://www.asv.org.ru/banks/renessans-kb-ooo',
 'https://www.asv.org.ru/banks/riabank-ao',
 'https://www.asv.org.ru/banks/rosinbank-sibir',
 'https://www.asv.org.ru/banks/rinvestbank-rikb-ooo',
 'https://www.asv.org.ru/banks/ringkombank-kb-ooo',
 'https://www.asv.org.ru/banks/rink-bank-ab-oao',
 'https://www.asv.org.ru/banks/rkb-ooo',
 'https://www.asv.org.ru/banks/rkb-bank-ao',
 'https://www.asv.org.ru/banks/rmb-bank-ao',
 'https://www.asv.org.ru/banks/rosavtobank-kb-ooo',
 'https://www.asv.org.ru/banks/pao-arkb-rosbiznesbank',
 'https://www.asv.org.ru/banks/rosinterbank-kb-ao',
 'https://www.asv.org.ru/banks/roskomveteranbank-kb-ooo',
 'https://www.asv.org.ru/banks/roskomsnabbank-pao',
 'https://www.asv.org.ru/banks/rosprombank-kb-ooo',
 'https://www.asv.org.ru/banks/rsb-24-bank-ao',
 'https://www.asv.org.ru/banks/rskb-ao',
 'https://www.asv.org.ru/banks/rtbk-kb-ooo',
 'https://www.asv.org.ru/banks/rts-bank-ao',
 'https://www.asv.org.ru/banks/rubank-kb-ao',
 'https://www.asv.org.ru/banks/rubin-kb-ooo',
 'https://www.asv.org.ru/banks/rublev-kb-ao',
 'https://www.asv.org.ru/banks/ao-runa-bank',
 'https://www.asv.org.ru/banks/ao-runetbank',
 'https://www.asv.org.ru/banks/rusich-tsentr-bank-oao',
 'https://www.asv.org.ru/banks/ruskobank-ao',
 'https://www.asv.org.ru/banks/russkaya-investitsionnaya-gruppa-kb-oao',
 'https://www.asv.org.ru/banks/russkiy-zemelnyy-bank-akb-oao',
 'https://www.asv.org.ru/banks/russkiy-ipotechnyy-bank-kb-ooo',
 'https://www.asv.org.ru/banks/russkiy-natsionalnyy-bank-ooo',
 'https://www.asv.org.ru/banks/russkiy-trastovyy-bank-akb-ao',
 'https://www.asv.org.ru/banks/russobank-akb-ao',
 'https://www.asv.org.ru/banks/russtroybank-ao',
 'https://www.asv.org.ru/banks/ao-rfi-bank',
 'https://www.asv.org.ru/banks/rybkhozbank-zao',
 'https://www.asv.org.ru/banks/reb-kb-ao',
 'https://www.asv.org.ru/banks/s-bank-zao',
 'https://www.asv.org.ru/banks/s-bank-akb-zao',
 'https://www.asv.org.ru/banks/sakhalin-vest-mabes-oao',
 'https://www.asv.org.ru/banks/sb-bank-ooo',
 'https://www.asv.org.ru/banks/sberinvestbank-zao',
 'https://www.asv.org.ru/banks/sbrr-bank-ooo',
 'https://www.asv.org.ru/banks/svedbank',
 'https://www.asv.org.ru/banks/svyaznoy-bank-ao',
 'https://www.asv.org.ru/banks/sakhadaymondbank-kb-ooo',
 'https://www.asv.org.ru/banks/severinvestbank-oao',
 'https://www.asv.org.ru/banks/severnyy-kredit-kb-ao',
 'https://www.asv.org.ru/banks/severo-zapadnyy-1-alyans-bank-akb-ao',
 'https://www.asv.org.ru/banks/sembank-kb-ooo',
 'https://www.asv.org.ru/banks/setevoy-neftyanoy-bank-ab-zao',
 'https://www.asv.org.ru/banks/ao-si-di-bi-bank',
 'https://www.asv.org.ru/banks/sibirgazbank',
 'https://www.asv.org.ru/banks/sibkontakt-kb-oao',
 'https://www.asv.org.ru/banks/sibneftebank-oao',
 'https://www.asv.org.ru/banks/sibes-bank-ao',
 'https://www.asv.org.ru/banks/simbirsk-gb-oao',
 'https://www.asv.org.ru/banks/ao-rnko-sinergiya',
 'https://www.asv.org.ru/banks/sir-ab-oao',
 'https://www.asv.org.ru/banks/slavyanskiy-bank-akb-zao',
 'https://www.asv.org.ru/banks/ooo-kb-slavyanskiy-kredit',
 'https://www.asv.org.ru/banks/smartbank-ao',
 'https://www.asv.org.ru/banks/smkb-kb-ooo',
 'https://www.asv.org.ru/banks/smolevich-bank-pao',
 'https://www.asv.org.ru/banks/smolenskiy-bank-oao',
 'https://www.asv.org.ru/banks/soveren-bank-ao',
 'https://www.asv.org.ru/banks/sovetskiy-bank-ao',
 'https://www.asv.org.ru/banks/sovinkom-kb-ooo',
 'https://www.asv.org.ru/banks/sodruzhestvo-bank-oao',
 'https://www.asv.org.ru/banks/solidarnost-bank-ao',
 'https://www.asv.org.ru/banks/sootechestvenniki-bank-oao',
 'https://www.asv.org.ru/banks/sofrino-kb-zao',
 'https://www.asv.org.ru/banks/sotsgorbank-kb-oao',
 'https://www.asv.org.ru/banks/sotskreditbank-kb-ooo',
 'https://www.asv.org.ru/banks/sochi-kiib-ooo',
 'https://www.asv.org.ru/banks/soyuznyy-kb-ooo',
 'https://www.asv.org.ru/banks/spbrr-kb-oao',
 'https://www.asv.org.ru/banks/spetssetstroybank-kb-ooo',
 'https://www.asv.org.ru/banks/spurt-akb-pao',
 'https://www.asv.org.ru/banks/kb-sputnik-pao',
 'https://www.asv.org.ru/banks/kb-sstb-ooo',
 'https://www.asv.org.ru/banks/stal-bank-ooo',
 'https://www.asv.org.ru/banks/starbank-ao',
 'https://www.asv.org.ru/banks/staryy-kreml-kb-ooo',
 'https://www.asv.org.ru/banks/stella-bank-akb-ao',
 'https://www.asv.org.ru/banks/strategiya-akb-pao',
 'https://www.asv.org.ru/banks/ao-sevzapinvestprombank',
 'https://www.asv.org.ru/banks/stroykombank-zao',
 'https://www.asv.org.ru/banks/stroykredit-kb-oao',
 'https://www.asv.org.ru/banks/sudkombank-ikb-ooo',
 'https://www.asv.org.ru/banks/sunzha-kb-ooo',
 'https://www.asv.org.ru/banks/surgutskiy-tsentralnyy-kb-ooo',
 'https://www.asv.org.ru/banks/taatta-bank-ao',
 'https://www.asv.org.ru/banks/tagilbank-ao',
 'https://www.asv.org.ru/banks/taym-bank-pao',
 'https://www.asv.org.ru/banks/talmenka-bank-kb-ooo',
 'https://www.asv.org.ru/banks/tatagroprombank-ooo',
 'https://www.asv.org.ru/banks/tatfondbank-pao',
 'https://www.asv.org.ru/banks/taurus-bank-ao',
 'https://www.asv.org.ru/banks/tgb-bank-ao',
 'https://www.asv.org.ru/banks/ao-teksbank',
 'https://www.asv.org.ru/banks/tempbank-mab-pao',
 'https://www.asv.org.ru/banks/tetrapolis-kb-ao',
 'https://www.asv.org.ru/banks/tikhookeanskiy-vneshtorgbank-oao',
 'https://www.asv.org.ru/banks/trado-bank-akb-zao',
 'https://www.asv.org.ru/banks/transinvestbank-kb-ooo',
 'https://www.asv.org.ru/banks/transnatsionalnyy-bank-kb-ooo',
 'https://www.asv.org.ru/banks/transportnyy-kb-ooo',
 'https://www.asv.org.ru/banks/transenergobank-ooo',
 'https://www.asv.org.ru/banks/trastovyy-respublikanskiy-bank-ooo',
 'https://www.asv.org.ru/banks/troyka-d-bank-ao',
 'https://www.asv.org.ru/banks/tsb-oao',
 'https://www.asv.org.ru/banks/tuvakredit-oao-nko',
 'https://www.asv.org.ru/banks/tulskiy-promyshlennik-bank-pao',
 'https://www.asv.org.ru/banks/turbobank-akb-ao',
 'https://www.asv.org.ru/banks/tusarbank-ao',
 'https://www.asv.org.ru/banks/ao-tembr-bank',
 'https://www.asv.org.ru/banks/tyumenagroprombank-zao',
 'https://www.asv.org.ru/banks/tyumenenergobank-oao',
 'https://www.asv.org.ru/banks/udmurtinveststroybank-oao',
 'https://www.asv.org.ru/banks/uik-bank-ooo',
 'https://www.asv.org.ru/banks/um-bank-ooo',
 'https://www.asv.org.ru/banks/univerbank-akb-oao',
 'https://www.asv.org.ru/banks/universal-kb-ooo',
 'https://www.asv.org.ru/banks/universalnyy-kredit-akb-oao',
 'https://www.asv.org.ru/banks/unifin-kb-ao',
 'https://www.asv.org.ru/banks/up-bank-oao',
 'https://www.asv.org.ru/banks/uraykombank-kb-ooo',
 'https://www.asv.org.ru/banks/uralkapitalbank-ooo',
 'https://www.asv.org.ru/banks/uralliga-kb-zao',
 'https://www.asv.org.ru/banks/uraltransbank-pao',
 'https://www.asv.org.ru/banks/uralfinprombank-oao',
 'https://www.asv.org.ru/banks/uralskiy-trastovyy-bank-oao',
 'https://www.asv.org.ru/banks/ussuri-bank-ao',
 'https://www.asv.org.ru/banks/femili-kb-ooo',
 'https://www.asv.org.ru/banks/fia-bank-ao',
 'https://www.asv.org.ru/banks/finansovyy-kapital-kb-ooo',
 'https://www.asv.org.ru/banks/finansovyy-standart-kb-ooo',
 'https://www.asv.org.ru/banks/finprombank-akb-pao',
 'https://www.asv.org.ru/banks/fkb-ooo',
 'https://www.asv.org.ru/banks/flora-moskva-kb-ao',
 'https://www.asv.org.ru/banks/fonon-kb-ooo',
 'https://www.asv.org.ru/banks/ao-kb-forbank',
 'https://www.asv.org.ru/banks/forus-bank-ao',
 'https://www.asv.org.ru/banks/fpk-kb-ooo',
 'https://www.asv.org.ru/banks/fundament-bank-kb-oao',
 'https://www.asv.org.ru/banks/khard-bank-kb-ooo',
 'https://www.asv.org.ru/banks/khlebobank-akb-zao',
 'https://www.asv.org.ru/banks/kholdinvestbank-kb-ooo',
 'https://www.asv.org.ru/banks/kholding-kredit-kb-ooo',
 'https://www.asv.org.ru/banks/tsentralno-evropeyskiy-bank-kb-ooo',
 'https://www.asv.org.ru/banks/tsentrkombank-ooo',
 'https://www.asv.org.ru/banks/tserikh-bank-zao',
 'https://www.asv.org.ru/banks/chuvashkreditprombank-akb-pao',
 'https://www.asv.org.ru/banks/ekonatsbank-zao',
 'https://www.asv.org.ru/banks/kb-ekonomiks-bank-ooo',
 'https://www.asv.org.ru/banks/ekoprombank-akb-oao',
 'https://www.asv.org.ru/banks/ekspert-bank-ao',
 'https://www.asv.org.ru/banks/ekspress-akb-oao',
 'https://www.asv.org.ru/banks/ekspress-kredit-kb-ao',
 'https://www.asv.org.ru/banks/el-bank-kb-ooo',
 'https://www.asv.org.ru/banks/elektronika-akb-oao',
 'https://www.asv.org.ru/banks/elbin-akb-ao',
 'https://www.asv.org.ru/banks/pao-energomashbank',
 'https://www.asv.org.ru/banks/eno-akb-pao',
 'https://www.asv.org.ru/banks/entuziastbank-ikb-ooo',
 'https://www.asv.org.ru/banks/ergobank-kb-ooo',
 'https://www.asv.org.ru/banks/esidbank-ooo',
 'https://www.asv.org.ru/banks/yugra-bank-pao',
 'https://www.asv.org.ru/banks/yuzhnyy-torgovyy-bank-ab-oao',
 'https://www.asv.org.ru/banks/yunikbank-akb-zao',
 'https://www.asv.org.ru/banks/yunikorbank-kb-oao',
 'https://www.asv.org.ru/banks/yunitbank-kb-zao',
 'https://www.asv.org.ru/banks/yurb-ooo',
 'https://www.asv.org.ru/banks/yaroslav-akb-zao']

In [50]:
"""Эта программа работает примерно 40 минут"""

chrome_options = Options()
driver = webdriver.Chrome(options=chrome_options)

try:
    bank_links = bank_links_2
    all_bank_data_2 = [] # Парсим данные с каждой страницы банка
    for link in bank_links:
        print(f"Переходим по ссылке: {link}")
        driver.get(link)
        time.sleep(5)

        bank_info = {}  # Создаем словарь для хранения информации о банке
        try:
            try:
                ul_element = WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, 'div.foldable__body > div > app-company-block-requisites-table > ul')) # В коде страницы этот элемент располагается в ветви с названием 'ul', поэтому нам надо найти все 'ul', чтобы найти все элементы
                )
            except Exception as e:
                ul_element = None
                print()

            if ul_element:
                li_elements = ul_element.find_elements(By.TAG_NAME, "li") # Находим все элементы <li> внутри <ul>

                for li in li_elements:
                    try:
                        
                        title_element = li.find_element(By.CSS_SELECTOR, 'div.requisite-item__title > div') # Извлекаем название поля
                        title = title_element.text.strip()
                        value_element = li.find_element(By.CSS_SELECTOR, 'div.requisite-item__value > div') # Извлекаем значение поля
                        value = value_element.text.strip()

                        bank_info[title] = value

                    except Exception as e:
                        print(f"Ошибка при парсинге элемента <li>")
            else:
                print("Не удалось найти элемент ul, пропуск парсинга информации о банке")
                continue  # Перейти к следующей ссылке, если ul не найден

        except Exception as e:
            print(f"Ошибка при парсинге страницы {link}")

        all_bank_data_2.append(bank_info)  # Сохраняем информацию о банке в список

    df_2 = pd.DataFrame(all_bank_data_2)
    df_2['Ссылка на банк'] = bank_links[:len(all_bank_data_2)]
    cols = df_2.columns.tolist()
    cols = ['Ссылка на банк'] + [col for col in cols if col != 'Ссылка на банк']
    df_2 = df_2[cols]

    # df_2.to_csv('bank_data.csv', index=False, encoding='utf-8-sig')

except Exception as e:
    print(f"Произошла общая ошибка")

finally:
    driver.quit()

Переходим по ссылке: https://www.asv.org.ru/banks/mvkb-kb-oao
Переходим по ссылке: https://www.asv.org.ru/banks/mgb-rnko-ooo
Переходим по ссылке: https://www.asv.org.ru/banks/mgmb-ooo
Переходим по ссылке: https://www.asv.org.ru/banks/mega-bank-zao
Переходим по ссылке: https://www.asv.org.ru/banks/ooo-kb-megapolis
Переходим по ссылке: https://www.asv.org.ru/banks/medinvestbank-ooo
Переходим по ссылке: https://www.asv.org.ru/banks/mezhdunarodnyy-bank-razvitiya-kb-ao
Переходим по ссылке: https://www.asv.org.ru/banks/mezhdunarodnyy-torgovo-promyshlennyy-bank-oao
Переходим по ссылке: https://www.asv.org.ru/banks/mezhregionalnyy-investitsionnyy-bank-akb-zao
Переходим по ссылке: https://www.asv.org.ru/banks/mezhregionalnyy-kliringovyy-bank-kb-ooo
Переходим по ссылке: https://www.asv.org.ru/banks/kb-mezhregionalnyy-pochtovyy-bank-ooo
Переходим по ссылке: https://www.asv.org.ru/banks/mezhtopenergobank-pao
Переходим по ссылке: https://www.asv.org.ru/banks/mezhtrastbank-kb-ooo
Переходим по ссылке

In [51]:
df_2

,Ссылка на банк,Полное фирменное наименование,Сокращенное фирменное наименование,Место нахождения,Почтовый адрес,ОГРН,Номер в Книге государственной регистрации кредитных организаций,Номер по реестру,Дата включения в реестр,Статусы,,Дата исключения из реестра,Основание для исключения из реестра,Адрес официального сайта в сети Интернет,Инвестор,Временная администрация
0,https://www.asv.org.ru/banks/mvkb-kb-oao,Открытое акционерное общество Коммерческий Бан...,"ОАО КБ ""МВКБ""","105062, г. Москва, Подсосенский переулок, д. 2...","109992, г. Москва, ГСП-2, ГК «АСВ» – конкурсны...",1027744003100,2863,413,11.01.2005,Лицензия отозвана 28 марта 2016\nУчастник ССВ,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.asv.org.ru/banks/mgb-rnko-ooo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN
2,https://www.asv.org.ru/banks/mgmb-ooo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN
3,https://www.asv.org.ru/banks/mega-bank-zao,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN
4,https://www.asv.org.ru/banks/ooo-kb-megapolis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,https://www.asv.org.ru/banks/khlebobank-akb-zao,Акционерный хлебный банк (закрытое акционерное...,Акционерный хлебный банк,"115093, г. Москва, ул. Щипок, д. 18",NaN,1027739649266,1812,821,21.03.2005,Лицензия отозвана 27 июля 2012,NaN,28.10.2013,Отзыв (аннулирование) лицензии и завершение пр...,NaN,NaN,NaN
292,https://www.asv.org.ru/banks/kholdinvestbank-k...,Общество с ограниченной ответственностью Комме...,"ООО КБ ""Холдинвестбанк""","143026, Московская область, Одинцовский район,...",NaN,1027739554743,2837,630,10.02.2005,Лицензия отозвана 26 апреля 2019,NaN,21.10.2020,Отзыв (аннулирование) лицензии и завершение пр...,NaN,NaN,NaN
293,https://www.asv.org.ru/banks/kholding-kredit-k...,"Коммерческий Банк ""Холдинг-Кредит"" (общество с...","КБ ""Холдинг-Кредит"" ООО","121069, г. Москва, ул. Садовая-Кудринская, д. ...","109992, г. Москва, ГСП-2, ГК «АСВ» – конкурсны...",1037739574773,3030,579,03.02.2005,Лицензия отозвана 16 мая 2012\nУчастник ССВ,NaN,NaN,NaN,NaN,NaN,NaN
294,https://www.asv.org.ru/banks/tsentralno-evrope...,АКЦИОНЕРНЫЙ НАЦИОНАЛЬНЫЙ БАНК ОХРАНЫ ОКРУЖАЮЩЕ...,ОХРАНЫ ОКРУЖАЮЩЕЙ СРЕДЫ,"109240, г. Москва, ул. Солянка, д.4, стр.1-1А",NaN,1027739505023,2765,854,11.08.2005,Лицензия отозвана 12 ноября 2008,NaN,01.10.2013,Отзыв (аннулирование) лицензии и завершение пр...,NaN,NaN,NaN


In [56]:
df_2.to_csv('bank_data_2.csv', index=False, encoding='utf-8-sig')

In [58]:
bank_links_3 = ['https://www.asv.org.ru/banks/tsentrkombank-ooo',
 'https://www.asv.org.ru/banks/tserikh-bank-zao',
 'https://www.asv.org.ru/banks/chuvashkreditprombank-akb-pao',
 'https://www.asv.org.ru/banks/ekonatsbank-zao',
 'https://www.asv.org.ru/banks/kb-ekonomiks-bank-ooo',
 'https://www.asv.org.ru/banks/ekoprombank-akb-oao',
 'https://www.asv.org.ru/banks/ekspert-bank-ao',
 'https://www.asv.org.ru/banks/ekspress-akb-oao',
 'https://www.asv.org.ru/banks/ekspress-kredit-kb-ao',
 'https://www.asv.org.ru/banks/el-bank-kb-ooo',
 'https://www.asv.org.ru/banks/elektronika-akb-oao',
 'https://www.asv.org.ru/banks/elbin-akb-ao',
 'https://www.asv.org.ru/banks/pao-energomashbank',
 'https://www.asv.org.ru/banks/eno-akb-pao',
 'https://www.asv.org.ru/banks/entuziastbank-ikb-ooo',
 'https://www.asv.org.ru/banks/ergobank-kb-ooo',
 'https://www.asv.org.ru/banks/esidbank-ooo',
 'https://www.asv.org.ru/banks/yugra-bank-pao',
 'https://www.asv.org.ru/banks/yuzhnyy-torgovyy-bank-ab-oao',
 'https://www.asv.org.ru/banks/yunikbank-akb-zao',
 'https://www.asv.org.ru/banks/yunikorbank-kb-oao',
 'https://www.asv.org.ru/banks/yunitbank-kb-zao',
 'https://www.asv.org.ru/banks/yurb-ooo',
 'https://www.asv.org.ru/banks/yaroslav-akb-zao']

In [59]:
"""Эта программа работает примерно 2 минуты"""

chrome_options = Options()
driver = webdriver.Chrome(options=chrome_options)

try:
    bank_links = bank_links_3
    all_bank_data_3 = [] # Парсим данные с каждой страницы банка
    for link in bank_links:
        print(f"Переходим по ссылке: {link}")
        driver.get(link)
        time.sleep(5)

        bank_info = {}  # Создаем словарь для хранения информации о банке
        try:
            try:
                ul_element = WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, 'div.foldable__body > div > app-company-block-requisites-table > ul')) # В коде страницы этот элемент располагается в ветви с названием 'ul', поэтому нам надо найти все 'ul', чтобы найти все элементы
                )
            except Exception as e:
                ul_element = None
                print()

            if ul_element:
                li_elements = ul_element.find_elements(By.TAG_NAME, "li") # Находим все элементы <li> внутри <ul>

                for li in li_elements:
                    try:
                        
                        title_element = li.find_element(By.CSS_SELECTOR, 'div.requisite-item__title > div') # Извлекаем название поля
                        title = title_element.text.strip()
                        value_element = li.find_element(By.CSS_SELECTOR, 'div.requisite-item__value > div') # Извлекаем значение поля
                        value = value_element.text.strip()

                        bank_info[title] = value

                    except Exception as e:
                        print(f"Ошибка при парсинге элемента <li>")
            else:
                print("Не удалось найти элемент ul, пропуск парсинга информации о банке")
                continue  # Перейти к следующей ссылке, если ul не найден

        except Exception as e:
            print(f"Ошибка при парсинге страницы {link}")

        all_bank_data_3.append(bank_info)  # Сохраняем информацию о банке в список

    df_3 = pd.DataFrame(all_bank_data_3)
    df_3['Ссылка на банк'] = bank_links[:len(all_bank_data_3)]
    cols = df_3.columns.tolist()
    cols = ['Ссылка на банк'] + [col for col in cols if col != 'Ссылка на банк']
    df_3 = df_3[cols]

except Exception as e:
    print(f"Произошла общая ошибка")

finally:
    driver.quit()

Переходим по ссылке: https://www.asv.org.ru/banks/tsentrkombank-ooo
Переходим по ссылке: https://www.asv.org.ru/banks/tserikh-bank-zao
Переходим по ссылке: https://www.asv.org.ru/banks/chuvashkreditprombank-akb-pao
Переходим по ссылке: https://www.asv.org.ru/banks/ekonatsbank-zao
Переходим по ссылке: https://www.asv.org.ru/banks/kb-ekonomiks-bank-ooo
Переходим по ссылке: https://www.asv.org.ru/banks/ekoprombank-akb-oao
Переходим по ссылке: https://www.asv.org.ru/banks/ekspert-bank-ao
Переходим по ссылке: https://www.asv.org.ru/banks/ekspress-akb-oao
Переходим по ссылке: https://www.asv.org.ru/banks/ekspress-kredit-kb-ao
Переходим по ссылке: https://www.asv.org.ru/banks/el-bank-kb-ooo
Переходим по ссылке: https://www.asv.org.ru/banks/elektronika-akb-oao
Переходим по ссылке: https://www.asv.org.ru/banks/elbin-akb-ao
Переходим по ссылке: https://www.asv.org.ru/banks/pao-energomashbank
Переходим по ссылке: https://www.asv.org.ru/banks/eno-akb-pao
Переходим по ссылке: https://www.asv.org.ru

In [60]:
df_3

,Ссылка на банк,Полное фирменное наименование,Сокращенное фирменное наименование,Место нахождения,Почтовый адрес,ОГРН,Номер в Книге государственной регистрации кредитных организаций,Номер по реестру,Дата включения в реестр,Статусы,Дата исключения из реестра,Основание для исключения из реестра
0,https://www.asv.org.ru/banks/tsentrkombank-ooo,Центральный коммерческий банк общество с огран...,Центркомбанк ООО,"115054, г. Москва, 3-й Монетчиковский пер., д....","109992, г. Москва, ГСП-2, ГК «АСВ» – конкурсны...",1027739019527,2983,775,14.03.2005,Лицензия отозвана 26 сентября 2016\nУчастник ССВ,NaN,NaN
1,https://www.asv.org.ru/banks/tserikh-bank-zao,"Банк ""ЦЕРИХ"" (закрытое акционерное общество)","Банк ""ЦЕРИХ"" (ЗАО)","302030, г. Орёл, ул. Московская, 29","109992, г. Москва, ГСП-2, ГК «АСВ» – конкурсны...",1025700000578,3278,341,16.12.2004,Лицензия отозвана 26 февраля 2016\nУчастник ССВ,NaN,NaN
2,https://www.asv.org.ru/banks/chuvashkreditprom...,"АКЦИОНЕРНЫЙ КОММЕРЧЕСКИЙ БАНК ""ЧУВАШКРЕДИТПРОМ...","АКБ ""ЧУВАШКРЕДИТПРОМБАНК"" ПАО","428018, Чувашская Республика, г.Чебоксары, Про...","109992, г. Москва, ГСП-2, ГК «АСВ» – конкурсны...",1022100000064,1280,354,23.12.2004,Лицензия отозвана 7 ноября 2019\nУчастник ССВ,NaN,NaN
3,https://www.asv.org.ru/banks/ekonatsbank-zao,АКЦИОНЕРНЫЙ НАЦИОНАЛЬНЫЙ БАНК ОХРАНЫ ОКРУЖАЮЩЕ...,ОХРАНЫ ОКРУЖАЮЩЕЙ СРЕДЫ,"109240, г. Москва, ул. Солянка, д.4, стр.1-1А",NaN,1027739505023,2765,854,11.08.2005,Лицензия отозвана 12 ноября 2008,01.10.2013,Отзыв (аннулирование) лицензии и завершение пр...
4,https://www.asv.org.ru/banks/kb-ekonomiks-bank...,"Коммерческий банк ""Экономикс-Банк"" (общество с...","КБ ""Экономикс-Банк"" (ООО)","101000, г. Москва, ул. Мясницкая, дом 46, стр.1",NaN,1027700523718,2136,541,27.01.2005,Лицензия аннулирована 6 декабря 2018,21.08.2019,Отзыв (аннулирование) лицензии и завершение пр...
5,https://www.asv.org.ru/banks/ekoprombank-akb-oao,"ОТКРЫТОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ""ПЕРМСКИЙ АКЦИОН...","ОАО АКБ ""ЭКОПРОМБАНК""","614000, г. Пермь, ул. Екатерининская, 120","614015, г. Пермь, ул. Монастырская, д. 57, оф....",1025900003854,2011,522,27.01.2005,Лицензия отозвана 18 августа 2014\nУчастник ССВ,NaN,NaN
6,https://www.asv.org.ru/banks/ekspert-bank-ao,"Акционерное общество ""Эксперт Банк""","АО ""Эксперт Банк""","644046, г.Омск, ул. Маршала Жукова, дом № 107","109992, г. Москва, ГСП-2, ГК «АСВ» – конкурсны...",1025500001328,2949,277,09.12.2004,Лицензия отозвана 1 ноября 2019\nУчастник ССВ,NaN,NaN
7,https://www.asv.org.ru/banks/ekspress-akb-oao,"Акционерный коммерческий банк ""ЭКСПРЕСС"" (Откр...","ОАО АКБ ""ЭКСПРЕСС""","367000, Республика Дагестан, г. Махачкала, ул....","367009, г. Махачкала, ул. Каммаева, д. 19ж",1020500000619,3084,114,28.10.2004,Лицензия отозвана 21 января 2013\nУчастник ССВ,NaN,NaN
8,https://www.asv.org.ru/banks/ekspress-kredit-k...,"Коммерческий банк ""Экспресс-кредит"" (акционерн...","КБ ""Экспресс-кредит"" (АО)","105037, г. Москва, ул.3-я Прядильная, д.3","109992, г. Москва, ГСП-2, ГК «АСВ» – конкурсны...",1027739290270,210,973,26.11.2008,Лицензия отозвана 16 ноября 2016\nУчастник ССВ,NaN,NaN
9,https://www.asv.org.ru/banks/el-bank-kb-ooo,Общество с ограниченной ответственностью комме...,"ООО КБ ""Эл банк""","445037, Самарская область, г. Тольятти, Новый ...","109992, г. Москва, ГСП-2, ГК «АСВ» – конкурсны...",1020400000070,1025,503,27.01.2005,Лицензия отозвана 5 мая 2016\nУчастник ССВ,NaN,NaN


In [61]:
df_3.to_csv('bank_data_3.csv', index=False, encoding='utf-8-sig')

In [63]:
import pandas as pd
combined_df = pd.concat([df_1, df_2, df_3], axis=0)
combined_df = combined_df.reset_index(drop=True)
combined_df

,Ссылка на банк,Полное фирменное наименование,Сокращенное фирменное наименование,Место нахождения,Почтовый адрес,ОГРН,Номер в Книге государственной регистрации кредитных организаций,Номер по реестру,Дата включения в реестр,Статусы,Дата исключения из реестра,Основание для исключения из реестра,,Инвестор,Примечание,Адрес официального сайта в сети Интернет,Временная администрация
0,https://www.asv.org.ru/banks/1bank-akb-pao,"Акционерный коммерческий банк ""1Банк"" (публичн...","ПАО АКБ ""1Банк""","362040, Республика Северная Осетия-Алания, г.В...","362001, г. Владикавказ, ул. Гагарина, д. 30",1021500000147,2896,985,20.10.2010,Лицензия отозвана 17 марта 2016\nУчастник ССВ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.asv.org.ru/banks/ao-avtogradbank,Акционерное общество «Автоградбанк»,АО «Автоградбанк»,"423831, Республика Татарстан (Татарстан), г. Н...","109992, Москва, «Москва ГСП-2» – ликвидатор АО...",1021600000806,1455,5,21.09.2004,Лицензия отозвана 17 июня 2024\nУчастник ССВ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.asv.org.ru/banks/agroinkombank-pao,Публичное акционерное общество Агроинвестицион...,ПАО АГРОИНКОМБАНК,"414000, город Астрахань, пл. Ленина/ул. Бурова...","109992, г. Москва, ГСП-2, ГК «АСВ» – конкурсны...",1023000927520,1946,22,21.09.2004,Лицензия отозвана 2 ноября 2015\nУчастник ССВ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.asv.org.ru/banks/agrosoyuz-kb-ooo,Общество с ограниченной ответственностью Комме...,"ООО КБ ""Агросоюз""","101000, г. Москва, Уланский пер., д. 13, стр. 1","109992, г. Москва, ГСП-2, ГК «АСВ» – конкурсны...",1025600001130,1459,649,17.02.2005,Лицензия отозвана 7 ноября 2018\nУчастник ССВ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://www.asv.org.ru/banks/agrokhimbank-oao,Банк развития аграрной и химической промышленн...,"ОАО ""Агрохимбанк""","119992, г. Москва, Большой Знаменский пер., д....",NaN,1027739282317,119,468,20.01.2005,Лицензия отозвана 30 декабря 2008,13.01.2016,Отзыв (аннулирование) лицензии и завершение пр...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
549,https://www.asv.org.ru/banks/yunikbank-akb-zao,Акционерный коммерческий банк Юникбанк (закрыт...,Юникбанк (закрытое акционерное общество),"115054, г. Москва, ул. Дубининская, д.35",NaN,1027700073884,2336,751,03.03.2005,Лицензия отозвана 23 апреля 2009,12.11.2012,Отзыв (аннулирование) лицензии и завершение пр...,NaN,NaN,NaN,NaN,NaN
550,https://www.asv.org.ru/banks/yunikorbank-kb-oao,"Коммерческий банк ""ЮНИКОР"" (открытое акционерн...","ОАО ""ЮНИКОРБАНК""","129515, г. Москва, ул. 1-я Останкинская, д. 26","109992, г. Москва, ГСП-2, ГК «АСВ» – конкурсны...",1027700035835,2586,701,24.02.2005,Лицензия отозвана 25 июля 2014\nУчастник ССВ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
551,https://www.asv.org.ru/banks/yunitbank-kb-zao,"КОММЕРЧЕСКИЙ БАНК ""ЮНИТБАНК"" (ЗАКРЫТОЕ АКЦИОНЕ...",ЮНИТБАНК,"443079, г. Самара, проезд им Г.Митирева, 11",NaN,1026300002519,2848,951,23.08.2006,Лицензия отозвана 16 октября 2008,13.08.2012,Отзыв (аннулирование) лицензии и завершение пр...,NaN,NaN,NaN,NaN,NaN
552,https://www.asv.org.ru/banks/yurb-ooo,"Общество с ограниченной ответственностью ""Южны...","ООО ""ЮРБ""","344006, г. Ростов-на-Дону, ул. Пушкинская, 144-а",NaN,1026100008373,3015,784,14.03.2005,Лицензия отозвана 20 июля 2018,11.08.2020,Отзыв (аннулирование) лицензии и завершение пр...,NaN,NaN,NaN,NaN,NaN


In [66]:
combined_df.to_csv('combined_df.csv', index=False, encoding='utf-8-sig')

По наблюдениям, если банк исключен из реестра, у него не будет информации про страховые выплаты. Следующий парсинг будет связан со всей оставшейся информацией на странице и он завтрачивает много времени, поэтому для оптимальности исключим все банки, у которых столбец "Основание для исключения из реестра" чем либо заполнен и оставим для дальнейшего парсинга только те, у которых он не заполнен.

In [67]:
filtered_df = combined_df[combined_df["Основание для исключения из реестра"].fillna("").eq("")][["Ссылка на банк"]].copy()
filtered_df = filtered_df.reset_index(drop=True)
print(filtered_df)

                                        Ссылка на банк
0           https://www.asv.org.ru/banks/1bank-akb-pao
1         https://www.asv.org.ru/banks/ao-avtogradbank
2       https://www.asv.org.ru/banks/agroinkombank-pao
3        https://www.asv.org.ru/banks/agrosoyuz-kb-ooo
4    https://www.asv.org.ru/banks/admiralteyskiy-kb...
..                                                 ...
290    https://www.asv.org.ru/banks/pao-energomashbank
291       https://www.asv.org.ru/banks/ergobank-kb-ooo
292          https://www.asv.org.ru/banks/esidbank-ooo
293        https://www.asv.org.ru/banks/yugra-bank-pao
294    https://www.asv.org.ru/banks/yunikorbank-kb-oao

[295 rows x 1 columns]


In [75]:
links = filtered_df['Ссылка на банк'].to_list()
links

['https://www.asv.org.ru/banks/1bank-akb-pao',
 'https://www.asv.org.ru/banks/ao-avtogradbank',
 'https://www.asv.org.ru/banks/agroinkombank-pao',
 'https://www.asv.org.ru/banks/agrosoyuz-kb-ooo',
 'https://www.asv.org.ru/banks/admiralteyskiy-kb-ooo',
 'https://www.asv.org.ru/banks/aymanibank-kb-ooo',
 'https://www.asv.org.ru/banks/ak-bank-ao',
 'https://www.asv.org.ru/banks/aksonbank-kb-ooo',
 'https://www.asv.org.ru/banks/aktsiya-akb-oao',
 'https://www.asv.org.ru/banks/aldanzolotobank-ab-ao',
 'https://www.asv.org.ru/banks/ooo-kb-alba-alyans',
 'https://www.asv.org.ru/banks/alta-bank-kb-zao',
 'https://www.asv.org.ru/banks/amb-bank-pao',
 'https://www.asv.org.ru/banks/amt-bank-ooo',
 'https://www.asv.org.ru/banks/ankor-bank-ao',
 'https://www.asv.org.ru/banks/antalbank-kb-ooo',
 'https://www.asv.org.ru/banks/arksbank-ao',
 'https://www.asv.org.ru/banks/arsenal-kb-ooo',
 'https://www.asv.org.ru/banks/aspekt-ab-ao',
 'https://www.asv.org.ru/banks/assotsiatsiya-kb-ao',
 'https://www.as

In [78]:
len(links)/4

73.75

## Парсинг данных с сайта каждого банка

Если парсить все 553 ссылки, получится перегруз оперативной памяти, поэтому разделим все ссылки на 4 части.

In [82]:
links_1 = ['https://www.asv.org.ru/banks/1bank-akb-pao',
 'https://www.asv.org.ru/banks/ao-avtogradbank',
 'https://www.asv.org.ru/banks/agroinkombank-pao',
 'https://www.asv.org.ru/banks/agrosoyuz-kb-ooo',
 'https://www.asv.org.ru/banks/admiralteyskiy-kb-ooo',
 'https://www.asv.org.ru/banks/aymanibank-kb-ooo',
 'https://www.asv.org.ru/banks/ak-bank-ao',
 'https://www.asv.org.ru/banks/aksonbank-kb-ooo',
 'https://www.asv.org.ru/banks/aktsiya-akb-oao',
 'https://www.asv.org.ru/banks/aldanzolotobank-ab-ao',
 'https://www.asv.org.ru/banks/ooo-kb-alba-alyans',
 'https://www.asv.org.ru/banks/alta-bank-kb-zao',
 'https://www.asv.org.ru/banks/amb-bank-pao',
 'https://www.asv.org.ru/banks/amt-bank-ooo',
 'https://www.asv.org.ru/banks/ankor-bank-ao',
 'https://www.asv.org.ru/banks/antalbank-kb-ooo',
 'https://www.asv.org.ru/banks/arksbank-ao',
 'https://www.asv.org.ru/banks/arsenal-kb-ooo',
 'https://www.asv.org.ru/banks/aspekt-ab-ao',
 'https://www.asv.org.ru/banks/assotsiatsiya-kb-ao',
 'https://www.asv.org.ru/banks/af-bank-oao',
 'https://www.asv.org.ru/banks/baykalbank-pao',
 'https://www.asv.org.ru/banks/baltika-akb-pao',
 'https://www.asv.org.ru/banks/ooo-bank-bkf',
 'https://www.asv.org.ru/banks/bank-bft-kb-ooo',
 'https://www.asv.org.ru/banks/bank-voronezh-ao',
 'https://www.asv.org.ru/banks/bank-gorod-ao',
 'https://www.asv.org.ru/banks/bank-ekaterininskiy-pao',
 'https://www.asv.org.ru/banks/bank-premer-kredit-pao',
 'https://www.asv.org.ru/banks/bank-rossiyskiy-kredit-oao',
 'https://www.asv.org.ru/banks/bank-fininvest-ooo',
 'https://www.asv.org.ru/banks/bank-ekonomicheskiy-soyuz-ao',
 'https://www.asv.org.ru/banks/bank-t-oao',
 'https://www.asv.org.ru/banks/bank24-ru-oao',
 'https://www.asv.org.ru/banks/bankirskiy-dom-akb-ao',
 'https://www.asv.org.ru/banks/bdb-kb-ooo',
 'https://www.asv.org.ru/banks/benifit-bank-akb-zao',
 'https://www.asv.org.ru/banks/bzl-investbank-oao',
 'https://www.asv.org.ru/banks/bmb-kb-ooo',
 'https://www.asv.org.ru/banks/bnkv-akb-ao',
 'https://www.asv.org.ru/banks/bogorodskiy-bank-ooo',
 'https://www.asv.org.ru/banks/bpf-ab-zao',
 'https://www.asv.org.ru/banks/bank-razvitiya-regiona-akb-oao',
 'https://www.asv.org.ru/banks/btf-kb-ao',
 'https://www.asv.org.ru/banks/bulgar-bank-ao',
 'https://www.asv.org.ru/banks/bum-bank-ooo',
 'https://www.asv.org.ru/banks/bumerang-komsotsbank-ao',
 'https://www.asv.org.ru/banks/bfg-kredit-kb-ooo',
 'https://www.asv.org.ru/banks/vvb-bank-pao',
 'https://www.asv.org.ru/banks/vega-bank-kb-ooo',
 'https://www.asv.org.ru/banks/vek-akb-ao',
 'https://www.asv.org.ru/banks/vestinterbank-ooo',
 'https://www.asv.org.ru/banks/vitas-bank-ooo',
 'https://www.asv.org.ru/banks/vkb-oao',
 'https://www.asv.org.ru/banks/vladprombank-ooo',
 'https://www.asv.org.ru/banks/vlbank-ao',
 'https://www.asv.org.ru/banks/vneshprombank-ooo',
 'https://www.asv.org.ru/banks/vologdabank-ao',
 'https://www.asv.org.ru/banks/vostsibtranskombank-ao',
 'https://www.asv.org.ru/banks/vpb-bank-ao',
 'https://www.asv.org.ru/banks/vyborg-bank-pao',
 'https://www.asv.org.ru/banks/gagarinskiy-kb-ao',
 'https://www.asv.org.ru/banks/gazbank-akb-ao',
 'https://www.asv.org.ru/banks/gazstroybank-akb-ao',
 'https://www.asv.org.ru/banks/kb-garant-invest-ao',
 'https://www.asv.org.ru/banks/kb-geobank-ooo']

In [105]:
links_2 = ['https://www.asv.org.ru/banks/ooo-kb-gefest',
 'https://www.asv.org.ru/banks/grinkombank-ao',
 'https://www.asv.org.ru/banks/grinfildbank-ao',
 'https://www.asv.org.ru/banks/dagenergobank-ooo',
 'https://www.asv.org.ru/banks/dig-bank-iab-oao',
 'https://www.asv.org.ru/banks/dil-bank-ooo',
 'https://www.asv.org.ru/banks/doninvest-kb-ooo',
 'https://www.asv.org.ru/banks/donkhlebbank-pao',
 'https://www.asv.org.ru/banks/doris-bank-kb-ooo',
 'https://www.asv.org.ru/banks/ooo-kb-evroaziatskiy-investitsionnyy-bank',
 'https://www.asv.org.ru/banks/evroaksis-bank-ao',
 'https://www.asv.org.ru/banks/evrokommerts-kb-pao',
 'https://www.asv.org.ru/banks/evrokredit-bank-ooo',
 'https://www.asv.org.ru/banks/evrositibank-kb-pao',
 'https://www.asv.org.ru/banks/evrostandart-kb-ooo',
 'https://www.asv.org.ru/banks/evropeyskiy-trastovyy-bank-kb-zao',
 'https://www.asv.org.ru/banks/edinstvennyy-kb-ooo',
 'https://www.asv.org.ru/banks/evropeyskiy-industrialnyy-bank-akb-zao',
 'https://www.asv.org.ru/banks/enisey-akb-pao',
 'https://www.asv.org.ru/banks/zhilkredit-kb-ooo',
 'https://www.asv.org.ru/banks/zamoskvoretskiy-mkb-oao',
 'https://www.asv.org.ru/banks/zapadnyy-bank-oao',
 'https://www.asv.org.ru/banks/ao-zauber-bank',
 'https://www.asv.org.ru/banks/zernobank-ao',
 'https://www.asv.org.ru/banks/zlatkombank-kb-ao',
 'https://www.asv.org.ru/banks/ivanovo-kb-ao',
 'https://www.asv.org.ru/banks/ideya-bank-pao',
 'https://www.asv.org.ru/banks/imbank-aib-oao',
 'https://www.asv.org.ru/banks/investbank-akb-oao',
 'https://www.asv.org.ru/banks/investitsionnyy-soyuz-kb-ooo',
 'https://www.asv.org.ru/banks/inkarobank-akb-ao',
 'https://www.asv.org.ru/banks/instroybank-kb-ooo',
 'https://www.asv.org.ru/banks/interaktivnyy-bank-ooo',
 'https://www.asv.org.ru/banks/interkommerts-kb-ooo',
 'https://www.asv.org.ru/banks/ao-kb-interprombank',
 'https://www.asv.org.ru/banks/intekhbank-pao',
 'https://www.asv.org.ru/banks/intrastbank-akb-oao',
 'https://www.asv.org.ru/banks/ipotek-bank-zao',
 'https://www.asv.org.ru/banks/itb-bank-ao',
 'https://www.asv.org.ru/banks/kamskiy-gorizont-kb-ooo',
 'https://www.asv.org.ru/banks/kanskiy-kb-ooo',
 'https://www.asv.org.ru/banks/kapitalbank-akb-pao',
 'https://www.asv.org.ru/banks/kbr-bank-kb-ooo',
 'https://www.asv.org.ru/banks/kemsotsinbank-ao',
 'https://www.asv.org.ru/banks/kivi-bank-ao',
 'https://www.asv.org.ru/banks/ao-bank-kkb',
 'https://www.asv.org.ru/banks/klientskiy-bank-ao',
 'https://www.asv.org.ru/banks/konstans-bank-akb-ao',
 'https://www.asv.org.ru/banks/konfidens-bank-kb-ooo',
 'https://www.asv.org.ru/banks/kranbank-ao',
 'https://www.asv.org.ru/banks/kredit-ekspress-kb-ooo',
 'https://www.asv.org.ru/banks/kredit-moskva-bank-pao',
 'https://www.asv.org.ru/banks/kreditbank-kkb-oao',
 'https://www.asv.org.ru/banks/krk-kb-oao',
 'https://www.asv.org.ru/banks/ao-krosna-bank',
 'https://www.asv.org.ru/banks/krossinvestbank-akb-oao',
 'https://www.asv.org.ru/banks/krylovskiy-akb-ao',
 'https://www.asv.org.ru/banks/kubanskiy-universalnyy-bank-kb-ooo',
 'https://www.asv.org.ru/banks/lada-kredit-kb-zao',
 'https://www.asv.org.ru/banks/laytbank-kb-ooo',
 'https://www.asv.org.ru/banks/legion-akb-ao',
 'https://www.asv.org.ru/banks/lenoblbank-ooo',
 'https://www.asv.org.ru/banks/lesbank-akb-oao',
 'https://www.asv.org.ru/banks/m-bank-zao',
 'https://www.asv.org.ru/banks/m2m-prayvet-bank-pao',
 'https://www.asv.org.ru/banks/mab-zao',
 'https://www.asv.org.ru/banks/maksimum-kb-oao',
 'https://www.asv.org.ru/banks/mast-bank-kb-oao',
 'https://www.asv.org.ru/banks/master-bank-kb-oao',
 'https://www.asv.org.ru/banks/master-kapital-akb-oao']

In [106]:
len(links_2)

70

In [107]:
links_3 = [ 'https://www.asv.org.ru/banks/mbsp-bank-ao',
 'https://www.asv.org.ru/banks/mbfi-bank-oao',
 'https://www.asv.org.ru/banks/mvkb-kb-oao',
 'https://www.asv.org.ru/banks/mgb-rnko-ooo',
 'https://www.asv.org.ru/banks/mgmb-ooo',
 'https://www.asv.org.ru/banks/mega-bank-zao',
 'https://www.asv.org.ru/banks/ooo-kb-megapolis',
 'https://www.asv.org.ru/banks/medinvestbank-ooo',
 'https://www.asv.org.ru/banks/mezhdunarodnyy-bank-razvitiya-kb-ao',
 'https://www.asv.org.ru/banks/mezhtopenergobank-pao',
 'https://www.asv.org.ru/banks/mezhtrastbank-kb-ooo',
 'https://www.asv.org.ru/banks/metrobank-ao',
 'https://www.asv.org.ru/banks/metropol-kb-ooo',
 'https://www.asv.org.ru/banks/miko-bank-kb-ooo',
 'https://www.asv.org.ru/banks/milbank-kb-ooo',
 'https://www.asv.org.ru/banks/millenium-bank-zao',
 'https://www.asv.org.ru/banks/miraf-bank-ao',
 'https://www.asv.org.ru/banks/mir-akb-ao',
 'https://www.asv.org.ru/banks/kb-mkb-pao',
 'https://www.asv.org.ru/banks/monolit-kb-ooo',
 'https://www.asv.org.ru/banks/mosuralbank-akb-ao',
 'https://www.asv.org.ru/banks/mrb-kb-ooo',
 'https://www.asv.org.ru/banks/mfbank-kb-ooo',
 'https://www.asv.org.ru/banks/navigator-bank-oao',
 'https://www.asv.org.ru/banks/narodnyy-kredit-bank-oao',
 'https://www.asv.org.ru/banks/natskorpbank-ao',
 'https://www.asv.org.ru/banks/nbb-oao',
 'https://www.asv.org.ru/banks/ao-bank-nbt',
 'https://www.asv.org.ru/banks/nvkbank-ao',
 'https://www.asv.org.ru/banks/nevskiy-bank-pao',
 'https://www.asv.org.ru/banks/bank-neyva-ooo',
 'https://www.asv.org.ru/banks/pir-bank-ooo',
 'https://www.asv.org.ru/banks/ooo-kb-platina',
 'https://www.asv.org.ru/banks/praym-finans-bank-ao',
 'https://www.asv.org.ru/banks/prb-oao',
 'https://www.asv.org.ru/banks/prisko-kapital-bank-kb-ao',
 'https://www.asv.org.ru/banks/probiznesbank-akb-oao',
 'https://www.asv.org.ru/banks/akb-proinvestbank-pao',
 'https://www.asv.org.ru/banks/promregionbank-ooo',
 'https://www.asv.org.ru/banks/promsberbank-zao',
 'https://www.asv.org.ru/banks/promenergobank-ao',
 'https://www.asv.org.ru/banks/psb-bank-oao',
 'https://www.asv.org.ru/banks/bank-ptb-ooo',
 'https://www.asv.org.ru/banks/puls-stolitsy-kb-ooo',
 'https://www.asv.org.ru/banks/pushkino-ab-oao',
 'https://www.asv.org.ru/banks/pao-kb-pfs-bank',
 'https://www.asv.org.ru/banks/pchrb-bank-ooo',
 'https://www.asv.org.ru/banks/radiotekhbank-nkb-pao',
 'https://www.asv.org.ru/banks/razvitie-kb-ooo',
 'https://www.asv.org.ru/banks/rbr-akb-pao',
 'https://www.asv.org.ru/banks/rbs-kb-ooo',
 'https://www.asv.org.ru/banks/regnum-bank-ooo',
 'https://www.asv.org.ru/banks/renessans-kb-ooo',
 'https://www.asv.org.ru/banks/riabank-ao',
 'https://www.asv.org.ru/banks/rinvestbank-rikb-ooo',
 'https://www.asv.org.ru/banks/rkb-bank-ao',
 'https://www.asv.org.ru/banks/rmb-bank-ao',
 'https://www.asv.org.ru/banks/rosavtobank-kb-ooo',
 'https://www.asv.org.ru/banks/pao-arkb-rosbiznesbank',
 'https://www.asv.org.ru/banks/roskomsnabbank-pao',
 'https://www.asv.org.ru/banks/rosprombank-kb-ooo',
 'https://www.asv.org.ru/banks/rsb-24-bank-ao',
 'https://www.asv.org.ru/banks/rskb-ao',
 'https://www.asv.org.ru/banks/rtbk-kb-ooo',
 'https://www.asv.org.ru/banks/rubank-kb-ao',
 'https://www.asv.org.ru/banks/rublev-kb-ao',
 'https://www.asv.org.ru/banks/ao-runa-bank',
 'https://www.asv.org.ru/banks/ao-runetbank',
 'https://www.asv.org.ru/banks/ruskobank-ao',
 'https://www.asv.org.ru/banks/russkiy-ipotechnyy-bank-kb-ooo']

In [108]:
len(links_3)

70

In [ ]:
links_4 = ['https://www.asv.org.ru/banks/russkiy-trastovyy-bank-akb-ao',
 'https://www.asv.org.ru/banks/russtroybank-ao',
 'https://www.asv.org.ru/banks/reb-kb-ao',
 'https://www.asv.org.ru/banks/s-bank-zao',
 'https://www.asv.org.ru/banks/sb-bank-ooo',
 'https://www.asv.org.ru/banks/sbrr-bank-ooo',
 'https://www.asv.org.ru/banks/sakhadaymondbank-kb-ooo',
 'https://www.asv.org.ru/banks/severnyy-kredit-kb-ao',
 'https://www.asv.org.ru/banks/sibneftebank-oao',
 'https://www.asv.org.ru/banks/sibes-bank-ao',
 'https://www.asv.org.ru/banks/simbirsk-gb-oao',
 'https://www.asv.org.ru/banks/slavyanskiy-bank-akb-zao',
 'https://www.asv.org.ru/banks/smartbank-ao',
 'https://www.asv.org.ru/banks/smolevich-bank-pao',
 'https://www.asv.org.ru/banks/smolenskiy-bank-oao',
 'https://www.asv.org.ru/banks/soveren-bank-ao',
 'https://www.asv.org.ru/banks/sovetskiy-bank-ao',
 'https://www.asv.org.ru/banks/sodruzhestvo-bank-oao',
 'https://www.asv.org.ru/banks/solidarnost-bank-ao',
 'https://www.asv.org.ru/banks/sofrino-kb-zao',
 'https://www.asv.org.ru/banks/soyuznyy-kb-ooo',
 'https://www.asv.org.ru/banks/spurt-akb-pao',
 'https://www.asv.org.ru/banks/kb-sputnik-pao',
 'https://www.asv.org.ru/banks/starbank-ao',
 'https://www.asv.org.ru/banks/stella-bank-akb-ao',
 'https://www.asv.org.ru/banks/strategiya-akb-pao',
 'https://www.asv.org.ru/banks/stroykredit-kb-oao',
 'https://www.asv.org.ru/banks/taatta-bank-ao',
 'https://www.asv.org.ru/banks/taym-bank-pao',
 'https://www.asv.org.ru/banks/talmenka-bank-kb-ooo',
 'https://www.asv.org.ru/banks/tatagroprombank-ooo',
 'https://www.asv.org.ru/banks/tatfondbank-pao',
 'https://www.asv.org.ru/banks/taurus-bank-ao',
 'https://www.asv.org.ru/banks/tgb-bank-ao',
 'https://www.asv.org.ru/banks/tempbank-mab-pao',
 'https://www.asv.org.ru/banks/transnatsionalnyy-bank-kb-ooo',
 'https://www.asv.org.ru/banks/transportnyy-kb-ooo',
 'https://www.asv.org.ru/banks/troyka-d-bank-ao',
 'https://www.asv.org.ru/banks/tusarbank-ao',
 'https://www.asv.org.ru/banks/ao-tembr-bank',
 'https://www.asv.org.ru/banks/tyumenagroprombank-zao',
 'https://www.asv.org.ru/banks/uik-bank-ooo',
 'https://www.asv.org.ru/banks/um-bank-ooo',
 'https://www.asv.org.ru/banks/unifin-kb-ao',
 'https://www.asv.org.ru/banks/uralkapitalbank-ooo',
 'https://www.asv.org.ru/banks/uralliga-kb-zao',
 'https://www.asv.org.ru/banks/uraltransbank-pao',
 'https://www.asv.org.ru/banks/uralskiy-trastovyy-bank-oao',
 'https://www.asv.org.ru/banks/ussuri-bank-ao',
 'https://www.asv.org.ru/banks/fia-bank-ao',
 'https://www.asv.org.ru/banks/finansovyy-kapital-kb-ooo',
 'https://www.asv.org.ru/banks/finansovyy-standart-kb-ooo',
 'https://www.asv.org.ru/banks/finprombank-akb-pao',
 'https://www.asv.org.ru/banks/forus-bank-ao',
 'https://www.asv.org.ru/banks/kholding-kredit-kb-ooo',
 'https://www.asv.org.ru/banks/tsentrkombank-ooo',
 'https://www.asv.org.ru/banks/tserikh-bank-zao',
 'https://www.asv.org.ru/banks/chuvashkreditprombank-akb-pao',
 'https://www.asv.org.ru/banks/ekoprombank-akb-oao',
 'https://www.asv.org.ru/banks/ekspert-bank-ao',
 'https://www.asv.org.ru/banks/ekspress-akb-oao',
 'https://www.asv.org.ru/banks/ekspress-kredit-kb-ao',
 'https://www.asv.org.ru/banks/el-bank-kb-ooo',
 'https://www.asv.org.ru/banks/elbin-akb-ao',
 'https://www.asv.org.ru/banks/pao-energomashbank',
 'https://www.asv.org.ru/banks/ergobank-kb-ooo',
 'https://www.asv.org.ru/banks/esidbank-ooo',
 'https://www.asv.org.ru/banks/yugra-bank-pao',
 'https://www.asv.org.ru/banks/yunikorbank-kb-oao',
 'https://www.asv.org.ru/banks/neftyanoy-alyans-kb-pao',
 'https://www.asv.org.ru/banks/nzbank-akb-oao',
 'https://www.asv.org.ru/banks/nkb-kb-ooo',
 'https://www.asv.org.ru/banks/novokuznetskiy-munitsipalnyy-bank-akb-oao',
 'https://www.asv.org.ru/banks/novatsiya-akb-pao',
 'https://www.asv.org.ru/banks/novopokrovskiy-kb-ooo',
 'https://www.asv.org.ru/banks/nota-bank-pao',
 'https://www.asv.org.ru/banks/nst-bank-ao',
 'https://www.asv.org.ru/banks/neklis-bank-kb-ooo',
 'https://www.asv.org.ru/banks/o-k-bank-pao',
 'https://www.asv.org.ru/banks/obpi-akb-pao',
 'https://www.asv.org.ru/banks/obrazovanie-akib-ao',
 'https://www.asv.org.ru/banks/ogni-moskvy-kb-ooo',
 'https://www.asv.org.ru/banks/opm-bank-kb-ooo',
 'https://www.asv.org.ru/banks/ao-orbank',
 'https://www.asv.org.ru/banks/ofk-bank-pao',
 'https://www.asv.org.ru/banks/pv-bank-zao',
 'https://www.asv.org.ru/banks/pervomayskiy-bank-pao',
 'https://www.asv.org.ru/banks/pervyy-ekspress-kb-oao',
 'https://www.asv.org.ru/banks/peresvet-akb-pao-prezhnee-naimenovanie-peresvet-akb-ao-']

In [110]:
len(links_4)

89

In [85]:
"""Эта программа работает примерно 66 минут"""

import pandas as pd
from selenium.webdriver.common.action_chains import ActionChains
import time
from datetime import datetime
import json

class UnifiedBankDataScraper:
    def __init__(self):
        self.driver = self._setup_driver()
        self.combined_df = pd.DataFrame()
        
    def _setup_driver(self):
        chrome_options = Options()
        chrome_options.add_argument('--disable-gpu')
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-dev-shm-usage')
        driver = webdriver.Chrome(options=chrome_options)
        driver.implicitly_wait(5)
        return driver
    
    def _scroll_to_element(self, element):
        actions = ActionChains(self.driver)
        actions.move_to_element(element).perform()
        time.sleep(5)
    
    def _expand_section(self, section):
        if 'card-foldable_open' not in section.get_attribute('class'):
            header = section.find_element(By.CSS_SELECTOR, 'div.card-foldable__header')
            header.click()
            time.sleep(5)
    
    def _parse_bank_info(self, url):
        """Парсинг основной информации о банке"""
        bank_info = {"URL": url, "Дата парсинга": datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
        
        try:
            ul_element = WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.foldable__body > div > app-company-block-requisites-table > ul')))
            
            for li in ul_element.find_elements(By.TAG_NAME, "li"):
                try:
                    title = li.find_element(By.CSS_SELECTOR, 'div.requisite-item__title > div').text.strip()
                    value = li.find_element(By.CSS_SELECTOR, 'div.requisite-item__value > div').text.strip()
                    bank_info[title] = value
                except:
                    continue
        except Exception as e:
            print(f"Ошибка парсинга информации о банке: {str(e)}")
        
        return bank_info
    
    def _parse_insurance_payments(self):
        """Парсинг страховых выплат"""
        payments = {}
        
        try:
            insurance_element = self.driver.find_element(By.CSS_SELECTOR, 'app-insurance-payments')
            
            paid = insurance_element.find_element(By.XPATH, ".//div[@class='col-24 col-md-auto'][1]//span/nobr").text.strip()
            remaining = insurance_element.find_element(By.XPATH, ".//div[@class='col-24 col-md-auto mt-4 mt-md-0'][1]//span/nobr").text.strip()
            percentage = insurance_element.find_element(By.CSS_SELECTOR, 'div.progress-performed-number').text.strip()
            
            payments.update({
                "Страховые выплаты - Выплачено": paid,
                "Страховые выплаты - Осталось выплатить": remaining,
                "Страховые выплаты - Процент выплаченного": percentage
            })
        except Exception as e:
            print(f"Ошибка парсинга страховых выплат: {str(e)}")
        
        return payments
    
    def _parse_competitive_mass(self, url):
        """Парсинг конкурсной массы и добавление в общий DataFrame"""
        try:
            sections = WebDriverWait(self.driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.card-foldable")))
            
            target_section = None
            for section in sections:
                try:
                    title = section.find_element(By.CSS_SELECTOR, "div.card-foldable-title__main-text").text
                    if "формировании конкурсной массы" in title.lower():
                        target_section = section
                        break
                except:
                    continue
            
            if not target_section:
                return pd.DataFrame()
            
            self._expand_section(target_section)
            self._scroll_to_element(target_section)
            time.sleep(2)
            
            # Создаем временный DataFrame для этого раздела
            temp_df = pd.DataFrame()
            
            # Общая сумма
            try:
                total = target_section.find_element(By.CSS_SELECTOR, "div.card-foldable-title__secondary-text").text.strip()
                temp_df["Конкурсная масса - Общая сумма"] = [total]
            except:
                pass
            
            # Динамика (преобразуем в строку JSON для хранения в одной ячейке)
            try:
                diagrams = target_section.find_elements(By.CSS_SELECTOR, "app-block-page-kp-diagram")
                dynamics = []
                for diagram in diagrams:
                    bars = diagram.find_elements(By.CSS_SELECTOR, "div.diagram__bar-wrapper")
                    for bar in bars:
                        try:
                            period = bar.find_element(By.CSS_SELECTOR, "div.diagram__month").text.strip()
                            amount = bar.find_element(By.CSS_SELECTOR, "div.diagram__percent span").text.strip()
                            currency = bar.find_elements(By.CSS_SELECTOR, "div.diagram__percent span")[1].text.strip()
                            dynamics.append(f"{period}: {amount} {currency}")
                        except:
                            continue
                temp_df["Конкурсная масса - Динамика"] = [json.dumps(dynamics, ensure_ascii=False)]
            except:
                pass
            
            # Таблицы (имущество, сформированная масса, реализация)
            try:
                tables = target_section.find_elements(By.CSS_SELECTOR, "div.card-body_foldable-card")
                for table in tables:
                    try:
                        header = table.find_element(By.CSS_SELECTOR, "span.body-bold").text
                        rows = table.find_elements(By.CSS_SELECTOR, "div.table-list__row_depth-0")
                        
                        table_data = []
                        for row in rows:
                            try:
                                if 'table-list__row_header' in row.get_attribute('class') or 'table-list__row_footer' in row.get_attribute('class'):
                                    continue
                                
                                title = row.find_element(By.CSS_SELECTOR, "span.table-list__cell-title-text").text.strip()
                                value = row.find_element(By.CSS_SELECTOR, "span.value").text.strip()
                                table_data.append(f"{title}: {value}")
                            except:
                                continue
                        
                        if "Имущество (активы)" in header:
                            temp_df["Конкурсная масса - Имущество"] = [json.dumps(table_data, ensure_ascii=False)]
                        elif "Сформированная конкурсная масса" in header:
                            temp_df["Конкурсная масса - Сформированная масса"] = [json.dumps(table_data, ensure_ascii=False)]
                        elif "Реализация имущества" in header:
                            temp_df["Конкурсная масса - Реализация имущества"] = [json.dumps(table_data, ensure_ascii=False)]
                    except:
                        continue
            except:
                pass
            
            # Добавляем URL для связи с основной таблицей
            temp_df["URL"] = url
            
            return temp_df
            
        except Exception as e:
            print(f"Ошибка парсинга конкурсной массы: {str(e)}")
            return pd.DataFrame()
    
    def _parse_expenses(self, url):
        """Парсинг расходов и добавление в общий DataFrame"""
        try:
            sections = WebDriverWait(self.driver, 20).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.card-foldable")))
            
            target_section = None
            for section in sections:
                try:
                    title = section.find_element(By.CSS_SELECTOR, "div.card-foldable-title__main-text").text
                    if "Расходы" in title:
                        target_section = section
                        break
                except:
                    continue
            
            if not target_section:
                return pd.DataFrame()
            
            self._expand_section(target_section)
            self._scroll_to_element(target_section)
            time.sleep(2)
            
            # Создаем временный DataFrame для этого раздела
            temp_df = pd.DataFrame()
            
            # Общая сумма и дата
            try:
                total = target_section.find_element(By.CSS_SELECTOR, "div.card-foldable-title__secondary-text").text.strip()
                temp_df["Расходы - Общая сумма"] = [total]
            except:
                pass
            
            try:
                date = target_section.find_element(By.CSS_SELECTOR, "div.card-foldable__desc").text.strip()
                temp_df["Расходы - Дата обновления"] = [date]
            except:
                pass
            
            # Динамика расходов
            try:
                diagrams = target_section.find_elements(By.CSS_SELECTOR, "app-block-page-kp-diagram")
                dynamics = []
                for diagram in diagrams:
                    bars = diagram.find_elements(By.CSS_SELECTOR, "div.diagram__bar-wrapper")
                    for bar in bars:
                        try:
                            period = bar.find_element(By.CSS_SELECTOR, "div.diagram__month").text.strip()
                            amount = bar.find_element(By.CSS_SELECTOR, "div.diagram__percent span").text.strip()
                            currency = bar.find_elements(By.CSS_SELECTOR, "div.diagram__percent span")[1].text.strip()
                            dynamics.append(f"{period}: {amount} {currency}")
                        except:
                            continue
                temp_df["Расходы - Динамика"] = [json.dumps(dynamics, ensure_ascii=False)]
            except:
                pass
            
            # Статьи расходов
            try:
                expenses_header = target_section.find_element(By.XPATH, ".//span[contains(text(), 'Наименования расходов')]")
                expenses_table = expenses_header.find_element(By.XPATH, "./following-sibling::div[contains(@class, 'table-expenses-list')]")
                
                expenses_data = []
                rows = expenses_table.find_elements(By.CSS_SELECTOR, "app-table-statistic-row")
                for row in rows:
                    try:
                        title = row.find_element(By.CSS_SELECTOR, "span.table-list__cell-title-text").text.strip()
                        value = row.find_element(By.CSS_SELECTOR, "span.value").text.strip()
                        expenses_data.append(f"{title}: {value}")
                    except:
                        continue
                
                temp_df["Расходы - Статьи"] = [json.dumps(expenses_data, ensure_ascii=False)]
            except:
                pass
            
            temp_df["URL"] = url
            
            return temp_df
            
        except Exception as e:
            print(f"Ошибка парсинга расходов: {str(e)}")
            return pd.DataFrame()
    
    def _parse_legal_work(self, url):
        """Парсинг юридической работы и добавление в общий DataFrame"""
        try:
            sections = WebDriverWait(self.driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.card-foldable")))
            
            target_section = None
            for section in sections:
                try:
                    title = section.find_element(By.CSS_SELECTOR, "div.card-foldable-title__main-text").text
                    if "Юридическая работа" in title:
                        target_section = section
                        break
                except:
                    continue
            
            if not target_section:
                return pd.DataFrame()
            
            self._expand_section(target_section)
            self._scroll_to_element(target_section)
            time.sleep(2)
            
            # Создаем временный DataFrame для этого раздела
            temp_df = pd.DataFrame()
            
            # Дата обновления
            try:
                date = target_section.find_element(By.CSS_SELECTOR, "div.card-foldable__desc").text.strip()
                temp_df["Юридическая работа - Дата обновления"] = [date]
            except:
                pass
            
            # Процессуальные действия
            try:
                table = target_section.find_element(By.CSS_SELECTOR, "div.table-list")
                actions_data = []
                rows = table.find_elements(By.CSS_SELECTOR, "div.table-list__row.ng-star-inserted")
                
                for row in rows:
                    try:
                        action = row.find_element(By.CSS_SELECTOR, "span.tooltip").text.strip()
                        count = row.find_element(By.CSS_SELECTOR, "div.table-list__sub-cell.second-value").text.strip()
                        amount = row.find_element(By.CSS_SELECTOR, "div.table-list__cell_number nobr").text.strip()
                        actions_data.append(f"{action} (Кол-во: {count}, Сумма: {amount})")
                    except:
                        continue
                
                temp_df["Юридическая работа - Процессуальные действия"] = [json.dumps(actions_data, ensure_ascii=False)]
            except:
                pass
            
            temp_df["URL"] = url
            
            return temp_df
            
        except Exception as e:
            print(f"Ошибка парсинга юридической работы: {str(e)}")
            return pd.DataFrame()
    
    def _parse_payments_to_creditors(self, url):
        """Парсинг выплат кредиторам и добавление в общий DataFrame"""
        try:
            payments_section = self.driver.find_element(By.XPATH, "//div[contains(., 'Выплачено кредиторам')]")
            self.driver.execute_script("arguments[0].scrollIntoView(true);", payments_section)
            time.sleep(2)
            
            # Создаем временный DataFrame для этого раздела
            temp_df = pd.DataFrame()
            
            # Данные по очередям выплат
            try:
                rows = self.driver.find_elements(By.CSS_SELECTOR, "div.table-list__row_depth-0")
                payments_data = []
                
                for row in rows:
                    try:
                        if ('table-list__row_header' in row.get_attribute('class') or 
                            'table-list__row_footer' in row.get_attribute('class')):
                            continue
                        
                        title = row.find_element(By.CSS_SELECTOR, "span.table-list__cell-title-text").text.strip()
                        cells = row.find_elements(By.CSS_SELECTOR, "div.table-list__cell-value")
                        
                        if len(cells) >= 3:
                            payments_data.append(
                                f"{title}: Установлено {cells[0].text.strip()}, "
                                f"Удовлетворено {cells[1].text.strip()}, "
                                f"Отступное {cells[2].text.strip()}"
                            )
                    except:
                        continue
                
                temp_df["Выплаты кредиторам - Данные"] = [json.dumps(payments_data, ensure_ascii=False)]
            except:
                pass
            
            # Итоговые данные
            try:
                total_row = self.driver.find_element(By.CSS_SELECTOR, "div.table-list__row_footer")
                cells = total_row.find_elements(By.CSS_SELECTOR, "div.table-list__cell-value")
                
                if len(cells) >= 3:
                    temp_df["Выплаты кредиторам - Итого"] = [
                        f"Установлено: {cells[0].text.strip()}, "
                        f"Удовлетворено: {cells[1].text.strip()}, "
                        f"Отступное: {cells[2].text.strip()}"
                    ]
            except:
                pass
            
            temp_df["URL"] = url
            
            return temp_df
            
        except Exception as e:
            print(f"Ошибка при парсинге выплат кредиторам: {str(e)}")
            return pd.DataFrame()
    
    def scrape_bank(self, url):
        """Сбор всех данных по одному банку и объединение в один DataFrame"""
        print(f"\nПарсинг банка: {url}")
        self.driver.get(url)
        time.sleep(5)
        
        # Прокрутка для активации ленивой загрузки
        self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
        self.driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(5)
        
        # Собираем все данные
        bank_info = self._parse_bank_info(url)
        insurance = self._parse_insurance_payments()
        competitive_mass = self._parse_competitive_mass(url)
        expenses = self._parse_expenses(url)
        legal_work = self._parse_legal_work(url)
        payments = self._parse_payments_to_creditors(url)
        
        # Создаем основной DataFrame для этого банка
        bank_df = pd.DataFrame([bank_info])
        
        # Объединяем все данные
        bank_df = bank_df.merge(pd.DataFrame([insurance]), how='left', left_index=True, right_index=True)
        bank_df = bank_df.merge(competitive_mass, how='left', on='URL')
        bank_df = bank_df.merge(expenses, how='left', on='URL')
        bank_df = bank_df.merge(legal_work, how='left', on='URL')
        bank_df = bank_df.merge(payments, how='left', on='URL')
        
        # Добавляем в общий DataFrame
        self.combined_df = pd.concat([self.combined_df, bank_df], ignore_index=True)
        
        print(f"Данные по банку {url} добавлены в общий DataFrame")
        return bank_df
    
    def scrape_all_banks(self, bank_urls):
        """Сбор данных по всем банкам"""
        for url in bank_urls:
            try:
                self.scrape_bank(url)
                time.sleep(3)
            except Exception as e:
                print(f"Ошибка при обработке банка {url}: {str(e)}")
                continue
        
        # Заполняем пропущенные значения
        self.combined_df.fillna("Нет данных", inplace=True)
        json_columns = [
            "Конкурсная масса - Динамика",
            "Конкурсная масса - Имущество",
            "Конкурсная масса - Сформированная масса",
            "Конкурсная масса - Реализация имущества",
            "Расходы - Динамика",
            "Расходы - Статьи",
            "Юридическая работа - Процессуальные действия",
            "Выплаты кредиторам - Данные"
        ]
        
        for col in json_columns:
            if col in self.combined_df.columns:
                self.combined_df[col] = self.combined_df[col].apply(
                    lambda x: "\n".join(json.loads(x)) if x != "Нет данных" else x
                )
        
        return self.combined_df
    
    def save_to_excel(self, filename="all_banks_combined.xlsx"):
        """Сохранение объединенного DataFrame в Excel"""
        if not self.combined_df.empty:
            self.combined_df.to_excel(filename, index=False, encoding='utf-8-sig')
            print(f"\n{filename}")
        else:
            print("\nНет данных для сохранения")
    
    def close(self):
        """Закрытие драйвера"""
        self.driver.quit()

if __name__ == "__main__":
    bank_urls = links_1
    
    scraper = UnifiedBankDataScraper()
    try:
        combined_data_1 = scraper.scrape_all_banks(bank_urls)
        print("\nСоздан объединенный DataFrame со следующими столбцами:")
        print(combined_data_1.columns.tolist())
        print(f"\nВсего записей: {len(combined_data_1)}")
        scraper.save_to_excel("bank_data.xlsx")
        print(combined_data_1.head())
        
    except Exception as e:
        print(f"Произошла ошибка: {str(e)}")
    finally:
        scraper.close()


Парсинг банка: https://www.asv.org.ru/banks/1bank-akb-pao
Данные по банку https://www.asv.org.ru/banks/1bank-akb-pao добавлены в общий DataFrame

Парсинг банка: https://www.asv.org.ru/banks/ao-avtogradbank
Данные по банку https://www.asv.org.ru/banks/ao-avtogradbank добавлены в общий DataFrame

Парсинг банка: https://www.asv.org.ru/banks/agroinkombank-pao
Данные по банку https://www.asv.org.ru/banks/agroinkombank-pao добавлены в общий DataFrame

Парсинг банка: https://www.asv.org.ru/banks/agrosoyuz-kb-ooo
Данные по банку https://www.asv.org.ru/banks/agrosoyuz-kb-ooo добавлены в общий DataFrame

Парсинг банка: https://www.asv.org.ru/banks/admiralteyskiy-kb-ooo
Данные по банку https://www.asv.org.ru/banks/admiralteyskiy-kb-ooo добавлены в общий DataFrame

Парсинг банка: https://www.asv.org.ru/banks/aymanibank-kb-ooo
Данные по банку https://www.asv.org.ru/banks/aymanibank-kb-ooo добавлены в общий DataFrame

Парсинг банка: https://www.asv.org.ru/banks/ak-bank-ao
Данные по банку https://ww

In [86]:
combined_data_1

,URL,Дата парсинга,Полное фирменное наименование,Сокращенное фирменное наименование,Место нахождения,Почтовый адрес,ОГРН,Номер в Книге государственной регистрации кредитных организаций,Номер по реестру,Дата включения в реестр,...,Конкурсная масса - Имущество,Конкурсная масса - Сформированная масса,Конкурсная масса - Реализация имущества,Расходы - Общая сумма,Расходы - Дата обновления,Расходы - Динамика,Юридическая работа - Дата обновления,Юридическая работа - Процессуальные действия,Выплаты кредиторам - Данные,Выплаты кредиторам - Итого
0,https://www.asv.org.ru/banks/1bank-akb-pao,2025-06-19 01:30:04,"Акционерный коммерческий банк ""1Банк"" (публичн...","ПАО АКБ ""1Банк""","362040, Республика Северная Осетия-Алания, г.В...","362001, г. Владикавказ, ул. Гагарина, д. 30",1021500000147,2896,985,20.10.2010,...,Денежные средства и драгоценные металлы: 0\nКо...,Денежные средства на дату открытия конкурсного...,Стоимость реализованного имущества: 9\nПоступи...,182 872 тыс. руб.,Обновлено 01.05.2025,янв: 442 \nфев: 343 \nмар: 436 \nапр: 409 \n20...,Обновлено 01.05.2025,"Предъявлено исков (Кол-во: 1 192, Сумма: 7 257...",1-я очередь (в реестре): Установлено 1 244 844...,"Установлено: 1 301 874, Удовлетворено: 82 050,..."
1,https://www.asv.org.ru/banks/ao-avtogradbank,2025-06-19 01:31:04,Акционерное общество «Автоградбанк»,АО «Автоградбанк»,"423831, Республика Татарстан (Татарстан), г. Н...","109992, Москва, «Москва ГСП-2» – ликвидатор АО...",1021600000806,1455,5,21.09.2004,...,Денежные средства и драгоценные металлы: 5 047...,Денежные средства на дату открытия конкурсного...,Стоимость реализованного имущества: 196 075\nП...,221 975 тыс. руб.,Обновлено 01.05.2025,янв: 17 439 \nфев: 34 835 \nмар: 6431 \nапр: 2...,Обновлено 01.05.2025,"Предъявлено исков (Кол-во: 303, Сумма: 3 425 2...",1-я очередь (в реестре): Установлено 2 208 780...,"Установлено: 6 222 267, Удовлетворено: 5 623 2..."
2,https://www.asv.org.ru/banks/agroinkombank-pao,2025-06-19 01:32:03,Публичное акционерное общество Агроинвестицион...,ПАО АГРОИНКОМБАНК,"414000, город Астрахань, пл. Ленина/ул. Бурова...","109992, г. Москва, ГСП-2, ГК «АСВ» – конкурсны...",1023000927520,1946,22,21.09.2004,...,Денежные средства и драгоценные металлы: 0\nКо...,Денежные средства на дату открытия конкурсного...,Стоимость реализованного имущества: 0\nПоступи...,68 176 тыс. руб.,Обновлено 01.05.2025,янв: 0 \nфев: 689 \nмар: 79 \nапр: 76 \n2016: ...,Обновлено 01.05.2025,"Предъявлено исков (Кол-во: 51, Сумма: 5 273 78...","1-я очередь (в реестре): Установлено 622 049, ...","Установлено: 991 747, Удовлетворено: 12 939, О..."
3,https://www.asv.org.ru/banks/agrosoyuz-kb-ooo,2025-06-19 01:33:08,Общество с ограниченной ответственностью Комме...,"ООО КБ ""Агросоюз""","101000, г. Москва, Уланский пер., д. 13, стр. 1","109992, г. Москва, ГСП-2, ГК «АСВ» – конкурсны...",1025600001130,1459,649,17.02.2005,...,Денежные средства и драгоценные металлы: 0\nКо...,Денежные средства на дату открытия конкурсного...,Стоимость реализованного имущества: 1 180\nПос...,924 795 тыс. руб.,Обновлено 01.05.2025,янв: 1749 \nфев: 7670 \nмар: 4870 \nапр: 5576 ...,Обновлено 01.05.2025,"Предъявлено исков (Кол-во: 1 975, Сумма: 63 71...",1-я очередь (в реестре): Установлено 8 303 165...,"Установлено: 9 169 142, Удовлетворено: 2 204 0..."
4,https://www.asv.org.ru/banks/admiralteyskiy-kb...,2025-06-19 01:34:08,"Коммерческий Банк ""Адмиралтейский"" (общество с...","ООО КБ ""Адмиралтейский""","119034, г. Москва, Лопухинский пер., д.6, стр.1","109992, г. Москва, ГСП-2, ГК «АСВ» – конкурсны...",1037739763753,3054,791,21.03.2005,...,Денежные средства и драгоценные металлы: 0\nКо...,Денежные средства на дату открытия конкурсного...,Стоимость реализованного имущества: 116\nПосту...,725 408 тыс. руб.,Обновлено 01.05.2025,янв: 683 \nфев: 1491 \nмар: 2802 \nапр: 2607 \...,Обновлено 01.05.2025,"Предъявлено исков (Кол-во: 260, Сумма: 22 091 ...",1-я очередь (в реестре): Установлено 3 867 592...,"Установлено: 7 089 011, Удовлетворено: 2 856 3...

In [ ]:
combined_data_1.to_csv('combined_data_1', encoding='utf-8')

In [88]:
len(combined_data_1) == len(links_1)

False

Получается, есть 1 банк, который не спарсился

In [89]:
len(links_1)

66

In [111]:
"""Эта программа работает примерно 30 минут"""

class UnifiedBankDataScraper:
    def __init__(self):
        self.driver = self._setup_driver()
        self.combined_df = pd.DataFrame()
        
    def _setup_driver(self):
        chrome_options = Options()
        chrome_options.add_argument('--disable-gpu')
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-dev-shm-usage')
        driver = webdriver.Chrome(options=chrome_options)
        driver.implicitly_wait(5)
        return driver
    
    def _scroll_to_element(self, element):
        actions = ActionChains(self.driver)
        actions.move_to_element(element).perform()
        time.sleep(5)
    
    def _expand_section(self, section):
        if 'card-foldable_open' not in section.get_attribute('class'):
            header = section.find_element(By.CSS_SELECTOR, 'div.card-foldable__header')
            header.click()
            time.sleep(5)
    
    def _parse_bank_info(self, url):
        """Парсинг основной информации о банке"""
        bank_info = {"URL": url, "Дата парсинга": datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
        
        try:
            ul_element = WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.foldable__body > div > app-company-block-requisites-table > ul')))
            
            for li in ul_element.find_elements(By.TAG_NAME, "li"):
                try:
                    title = li.find_element(By.CSS_SELECTOR, 'div.requisite-item__title > div').text.strip()
                    value = li.find_element(By.CSS_SELECTOR, 'div.requisite-item__value > div').text.strip()
                    bank_info[title] = value
                except:
                    continue
        except Exception as e:
            print(f"Ошибка парсинга информации о банке: {str(e)}")
        
        return bank_info
    
    def _parse_insurance_payments(self):
        """Парсинг страховых выплат"""
        payments = {}
        
        try:
            insurance_element = self.driver.find_element(By.CSS_SELECTOR, 'app-insurance-payments')
            
            paid = insurance_element.find_element(By.XPATH, ".//div[@class='col-24 col-md-auto'][1]//span/nobr").text.strip()
            remaining = insurance_element.find_element(By.XPATH, ".//div[@class='col-24 col-md-auto mt-4 mt-md-0'][1]//span/nobr").text.strip()
            percentage = insurance_element.find_element(By.CSS_SELECTOR, 'div.progress-performed-number').text.strip()
            
            payments.update({
                "Страховые выплаты - Выплачено": paid,
                "Страховые выплаты - Осталось выплатить": remaining,
                "Страховые выплаты - Процент выплаченного": percentage
            })
        except Exception as e:
            print(f"Ошибка парсинга страховых выплат: {str(e)}")
        
        return payments
    
    def _parse_competitive_mass(self, url):
        """Парсинг конкурсной массы и добавление в общий DataFrame"""
        try:
            sections = WebDriverWait(self.driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.card-foldable")))
            
            target_section = None
            for section in sections:
                try:
                    title = section.find_element(By.CSS_SELECTOR, "div.card-foldable-title__main-text").text
                    if "формировании конкурсной массы" in title.lower():
                        target_section = section
                        break
                except:
                    continue
            
            if not target_section:
                return pd.DataFrame()
            
            self._expand_section(target_section)
            self._scroll_to_element(target_section)
            time.sleep(2)
            
            temp_df = pd.DataFrame()
            
            try:
                total = target_section.find_element(By.CSS_SELECTOR, "div.card-foldable-title__secondary-text").text.strip()
                temp_df["Конкурсная масса - Общая сумма"] = [total]
            except:
                pass
            
            try:
                diagrams = target_section.find_elements(By.CSS_SELECTOR, "app-block-page-kp-diagram")
                dynamics = []
                for diagram in diagrams:
                    bars = diagram.find_elements(By.CSS_SELECTOR, "div.diagram__bar-wrapper")
                    for bar in bars:
                        try:
                            period = bar.find_element(By.CSS_SELECTOR, "div.diagram__month").text.strip()
                            amount = bar.find_element(By.CSS_SELECTOR, "div.diagram__percent span").text.strip()
                            currency = bar.find_elements(By.CSS_SELECTOR, "div.diagram__percent span")[1].text.strip()
                            dynamics.append(f"{period}: {amount} {currency}")
                        except:
                            continue
                temp_df["Конкурсная масса - Динамика"] = [json.dumps(dynamics, ensure_ascii=False)]
            except:
                pass
            
            try:
                tables = target_section.find_elements(By.CSS_SELECTOR, "div.card-body_foldable-card")
                for table in tables:
                    try:
                        header = table.find_element(By.CSS_SELECTOR, "span.body-bold").text
                        rows = table.find_elements(By.CSS_SELECTOR, "div.table-list__row_depth-0")
                        
                        table_data = []
                        for row in rows:
                            try:
                                if 'table-list__row_header' in row.get_attribute('class') or 'table-list__row_footer' in row.get_attribute('class'):
                                    continue
                                
                                title = row.find_element(By.CSS_SELECTOR, "span.table-list__cell-title-text").text.strip()
                                value = row.find_element(By.CSS_SELECTOR, "span.value").text.strip()
                                table_data.append(f"{title}: {value}")
                            except:
                                continue
                        
                        if "Имущество (активы)" in header:
                            temp_df["Конкурсная масса - Имущество"] = [json.dumps(table_data, ensure_ascii=False)]
                        elif "Сформированная конкурсная масса" in header:
                            temp_df["Конкурсная масса - Сформированная масса"] = [json.dumps(table_data, ensure_ascii=False)]
                        elif "Реализация имущества" in header:
                            temp_df["Конкурсная масса - Реализация имущества"] = [json.dumps(table_data, ensure_ascii=False)]
                    except:
                        continue
            except:
                pass
            
            temp_df["URL"] = url
            
            return temp_df
            
        except Exception as e:
            print(f"Ошибка парсинга конкурсной массы: {str(e)}")
            return pd.DataFrame()
    
    def _parse_expenses(self, url):
        """Парсинг расходов и добавление в общий DataFrame"""
        try:
            sections = WebDriverWait(self.driver, 20).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.card-foldable")))
            
            target_section = None
            for section in sections:
                try:
                    title = section.find_element(By.CSS_SELECTOR, "div.card-foldable-title__main-text").text
                    if "Расходы" in title:
                        target_section = section
                        break
                except:
                    continue
            
            if not target_section:
                return pd.DataFrame()
            
            self._expand_section(target_section)
            self._scroll_to_element(target_section)
            time.sleep(2)
            
            temp_df = pd.DataFrame()

            try:
                total = target_section.find_element(By.CSS_SELECTOR, "div.card-foldable-title__secondary-text").text.strip()
                temp_df["Расходы - Общая сумма"] = [total]
            except:
                pass
            
            try:
                date = target_section.find_element(By.CSS_SELECTOR, "div.card-foldable__desc").text.strip()
                temp_df["Расходы - Дата обновления"] = [date]
            except:
                pass

            try:
                diagrams = target_section.find_elements(By.CSS_SELECTOR, "app-block-page-kp-diagram")
                dynamics = []
                for diagram in diagrams:
                    bars = diagram.find_elements(By.CSS_SELECTOR, "div.diagram__bar-wrapper")
                    for bar in bars:
                        try:
                            period = bar.find_element(By.CSS_SELECTOR, "div.diagram__month").text.strip()
                            amount = bar.find_element(By.CSS_SELECTOR, "div.diagram__percent span").text.strip()
                            currency = bar.find_elements(By.CSS_SELECTOR, "div.diagram__percent span")[1].text.strip()
                            dynamics.append(f"{period}: {amount} {currency}")
                        except:
                            continue
                temp_df["Расходы - Динамика"] = [json.dumps(dynamics, ensure_ascii=False)]
            except:
                pass
            
            try:
                expenses_header = target_section.find_element(By.XPATH, ".//span[contains(text(), 'Наименования расходов')]")
                expenses_table = expenses_header.find_element(By.XPATH, "./following-sibling::div[contains(@class, 'table-expenses-list')]")
                
                expenses_data = []
                rows = expenses_table.find_elements(By.CSS_SELECTOR, "app-table-statistic-row")
                for row in rows:
                    try:
                        title = row.find_element(By.CSS_SELECTOR, "span.table-list__cell-title-text").text.strip()
                        value = row.find_element(By.CSS_SELECTOR, "span.value").text.strip()
                        expenses_data.append(f"{title}: {value}")
                    except:
                        continue
                
                temp_df["Расходы - Статьи"] = [json.dumps(expenses_data, ensure_ascii=False)]
            except:
                pass
            
            temp_df["URL"] = url
            
            return temp_df
            
        except Exception as e:
            print(f"Ошибка парсинга расходов: {str(e)}")
            return pd.DataFrame()
    
    def _parse_legal_work(self, url):
        """Парсинг юридической работы и добавление в общий DataFrame"""
        try:
            sections = WebDriverWait(self.driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.card-foldable")))
            
            target_section = None
            for section in sections:
                try:
                    title = section.find_element(By.CSS_SELECTOR, "div.card-foldable-title__main-text").text
                    if "Юридическая работа" in title:
                        target_section = section
                        break
                except:
                    continue
            
            if not target_section:
                return pd.DataFrame()
            
            self._expand_section(target_section)
            self._scroll_to_element(target_section)
            time.sleep(2)
            
            temp_df = pd.DataFrame()
            
            try:
                date = target_section.find_element(By.CSS_SELECTOR, "div.card-foldable__desc").text.strip()
                temp_df["Юридическая работа - Дата обновления"] = [date]
            except:
                pass
            
            try:
                table = target_section.find_element(By.CSS_SELECTOR, "div.table-list")
                actions_data = []
                rows = table.find_elements(By.CSS_SELECTOR, "div.table-list__row.ng-star-inserted")
                
                for row in rows:
                    try:
                        action = row.find_element(By.CSS_SELECTOR, "span.tooltip").text.strip()
                        count = row.find_element(By.CSS_SELECTOR, "div.table-list__sub-cell.second-value").text.strip()
                        amount = row.find_element(By.CSS_SELECTOR, "div.table-list__cell_number nobr").text.strip()
                        actions_data.append(f"{action} (Кол-во: {count}, Сумма: {amount})")
                    except:
                        continue
                
                temp_df["Юридическая работа - Процессуальные действия"] = [json.dumps(actions_data, ensure_ascii=False)]
            except:
                pass
            
            temp_df["URL"] = url
            
            return temp_df
            
        except Exception as e:
            print(f"Ошибка парсинга юридической работы: {str(e)}")
            return pd.DataFrame()
    
    def _parse_payments_to_creditors(self, url):
        """Парсинг выплат кредиторам и добавление в общий DataFrame"""
        try:
            payments_section = self.driver.find_element(By.XPATH, "//div[contains(., 'Выплачено кредиторам')]")
            self.driver.execute_script("arguments[0].scrollIntoView(true);", payments_section)
            time.sleep(2)
            
            temp_df = pd.DataFrame()
            
            try:
                rows = self.driver.find_elements(By.CSS_SELECTOR, "div.table-list__row_depth-0")
                payments_data = []
                
                for row in rows:
                    try:
                        if ('table-list__row_header' in row.get_attribute('class') or 
                            'table-list__row_footer' in row.get_attribute('class')):
                            continue
                        
                        title = row.find_element(By.CSS_SELECTOR, "span.table-list__cell-title-text").text.strip()
                        cells = row.find_elements(By.CSS_SELECTOR, "div.table-list__cell-value")
                        
                        if len(cells) >= 3:
                            payments_data.append(
                                f"{title}: Установлено {cells[0].text.strip()}, "
                                f"Удовлетворено {cells[1].text.strip()}, "
                                f"Отступное {cells[2].text.strip()}"
                            )
                    except:
                        continue
                
                temp_df["Выплаты кредиторам - Данные"] = [json.dumps(payments_data, ensure_ascii=False)]
            except:
                pass
            
            try:
                total_row = self.driver.find_element(By.CSS_SELECTOR, "div.table-list__row_footer")
                cells = total_row.find_elements(By.CSS_SELECTOR, "div.table-list__cell-value")
                
                if len(cells) >= 3:
                    temp_df["Выплаты кредиторам - Итого"] = [
                        f"Установлено: {cells[0].text.strip()}, "
                        f"Удовлетворено: {cells[1].text.strip()}, "
                        f"Отступное: {cells[2].text.strip()}"
                    ]
            except:
                pass
            
            temp_df["URL"] = url
            
            return temp_df
            
        except Exception as e:
            print(f"Ошибка при парсинге выплат кредиторам: {str(e)}")
            return pd.DataFrame()
    
    def scrape_bank(self, url):
        """Сбор всех данных по одному банку и объединение в один DataFrame"""
        print(f"\nПарсинг банка: {url}")
        self.driver.get(url)
        time.sleep(5)
        
        self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
        self.driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(5)
        
        bank_info = self._parse_bank_info(url)
        insurance = self._parse_insurance_payments()
        competitive_mass = self._parse_competitive_mass(url)
        expenses = self._parse_expenses(url)
        legal_work = self._parse_legal_work(url)
        payments = self._parse_payments_to_creditors(url)
        
        bank_df = pd.DataFrame([bank_info])
        
        bank_df = bank_df.merge(pd.DataFrame([insurance]), how='left', left_index=True, right_index=True)
        bank_df = bank_df.merge(competitive_mass, how='left', on='URL')
        bank_df = bank_df.merge(expenses, how='left', on='URL')
        bank_df = bank_df.merge(legal_work, how='left', on='URL')
        bank_df = bank_df.merge(payments, how='left', on='URL')
        
        self.combined_df = pd.concat([self.combined_df, bank_df], ignore_index=True)
        
        print(f"Данные по банку {url} добавлены в общий DataFrame")
        return bank_df
    
    def scrape_all_banks(self, bank_urls):
        """Сбор данных по всем банкам"""
        for url in bank_urls:
            try:
                self.scrape_bank(url)
                time.sleep(3)
            except Exception as e:
                print(f"Ошибка при обработке банка {url}: {str(e)}")
                continue
        
        self.combined_df.fillna("Нет данных", inplace=True)
        
        json_columns = [
            "Конкурсная масса - Динамика",
            "Конкурсная масса - Имущество",
            "Конкурсная масса - Сформированная масса",
            "Конкурсная масса - Реализация имущества",
            "Расходы - Динамика",
            "Расходы - Статьи",
            "Юридическая работа - Процессуальные действия",
            "Выплаты кредиторам - Данные"
        ]
        
        for col in json_columns:
            if col in self.combined_df.columns:
                self.combined_df[col] = self.combined_df[col].apply(
                    lambda x: "\n".join(json.loads(x)) if x != "Нет данных" else x
                )
        
        return self.combined_df
    
    def save_to_excel(self, filename="all_banks_combined.xlsx"):
        """Сохранение объединенного DataFrame в Excel"""
        if not self.combined_df.empty:
            self.combined_df.to_excel(filename, index=False, encoding='utf-8-sig')
            print(f"\nВсе данные сохранены в файл {filename}")
        else:
            print("\nНет данных для сохранения")
    
    def close(self):
        """Закрытие драйвера"""
        self.driver.quit()

if __name__ == "__main__":
    bank_urls = links_2
    
    scraper = UnifiedBankDataScraper()
    try:
        combined_data_2 = scraper.scrape_all_banks(bank_urls)
        
        print("\nСоздан объединенный DataFrame со следующими столбцами:")
        print(combined_data_2.columns.tolist())
        print(f"\nВсего записей: {len(combined_data_2)}")
        
        scraper.save_to_excel("bank_data.xlsx")
        
        print("\nПервые строки данных:")
        print(combined_data_2.head())
        
    except Exception as e:
        print(f"Произошла ошибка: {str(e)}")
    finally:
        scraper.close()


Парсинг банка: https://www.asv.org.ru/banks/ooo-kb-gefest
Данные по банку https://www.asv.org.ru/banks/ooo-kb-gefest добавлены в общий DataFrame

Парсинг банка: https://www.asv.org.ru/banks/grinkombank-ao
Данные по банку https://www.asv.org.ru/banks/grinkombank-ao добавлены в общий DataFrame

Парсинг банка: https://www.asv.org.ru/banks/grinfildbank-ao
Данные по банку https://www.asv.org.ru/banks/grinfildbank-ao добавлены в общий DataFrame

Парсинг банка: https://www.asv.org.ru/banks/dagenergobank-ooo
Данные по банку https://www.asv.org.ru/banks/dagenergobank-ooo добавлены в общий DataFrame

Парсинг банка: https://www.asv.org.ru/banks/dig-bank-iab-oao
Данные по банку https://www.asv.org.ru/banks/dig-bank-iab-oao добавлены в общий DataFrame

Парсинг банка: https://www.asv.org.ru/banks/dil-bank-ooo
Данные по банку https://www.asv.org.ru/banks/dil-bank-ooo добавлены в общий DataFrame

Парсинг банка: https://www.asv.org.ru/banks/doninvest-kb-ooo
Данные по банку https://www.asv.org.ru/banks

In [112]:
combined_data_2

,URL,Дата парсинга,Полное фирменное наименование,Сокращенное фирменное наименование,Место нахождения,Почтовый адрес,ОГРН,Номер в Книге государственной регистрации кредитных организаций,Номер по реестру,Дата включения в реестр,...,Конкурсная масса - Сформированная масса,Конкурсная масса - Реализация имущества,Расходы - Общая сумма,Расходы - Дата обновления,Расходы - Динамика,Юридическая работа - Дата обновления,Юридическая работа - Процессуальные действия,Выплаты кредиторам - Данные,Выплаты кредиторам - Итого,Примечание
0,https://www.asv.org.ru/banks/ooo-kb-gefest,2025-06-19 14:57:31,Общество с ограниченной ответственностью Комме...,"ООО КБ ""Гефест""","171506, Тверская область, г.Кимры, ул.Урицкого...","109992, Москва, «Москва ГСП-2» – ликвидатор ОО...",1026900001347,1046,132,04.11.2004,...,Денежные средства на дату открытия конкурсного...,Стоимость реализованного имущества: 68\nПоступ...,15 612 тыс. руб.,Обновлено 01.05.2025,янв: 230 \nфев: 237 \nмар: 542 \nапр: 1872 \n2...,Обновлено 01.05.2025,"Предъявлено исков (Кол-во: 26, Сумма: 405 995)...","1-я очередь (в реестре): Установлено 92 129, У...","Установлено: 221 159, Удовлетворено: 221 159, ...",Нет данных
1,https://www.asv.org.ru/banks/grinkombank-ao,2025-06-19 14:58:35,"Акционерное общество ""ГринКомБанк""","АО ""ГринКомБанк""","664007, Иркутская область, г. Иркутск, ул. Фра...","109992, г. Москва, ГСП-2, ГК «АСВ» – конкурсны...",1023800000190,1184,125,28.10.2004,...,Денежные средства на дату открытия конкурсного...,Стоимость реализованного имущества: 0\nПоступи...,114 323 тыс. руб.,Обновлено 01.05.2025,янв: 529 \nфев: 710 \nмар: 590 \nапр: 882 \n20...,Обновлено 01.05.2025,"Предъявлено исков (Кол-во: 42, Сумма: 2 466 31...","1-я очередь (в реестре): Установлено 80 377, У...","Установлено: 373 659, Удовлетворено: 80 377, О...",Нет данных
2,https://www.asv.org.ru/banks/grinfildbank-ao,2025-06-19 14:59:40,"Акционерный коммерческий банк ""Гринфилд"" (Акци...","АО ""Гринфилдбанк""","107045, г. Москва, Малый Головин переулок, д.8","109992, г. Москва, ГСП-2, ГК «АСВ» – конкурсны...",1027700314113,2711,693,24.02.2005,...,Денежные средства на дату открытия конкурсного...,Стоимость реализованного имущества: 65 684\nПо...,491 327 тыс. руб.,Обновлено 01.05.2025,янв: 385 \nфев: 727 \nмар: 574 \nапр: 839 \n20...,Обновлено 01.05.2025,"Предъявлено исков (Кол-во: 1 712, Сумма: 51 21...",1-я очередь (в реестре): Установлено 6 296 220...,"Установлено: 6 748 763, Удовлетворено: 895 693...",Нет данных
3,https://www.asv.org.ru/banks/dagenergobank-ooo,2025-06-19 15:00:45,Дагестанский коммерческий энергетический банк ...,"ООО ""Дагэнергобанк""","367000, Республика Дагестан, г.Махачкала, пр.Р...","367009, г. Махачкала, ул. Каммаева, д. 19ж",1020500000553,3286,205,18.11.2004,...,Денежные средства на дату открытия конкурсного...,Стоимость реализованного имущества: 71 410\nПо...,350 521 тыс. руб.,Обновлено 01.05.2025,янв: 878 \nфев: 867 \nмар: 860 \nапр: 957 \n20...,Обновлено 01.05.2025,"Предъявлено исков (Кол-во: 447, Сумма: 9 335 4...",1-я очередь (в реестре): Установлено 2 380 657...,"Установлено: 2 701 653, Удовлетворено: 1 394 6...",Нет данных
4,https://www.asv.org.ru/banks/dig-bank-iab-oao,2025-06-19 15:01:51,Открытое акционерное общество Инвестиционный А...,"ИАБ ""Диг-Банк"" (ОАО)","362003, Республика Северная Осетия-Алания, г. ...","362001, г. Владикавказ, ул. Гагарина, д. 30",1021500000279,2423,968,06.08.2008,...,Денежные средства на дату открытия конкурсного...,Стоимость реализованного имущества: 0\nПоступи...,119 665 тыс. руб.,Обновлено 01.05.2025,янв: 24 \nфев: 137 \nмар: 140 \nапр: 63 \n2014...,Обновлено 01.05.2025,"Предъявлено исков (Кол-во: 213, Сумма: 6 859 0...",1-я очередь (в реестре): Установлено 2 206 339...,"Установлено: 3 092 953, Удовлетворено: 63 766,...",Нет данных
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,https://www.asv.org.ru/banks/mab-zao,2025-06-19 16:04:46,"Закрытое акционерное общество ""Международный а...","ЗАО ""МАБ""

In [120]:
combined_data_2.to_csv('combined_data_2', encoding= ' utf-8')

In [116]:
"""Эта программа работает примерно 70 минуты"""

class UnifiedBankDataScraper:
    def __init__(self):
        self.driver = self._setup_driver()
        self.combined_df = pd.DataFrame()
        
    def _setup_driver(self):
        chrome_options = Options()
        chrome_options.add_argument('--disable-gpu')
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-dev-shm-usage')
        driver = webdriver.Chrome(options=chrome_options)
        driver.implicitly_wait(5)
        return driver
    
    def _scroll_to_element(self, element):
        actions = ActionChains(self.driver)
        actions.move_to_element(element).perform()
        time.sleep(5)
    
    def _expand_section(self, section):
        if 'card-foldable_open' not in section.get_attribute('class'):
            header = section.find_element(By.CSS_SELECTOR, 'div.card-foldable__header')
            header.click()
            time.sleep(5)
    
    def _parse_bank_info(self, url):
        """Парсинг основной информации о банке"""
        bank_info = {"URL": url, "Дата парсинга": datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
        
        try:
            ul_element = WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div.foldable__body > div > app-company-block-requisites-table > ul'))
            )
            
            for li in ul_element.find_elements(By.TAG_NAME, "li"):
                try:
                    title = li.find_element(By.CSS_SELECTOR, 'div.requisite-item__title > div').text.strip()
                    value = li.find_element(By.CSS_SELECTOR, 'div.requisite-item__value > div').text.strip()
                    bank_info[title] = value
                except:
                    continue
        except Exception as e:
            print(f"Ошибка парсинга информации о банке: {str(e)}")
        
        return bank_info
    
    def _parse_insurance_payments(self):
        """Парсинг страховых выплат"""
        payments = {}
        
        try:
            insurance_element = self.driver.find_element(By.CSS_SELECTOR, 'app-insurance-payments')
            
            paid = insurance_element.find_element(By.XPATH, ".//div[@class='col-24 col-md-auto'][1]//span/nobr").text.strip()
            remaining = insurance_element.find_element(By.XPATH, ".//div[@class='col-24 col-md-auto mt-4 mt-md-0'][1]//span/nobr").text.strip()
            percentage = insurance_element.find_element(By.CSS_SELECTOR, 'div.progress-performed-number').text.strip()
            
            payments.update({
                "Страховые выплаты - Выплачено": paid,
                "Страховые выплаты - Осталось выплатить": remaining,
                "Страховые выплаты - Процент выплаченного": percentage
            })
        except Exception as e:
            print(f"Ошибка парсинга страховых выплат: {str(e)}")
        
        return payments
    
    def _parse_competitive_mass(self, url):
        """Парсинг конкурсной массы и добавление в общий DataFrame"""
        try:
            sections = WebDriverWait(self.driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.card-foldable")))
            
            target_section = None
            for section in sections:
                try:
                    title = section.find_element(By.CSS_SELECTOR, "div.card-foldable-title__main-text").text
                    if "формировании конкурсной массы" in title.lower():
                        target_section = section
                        break
                except:
                    continue
            
            if not target_section:
                return pd.DataFrame()
            
            self._expand_section(target_section)
            self._scroll_to_element(target_section)
            time.sleep(2)
            
            temp_df = pd.DataFrame()
            
            try:
                total = target_section.find_element(By.CSS_SELECTOR, "div.card-foldable-title__secondary-text").text.strip()
                temp_df["Конкурсная масса - Общая сумма"] = [total]
            except:
                pass
            
            try:
                diagrams = target_section.find_elements(By.CSS_SELECTOR, "app-block-page-kp-diagram")
                dynamics = []
                for diagram in diagrams:
                    bars = diagram.find_elements(By.CSS_SELECTOR, "div.diagram__bar-wrapper")
                    for bar in bars:
                        try:
                            period = bar.find_element(By.CSS_SELECTOR, "div.diagram__month").text.strip()
                            amount = bar.find_element(By.CSS_SELECTOR, "div.diagram__percent span").text.strip()
                            currency = bar.find_elements(By.CSS_SELECTOR, "div.diagram__percent span")[1].text.strip()
                            dynamics.append(f"{period}: {amount} {currency}")
                        except:
                            continue
                temp_df["Конкурсная масса - Динамика"] = [json.dumps(dynamics, ensure_ascii=False)]
            except:
                pass
            
            try:
                tables = target_section.find_elements(By.CSS_SELECTOR, "div.card-body_foldable-card")
                for table in tables:
                    try:
                        header = table.find_element(By.CSS_SELECTOR, "span.body-bold").text
                        rows = table.find_elements(By.CSS_SELECTOR, "div.table-list__row_depth-0")
                        
                        table_data = []
                        for row in rows:
                            try:
                                if 'table-list__row_header' in row.get_attribute('class') or 'table-list__row_footer' in row.get_attribute('class'):
                                    continue
                                
                                title = row.find_element(By.CSS_SELECTOR, "span.table-list__cell-title-text").text.strip()
                                value = row.find_element(By.CSS_SELECTOR, "span.value").text.strip()
                                table_data.append(f"{title}: {value}")
                            except:
                                continue
                        
                        if "Имущество (активы)" in header:
                            temp_df["Конкурсная масса - Имущество"] = [json.dumps(table_data, ensure_ascii=False)]
                        elif "Сформированная конкурсная масса" in header:
                            temp_df["Конкурсная масса - Сформированная масса"] = [json.dumps(table_data, ensure_ascii=False)]
                        elif "Реализация имущества" in header:
                            temp_df["Конкурсная масса - Реализация имущества"] = [json.dumps(table_data, ensure_ascii=False)]
                    except:
                        continue
            except:
                pass
            
            temp_df["URL"] = url
            
            return temp_df
            
        except Exception as e:
            print(f"Ошибка парсинга конкурсной массы: {str(e)}")
            return pd.DataFrame()
    
    def _parse_expenses(self, url):
        """Парсинг расходов и добавление в общий DataFrame"""
        try:
            sections = WebDriverWait(self.driver, 20).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.card-foldable")))
            
            target_section = None
            for section in sections:
                try:
                    title = section.find_element(By.CSS_SELECTOR, "div.card-foldable-title__main-text").text
                    if "Расходы" in title:
                        target_section = section
                        break
                except:
                    continue
            
            if not target_section:
                return pd.DataFrame()
            
            self._expand_section(target_section)
            self._scroll_to_element(target_section)
            time.sleep(2)
            
            temp_df = pd.DataFrame()
            
            try:
                total = target_section.find_element(By.CSS_SELECTOR, "div.card-foldable-title__secondary-text").text.strip()
                temp_df["Расходы - Общая сумма"] = [total]
            except:
                pass
            
            try:
                date = target_section.find_element(By.CSS_SELECTOR, "div.card-foldable__desc").text.strip()
                temp_df["Расходы - Дата обновления"] = [date]
            except:
                pass
            
            try:
                diagrams = target_section.find_elements(By.CSS_SELECTOR, "app-block-page-kp-diagram")
                dynamics = []
                for diagram in diagrams:
                    bars = diagram.find_elements(By.CSS_SELECTOR, "div.diagram__bar-wrapper")
                    for bar in bars:
                        try:
                            period = bar.find_element(By.CSS_SELECTOR, "div.diagram__month").text.strip()
                            amount = bar.find_element(By.CSS_SELECTOR, "div.diagram__percent span").text.strip()
                            currency = bar.find_elements(By.CSS_SELECTOR, "div.diagram__percent span")[1].text.strip()
                            dynamics.append(f"{period}: {amount} {currency}")
                        except:
                            continue
                temp_df["Расходы - Динамика"] = [json.dumps(dynamics, ensure_ascii=False)]
            except:
                pass
            
            try:
                expenses_header = target_section.find_element(By.XPATH, ".//span[contains(text(), 'Наименования расходов')]")
                expenses_table = expenses_header.find_element(By.XPATH, "./following-sibling::div[contains(@class, 'table-expenses-list')]")
                
                expenses_data = []
                rows = expenses_table.find_elements(By.CSS_SELECTOR, "app-table-statistic-row")
                for row in rows:
                    try:
                        title = row.find_element(By.CSS_SELECTOR, "span.table-list__cell-title-text").text.strip()
                        value = row.find_element(By.CSS_SELECTOR, "span.value").text.strip()
                        expenses_data.append(f"{title}: {value}")
                    except:
                        continue
                
                temp_df["Расходы - Статьи"] = [json.dumps(expenses_data, ensure_ascii=False)]
            except:
                pass
            
            temp_df["URL"] = url
            
            return temp_df
            
        except Exception as e:
            print(f"Ошибка парсинга расходов: {str(e)}")
            return pd.DataFrame()
    
    def _parse_legal_work(self, url):
        """Парсинг юридической работы и добавление в общий DataFrame"""
        try:
            sections = WebDriverWait(self.driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.card-foldable")))
            
            target_section = None
            for section in sections:
                try:
                    title = section.find_element(By.CSS_SELECTOR, "div.card-foldable-title__main-text").text
                    if "Юридическая работа" in title:
                        target_section = section
                        break
                except:
                    continue
            
            if not target_section:
                return pd.DataFrame()
            
            self._expand_section(target_section)
            self._scroll_to_element(target_section)
            time.sleep(2)
            
            temp_df = pd.DataFrame()
            
            try:
                date = target_section.find_element(By.CSS_SELECTOR, "div.card-foldable__desc").text.strip()
                temp_df["Юридическая работа - Дата обновления"] = [date]
            except:
                pass
            
            try:
                table = target_section.find_element(By.CSS_SELECTOR, "div.table-list")
                actions_data = []
                rows = table.find_elements(By.CSS_SELECTOR, "div.table-list__row.ng-star-inserted")
                
                for row in rows:
                    try:
                        action = row.find_element(By.CSS_SELECTOR, "span.tooltip").text.strip()
                        count = row.find_element(By.CSS_SELECTOR, "div.table-list__sub-cell.second-value").text.strip()
                        amount = row.find_element(By.CSS_SELECTOR, "div.table-list__cell_number nobr").text.strip()
                        actions_data.append(f"{action} (Кол-во: {count}, Сумма: {amount})")
                    except:
                        continue
                
                temp_df["Юридическая работа - Процессуальные действия"] = [json.dumps(actions_data, ensure_ascii=False)]
            except:
                pass
            
            temp_df["URL"] = url
            
            return temp_df
            
        except Exception as e:
            print(f"Ошибка парсинга юридической работы: {str(e)}")
            return pd.DataFrame()
    
    def _parse_payments_to_creditors(self, url):
        """Парсинг выплат кредиторам и добавление в общий DataFrame"""
        try:
            payments_section = self.driver.find_element(By.XPATH, "//div[contains(., 'Выплачено кредиторам')]")
            self.driver.execute_script("arguments[0].scrollIntoView(true);", payments_section)
            time.sleep(2)
            
            temp_df = pd.DataFrame()
            
            try:
                rows = self.driver.find_elements(By.CSS_SELECTOR, "div.table-list__row_depth-0")
                payments_data = []
                
                for row in rows:
                    try:
                        if ('table-list__row_header' in row.get_attribute('class') or 
                            'table-list__row_footer' in row.get_attribute('class')):
                            continue
                        
                        title = row.find_element(By.CSS_SELECTOR, "span.table-list__cell-title-text").text.strip()
                        cells = row.find_elements(By.CSS_SELECTOR, "div.table-list__cell-value")
                        
                        if len(cells) >= 3:
                            payments_data.append(
                                f"{title}: Установлено {cells[0].text.strip()}, "
                                f"Удовлетворено {cells[1].text.strip()}, "
                                f"Отступное {cells[2].text.strip()}"
                            )
                    except:
                        continue
                
                temp_df["Выплаты кредиторам - Данные"] = [json.dumps(payments_data, ensure_ascii=False)]
            except:
                pass
            
            try:
                total_row = self.driver.find_element(By.CSS_SELECTOR, "div.table-list__row_footer")
                cells = total_row.find_elements(By.CSS_SELECTOR, "div.table-list__cell-value")
                
                if len(cells) >= 3:
                    temp_df["Выплаты кредиторам - Итого"] = [
                        f"Установлено: {cells[0].text.strip()}, "
                        f"Удовлетворено: {cells[1].text.strip()}, "
                        f"Отступное: {cells[2].text.strip()}"
                    ]
            except:
                pass
            
            temp_df["URL"] = url
            
            return temp_df
            
        except Exception as e:
            print(f"Ошибка при парсинге выплат кредиторам: {str(e)}")
            return pd.DataFrame()
    
    def scrape_bank(self, url):
        """Сбор всех данных по одному банку и объединение в один DataFrame"""
        print(f"\nПарсинг банка: {url}")
        self.driver.get(url)
        time.sleep(5)
        
        self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
        self.driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(5)
        
        bank_info = self._parse_bank_info(url)
        insurance = self._parse_insurance_payments()
        competitive_mass = self._parse_competitive_mass(url)
        expenses = self._parse_expenses(url)
        legal_work = self._parse_legal_work(url)
        payments = self._parse_payments_to_creditors(url)
        
        bank_df = pd.DataFrame([bank_info])
        
        bank_df = bank_df.merge(pd.DataFrame([insurance]), how='left', left_index=True, right_index=True)
        bank_df = bank_df.merge(competitive_mass, how='left', on='URL')
        bank_df = bank_df.merge(expenses, how='left', on='URL')
        bank_df = bank_df.merge(legal_work, how='left', on='URL')
        bank_df = bank_df.merge(payments, how='left', on='URL')
        
        self.combined_df = pd.concat([self.combined_df, bank_df], ignore_index=True)
        
        print(f"Данные по банку {url} добавлены в общий DataFrame")
        return bank_df
    
    def scrape_all_banks(self, bank_urls):
        """Сбор данных по всем банкам"""
        for url in bank_urls:
            try:
                self.scrape_bank(url)
                time.sleep(3)
            except Exception as e:
                print(f"Ошибка при обработке банка {url}: {str(e)}")
                continue
        
        self.combined_df.fillna("Нет данных", inplace=True)
        
        json_columns = [
            "Конкурсная масса - Динамика",
            "Конкурсная масса - Имущество",
            "Конкурсная масса - Сформированная масса",
            "Конкурсная масса - Реализация имущества",
            "Расходы - Динамика",
            "Расходы - Статьи",
            "Юридическая работа - Процессуальные действия",
            "Выплаты кредиторам - Данные"
        ]
        
        for col in json_columns:
            if col in self.combined_df.columns:
                self.combined_df[col] = self.combined_df[col].apply(
                    lambda x: "\n".join(json.loads(x)) if x != "Нет данных" else x
                )
        
        return self.combined_df
    
    def save_to_excel(self, filename="all_banks_combined.xlsx"):
        """Сохранение объединенного DataFrame в Excel"""
        if not self.combined_df.empty:
            self.combined_df.to_excel(filename, index=False, encoding='utf-8-sig')
            print(f"\nВсе данные сохранены в файл {filename}")
        else:
            print("\nНет данных для сохранения")
    
    def close(self):
        """Закрытие драйвера"""
        self.driver.quit()

if __name__ == "__main__":
    bank_urls = links_3
    
    scraper = UnifiedBankDataScraper()
    try:
        combined_data_3 = scraper.scrape_all_banks(bank_urls)
        
        print("\nСоздан объединенный DataFrame со следующими столбцами:")
        print(combined_data_3.columns.tolist())
        print(f"\nВсего записей: {len(combined_data_3)}")
        
        scraper.save_to_excel("bank_data.xlsx")
        
        print("\nПервые строки данных:")
        print(combined_data_3.head())
        
    except Exception as e:
        print(f"Произошла ошибка: {str(e)}")
    finally:
        scraper.close()


Парсинг банка: https://www.asv.org.ru/banks/mbsp-bank-ao
Данные по банку https://www.asv.org.ru/banks/mbsp-bank-ao добавлены в общий DataFrame

Парсинг банка: https://www.asv.org.ru/banks/mbfi-bank-oao
Данные по банку https://www.asv.org.ru/banks/mbfi-bank-oao добавлены в общий DataFrame

Парсинг банка: https://www.asv.org.ru/banks/mvkb-kb-oao
Данные по банку https://www.asv.org.ru/banks/mvkb-kb-oao добавлены в общий DataFrame

Парсинг банка: https://www.asv.org.ru/banks/mgb-rnko-ooo
Ошибка парсинга страховых выплат: Message: no such element: Unable to locate element: {"method":"css selector","selector":"app-insurance-payments"}
  (Session info: chrome=137.0.7151.104); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x0x7ff7eb6ccda5+78885]
	GetHandleVerifier [0x0x7ff7eb6cce00+78976]
	(No symbol) [0x0x7ff7eb489bca]
	(No symbol) [0x0x7ff7eb4e0766]
	(No symbol)

In [117]:
combined_data_3

,URL,Дата парсинга,Полное фирменное наименование,Сокращенное фирменное наименование,Место нахождения,Почтовый адрес,ОГРН,Номер в Книге государственной регистрации кредитных организаций,Номер по реестру,Дата включения в реестр,...,Конкурсная масса - Имущество,Конкурсная масса - Сформированная масса,Конкурсная масса - Реализация имущества,Расходы - Общая сумма,Расходы - Дата обновления,Расходы - Динамика,Юридическая работа - Дата обновления,Юридическая работа - Процессуальные действия,Выплаты кредиторам - Данные,Выплаты кредиторам - Итого
0,https://www.asv.org.ru/banks/mbsp-bank-ao,2025-06-20 00:47:14,Международный банк Санкт-Петербурга (Акционерн...,Банк МБСП (АО),"194044, Санкт-Петербург, Крапивный пер., 5","199106, г. Санкт-Петербург, пр-кт Средний В.О....",1027800001547,197,331,16.12.2004,...,Денежные средства и драгоценные металлы: 0\nКо...,Денежные средства на дату открытия конкурсного...,Стоимость реализованного имущества: 9 079\nПос...,1 433 985 тыс. руб.,Обновлено 01.05.2025,янв: 2204 \nфев: 9266 \nмар: 12 354 \nапр: 23 ...,Обновлено 01.05.2025,"Предъявлено исков (Кол-во: 178, Сумма: 96 145 ...",1-я очередь (в реестре): Установлено 17 571 12...,"Установлено: 23 454 750, Удовлетворено: 8 025 ..."
1,https://www.asv.org.ru/banks/mbfi-bank-oao,2025-06-20 00:48:14,"ОТКРЫТОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ""МЕЖДУНАРОДНЫЙ Б...","ОАО БАНК ""МБФИ""","105062, г. Москва, Подсосенский пер, д. 17","109992, г. Москва, ГСП-2, ГК «АСВ» – конкурсны...",1020500000510,2073,987,28.12.2010,...,Денежные средства и драгоценные металлы: 0\nКо...,Денежные средства на дату открытия конкурсного...,Стоимость реализованного имущества: 0\nПоступи...,19 739 тыс. руб.,Обновлено 01.05.2025,янв: 43 \nфев: 21 \nмар: 6 \nапр: 145 \n2016: ...,Обновлено 01.05.2025,"Предъявлено исков (Кол-во: 35, Сумма: 2 553 34...","1-я очередь (в реестре): Установлено 270 727, ...","Установлено: 279 694, Удовлетворено: 2 680, От..."
2,https://www.asv.org.ru/banks/mvkb-kb-oao,2025-06-20 00:49:15,Открытое акционерное общество Коммерческий Бан...,"ОАО КБ ""МВКБ""","105062, г. Москва, Подсосенский переулок, д. 2...","109992, г. Москва, ГСП-2, ГК «АСВ» – конкурсны...",1027744003100,2863,413,11.01.2005,...,Денежные средства и драгоценные металлы: 0\nКо...,Денежные средства на дату открытия конкурсного...,Стоимость реализованного имущества: 0\nПоступи...,126 654 тыс. руб.,Обновлено 01.05.2025,янв: 0 \nфев: 0 \nмар: 0 \nапр: 0 \n2016: 34 1...,Обновлено 01.05.2025,"Предъявлено исков (Кол-во: 53, Сумма: 4 596 77...",1-я очередь (в реестре): Установлено 1 054 679...,"Установлено: 1 197 851, Удовлетворено: 274 026..."
3,https://www.asv.org.ru/banks/ooo-kb-megapolis,2025-06-20 00:53:41,Общество с ограниченной ответственностью Комме...,"ООО КБ ""Мегаполис""","428032, Чувашская Республика-Чувашия, г.Чебокс...","109992, г. Москва, ГСП-2, ГК «АСВ» – конкурсны...",1022100000317,3265,352,23.12.2004,...,Денежные средства и драгоценные металлы: 0\nКо...,Денежные средства на дату открытия конкурсного...,Стоимость реализованного имущества: 3 349\nПос...,262 758 тыс. руб.,Обновлено 01.05.2025,янв: 1624 \nфев: 3320 \nмар: 5750 \nапр: 2428 ...,Обновлено 01.05.2025,"Предъявлено исков (Кол-во: 568, Сумма: 14 150 ...",1-я очередь (в реестре): Установлено 2 554 660...,"Установлено: 4 094 496, Удовлетворено: 1 972 6..."
4,https://www.asv.org.ru/banks/mezhdunarodnyy-ba...,2025-06-20 00:55:45,"Коммерческий Банк ""Международный Банк Развития...","КБ ""Международный Банк Развития"" (АО)","123610, г. Москва, Краснопресненская набережна...","109992, г. Москва, ГСП-2, ГК «АСВ» – конкурсны...",1027739378600,2704,750,03.03.2005,...,Денежные средства и драгоценные металлы: 0\nКо...,Денежные средства на дату открытия конкурсного...,Стоимость реализованного имущества: 0\nПоступи...,63 750 тыс. руб.,Обновлено 01.05.2025,янв: 160 \nфев: 134 \nмар: 152 \nапр: 103 \n20...,Обновлено 01.05.2025,"Предъявлено исков (Кол-во: 80, Сумма: 9 733 00...","1-я очередь (в реестре): Установлено 342 918, ...","Установлено: 860 635, Удовлетворе

In [119]:
combined_data_3.to_csv('combined_data_3', encoding= ' utf-8')

In [122]:
"""Эта программа работает примерно 88 минут"""

class UnifiedBankDataScraper:
    def __init__(self):
        self.driver = self._setup_driver()
        self.combined_df = pd.DataFrame()
        
    def _setup_driver(self):
        chrome_options = Options()
        chrome_options.add_argument('--disable-gpu')
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-dev-shm-usage')
        driver = webdriver.Chrome(options=chrome_options)
        driver.implicitly_wait(5)
        return driver
    
    def _scroll_to_element(self, element):
        actions = ActionChains(self.driver)
        actions.move_to_element(element).perform()
        time.sleep(5)
    
    def _expand_section(self, section):
        if 'card-foldable_open' not in section.get_attribute('class'):
            header = section.find_element(By.CSS_SELECTOR, 'div.card-foldable__header')
            header.click()
            time.sleep(5)
    
    def _parse_bank_info(self, url):
        """Парсинг основной информации о банке"""
        bank_info = {"URL": url, "Дата парсинга": datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
        
        try:
            ul_element = WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div.foldable__body > div > app-company-block-requisites-table > ul'))
            )
            
            for li in ul_element.find_elements(By.TAG_NAME, "li"):
                try:
                    title = li.find_element(By.CSS_SELECTOR, 'div.requisite-item__title > div').text.strip()
                    value = li.find_element(By.CSS_SELECTOR, 'div.requisite-item__value > div').text.strip()
                    bank_info[title] = value
                except:
                    continue
        except Exception as e:
            print(f"Ошибка парсинга информации о банке: {str(e)}")
        
        return bank_info
    
    def _parse_insurance_payments(self):
        """Парсинг страховых выплат"""
        payments = {}
        
        try:
            insurance_element = self.driver.find_element(By.CSS_SELECTOR, 'app-insurance-payments')
            
            paid = insurance_element.find_element(By.XPATH, ".//div[@class='col-24 col-md-auto'][1]//span/nobr").text.strip()
            remaining = insurance_element.find_element(By.XPATH, ".//div[@class='col-24 col-md-auto mt-4 mt-md-0'][1]//span/nobr").text.strip()
            percentage = insurance_element.find_element(By.CSS_SELECTOR, 'div.progress-performed-number').text.strip()
            
            payments.update({
                "Страховые выплаты - Выплачено": paid,
                "Страховые выплаты - Осталось выплатить": remaining,
                "Страховые выплаты - Процент выплаченного": percentage
            })
        except Exception as e:
            print(f"Ошибка парсинга страховых выплат: {str(e)}")
        
        return payments
    
    def _parse_competitive_mass(self, url):
        """Парсинг конкурсной массы и добавление в общий DataFrame"""
        try:
            sections = WebDriverWait(self.driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.card-foldable")))
            
            target_section = None
            for section in sections:
                try:
                    title = section.find_element(By.CSS_SELECTOR, "div.card-foldable-title__main-text").text
                    if "формировании конкурсной массы" in title.lower():
                        target_section = section
                        break
                except:
                    continue
            
            if not target_section:
                return pd.DataFrame()
            
            self._expand_section(target_section)
            self._scroll_to_element(target_section)
            time.sleep(2)
            
            temp_df = pd.DataFrame()
            
            try:
                total = target_section.find_element(By.CSS_SELECTOR, "div.card-foldable-title__secondary-text").text.strip()
                temp_df["Конкурсная масса - Общая сумма"] = [total]
            except:
                pass
            
            try:
                diagrams = target_section.find_elements(By.CSS_SELECTOR, "app-block-page-kp-diagram")
                dynamics = []
                for diagram in diagrams:
                    bars = diagram.find_elements(By.CSS_SELECTOR, "div.diagram__bar-wrapper")
                    for bar in bars:
                        try:
                            period = bar.find_element(By.CSS_SELECTOR, "div.diagram__month").text.strip()
                            amount = bar.find_element(By.CSS_SELECTOR, "div.diagram__percent span").text.strip()
                            currency = bar.find_elements(By.CSS_SELECTOR, "div.diagram__percent span")[1].text.strip()
                            dynamics.append(f"{period}: {amount} {currency}")
                        except:
                            continue
                temp_df["Конкурсная масса - Динамика"] = [json.dumps(dynamics, ensure_ascii=False)]
            except:
                pass
            
            try:
                tables = target_section.find_elements(By.CSS_SELECTOR, "div.card-body_foldable-card")
                for table in tables:
                    try:
                        header = table.find_element(By.CSS_SELECTOR, "span.body-bold").text
                        rows = table.find_elements(By.CSS_SELECTOR, "div.table-list__row_depth-0")
                        
                        table_data = []
                        for row in rows:
                            try:
                                if 'table-list__row_header' in row.get_attribute('class') or 'table-list__row_footer' in row.get_attribute('class'):
                                    continue
                                
                                title = row.find_element(By.CSS_SELECTOR, "span.table-list__cell-title-text").text.strip()
                                value = row.find_element(By.CSS_SELECTOR, "span.value").text.strip()
                                table_data.append(f"{title}: {value}")
                            except:
                                continue
                        
                        if "Имущество (активы)" in header:
                            temp_df["Конкурсная масса - Имущество"] = [json.dumps(table_data, ensure_ascii=False)]
                        elif "Сформированная конкурсная масса" in header:
                            temp_df["Конкурсная масса - Сформированная масса"] = [json.dumps(table_data, ensure_ascii=False)]
                        elif "Реализация имущества" in header:
                            temp_df["Конкурсная масса - Реализация имущества"] = [json.dumps(table_data, ensure_ascii=False)]
                    except:
                        continue
            except:
                pass
            
            temp_df["URL"] = url
            
            return temp_df
            
        except Exception as e:
            print(f"Ошибка парсинга конкурсной массы: {str(e)}")
            return pd.DataFrame()
    
    def _parse_expenses(self, url):
        """Парсинг расходов и добавление в общий DataFrame"""
        try:
            sections = WebDriverWait(self.driver, 20).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.card-foldable")))
            
            target_section = None
            for section in sections:
                try:
                    title = section.find_element(By.CSS_SELECTOR, "div.card-foldable-title__main-text").text
                    if "Расходы" in title:
                        target_section = section
                        break
                except:
                    continue
            
            if not target_section:
                return pd.DataFrame()
            
            self._expand_section(target_section)
            self._scroll_to_element(target_section)
            time.sleep(2)
            
            temp_df = pd.DataFrame()
            
            try:
                total = target_section.find_element(By.CSS_SELECTOR, "div.card-foldable-title__secondary-text").text.strip()
                temp_df["Расходы - Общая сумма"] = [total]
            except:
                pass
            
            try:
                date = target_section.find_element(By.CSS_SELECTOR, "div.card-foldable__desc").text.strip()
                temp_df["Расходы - Дата обновления"] = [date]
            except:
                pass
            
            try:
                diagrams = target_section.find_elements(By.CSS_SELECTOR, "app-block-page-kp-diagram")
                dynamics = []
                for diagram in diagrams:
                    bars = diagram.find_elements(By.CSS_SELECTOR, "div.diagram__bar-wrapper")
                    for bar in bars:
                        try:
                            period = bar.find_element(By.CSS_SELECTOR, "div.diagram__month").text.strip()
                            amount = bar.find_element(By.CSS_SELECTOR, "div.diagram__percent span").text.strip()
                            currency = bar.find_elements(By.CSS_SELECTOR, "div.diagram__percent span")[1].text.strip()
                            dynamics.append(f"{period}: {amount} {currency}")
                        except:
                            continue
                temp_df["Расходы - Динамика"] = [json.dumps(dynamics, ensure_ascii=False)]
            except:
                pass
            
            try:
                expenses_header = target_section.find_element(By.XPATH, ".//span[contains(text(), 'Наименования расходов')]")
                expenses_table = expenses_header.find_element(By.XPATH, "./following-sibling::div[contains(@class, 'table-expenses-list')]")
                
                expenses_data = []
                rows = expenses_table.find_elements(By.CSS_SELECTOR, "app-table-statistic-row")
                for row in rows:
                    try:
                        title = row.find_element(By.CSS_SELECTOR, "span.table-list__cell-title-text").text.strip()
                        value = row.find_element(By.CSS_SELECTOR, "span.value").text.strip()
                        expenses_data.append(f"{title}: {value}")
                    except:
                        continue
                
                temp_df["Расходы - Статьи"] = [json.dumps(expenses_data, ensure_ascii=False)]
            except:
                pass
            
            temp_df["URL"] = url
            
            return temp_df
            
        except Exception as e:
            print(f"Ошибка парсинга расходов: {str(e)}")
            return pd.DataFrame()
    
    def _parse_legal_work(self, url):
        """Парсинг юридической работы и добавление в общий DataFrame"""
        try:
            sections = WebDriverWait(self.driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.card-foldable")))
            
            target_section = None
            for section in sections:
                try:
                    title = section.find_element(By.CSS_SELECTOR, "div.card-foldable-title__main-text").text
                    if "Юридическая работа" in title:
                        target_section = section
                        break
                except:
                    continue
            
            if not target_section:
                return pd.DataFrame()
            
            self._expand_section(target_section)
            self._scroll_to_element(target_section)
            time.sleep(2)
            
            temp_df = pd.DataFrame()
            
            try:
                date = target_section.find_element(By.CSS_SELECTOR, "div.card-foldable__desc").text.strip()
                temp_df["Юридическая работа - Дата обновления"] = [date]
            except:
                pass
            
            try:
                table = target_section.find_element(By.CSS_SELECTOR, "div.table-list")
                actions_data = []
                rows = table.find_elements(By.CSS_SELECTOR, "div.table-list__row.ng-star-inserted")
                
                for row in rows:
                    try:
                        action = row.find_element(By.CSS_SELECTOR, "span.tooltip").text.strip()
                        count = row.find_element(By.CSS_SELECTOR, "div.table-list__sub-cell.second-value").text.strip()
                        amount = row.find_element(By.CSS_SELECTOR, "div.table-list__cell_number nobr").text.strip()
                        actions_data.append(f"{action} (Кол-во: {count}, Сумма: {amount})")
                    except:
                        continue
                
                temp_df["Юридическая работа - Процессуальные действия"] = [json.dumps(actions_data, ensure_ascii=False)]
            except:
                pass
            
            temp_df["URL"] = url
            
            return temp_df
            
        except Exception as e:
            print(f"Ошибка парсинга юридической работы: {str(e)}")
            return pd.DataFrame()
    
    def _parse_payments_to_creditors(self, url):
        """Парсинг выплат кредиторам и добавление в общий DataFrame"""
        try:
            payments_section = self.driver.find_element(By.XPATH, "//div[contains(., 'Выплачено кредиторам')]")
            self.driver.execute_script("arguments[0].scrollIntoView(true);", payments_section)
            time.sleep(2)
            
            temp_df = pd.DataFrame()
            
            try:
                rows = self.driver.find_elements(By.CSS_SELECTOR, "div.table-list__row_depth-0")
                payments_data = []
                
                for row in rows:
                    try:
                        if ('table-list__row_header' in row.get_attribute('class') or 
                            'table-list__row_footer' in row.get_attribute('class')):
                            continue
                        
                        title = row.find_element(By.CSS_SELECTOR, "span.table-list__cell-title-text").text.strip()
                        cells = row.find_elements(By.CSS_SELECTOR, "div.table-list__cell-value")
                        
                        if len(cells) >= 3:
                            payments_data.append(
                                f"{title}: Установлено {cells[0].text.strip()}, "
                                f"Удовлетворено {cells[1].text.strip()}, "
                                f"Отступное {cells[2].text.strip()}"
                            )
                    except:
                        continue
                
                temp_df["Выплаты кредиторам - Данные"] = [json.dumps(payments_data, ensure_ascii=False)]
            except:
                pass
            
            try:
                total_row = self.driver.find_element(By.CSS_SELECTOR, "div.table-list__row_footer")
                cells = total_row.find_elements(By.CSS_SELECTOR, "div.table-list__cell-value")
                
                if len(cells) >= 3:
                    temp_df["Выплаты кредиторам - Итого"] = [
                        f"Установлено: {cells[0].text.strip()}, "
                        f"Удовлетворено: {cells[1].text.strip()}, "
                        f"Отступное: {cells[2].text.strip()}"
                    ]
            except:
                pass
            
            temp_df["URL"] = url
            
            return temp_df
            
        except Exception as e:
            print(f"Ошибка при парсинге выплат кредиторам: {str(e)}")
            return pd.DataFrame()
    
    def scrape_bank(self, url):
        """Сбор всех данных по одному банку и объединение в один DataFrame"""
        print(f"\nПарсинг банка: {url}")
        self.driver.get(url)
        time.sleep(5)
        
        self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
        self.driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(5)

        bank_info = self._parse_bank_info(url)
        insurance = self._parse_insurance_payments()
        competitive_mass = self._parse_competitive_mass(url)
        expenses = self._parse_expenses(url)
        legal_work = self._parse_legal_work(url)
        payments = self._parse_payments_to_creditors(url)
        
        bank_df = pd.DataFrame([bank_info])
        
        bank_df = bank_df.merge(pd.DataFrame([insurance]), how='left', left_index=True, right_index=True)
        bank_df = bank_df.merge(competitive_mass, how='left', on='URL')
        bank_df = bank_df.merge(expenses, how='left', on='URL')
        bank_df = bank_df.merge(legal_work, how='left', on='URL')
        bank_df = bank_df.merge(payments, how='left', on='URL')
        
        self.combined_df = pd.concat([self.combined_df, bank_df], ignore_index=True)
        
        print(f"Данные по банку {url} добавлены в общий DataFrame")
        return bank_df
    
    def scrape_all_banks(self, bank_urls):
        """Сбор данных по всем банкам"""
        for url in bank_urls:
            try:
                self.scrape_bank(url)
                time.sleep(3)
            except Exception as e:
                print(f"Ошибка при обработке банка {url}: {str(e)}")
                continue

        self.combined_df.fillna("Нет данных", inplace=True)
        
        json_columns = [
            "Конкурсная масса - Динамика",
            "Конкурсная масса - Имущество",
            "Конкурсная масса - Сформированная масса",
            "Конкурсная масса - Реализация имущества",
            "Расходы - Динамика",
            "Расходы - Статьи",
            "Юридическая работа - Процессуальные действия",
            "Выплаты кредиторам - Данные"
        ]
        
        for col in json_columns:
            if col in self.combined_df.columns:
                self.combined_df[col] = self.combined_df[col].apply(
                    lambda x: "\n".join(json.loads(x)) if x != "Нет данных" else x
                )
        
        return self.combined_df
    
    def save_to_excel(self, filename="all_banks_combined.xlsx"):
        """Сохранение объединенного DataFrame в Excel"""
        if not self.combined_df.empty:
            self.combined_df.to_excel(filename, index=False, encoding='utf-8-sig')
            print(f"\nВсе данные сохранены в файл {filename}")
        else:
            print("\nНет данных для сохранения")
    
    def close(self):
        """Закрытие драйвера"""
        self.driver.quit()

if __name__ == "__main__":
    bank_urls = links_4
    
    scraper = UnifiedBankDataScraper()
    try:
        combined_data_4 = scraper.scrape_all_banks(bank_urls)
        
        print("\nСоздан объединенный DataFrame со следующими столбцами:")
        print(combined_data_4.columns.tolist())
        print(f"\nВсего записей: {len(combined_data_4)}")
        
        scraper.save_to_excel("bank_data.xlsx")

        print("\nПервые строки данных:")
        print(combined_data_4.head())
        
    except Exception as e:
        print(f"Произошла ошибка: {str(e)}")
    finally:
        scraper.close()


Парсинг банка: https://www.asv.org.ru/banks/russkiy-trastovyy-bank-akb-ao
Данные по банку https://www.asv.org.ru/banks/russkiy-trastovyy-bank-akb-ao добавлены в общий DataFrame

Парсинг банка: https://www.asv.org.ru/banks/russtroybank-ao
Данные по банку https://www.asv.org.ru/banks/russtroybank-ao добавлены в общий DataFrame

Парсинг банка: https://www.asv.org.ru/banks/reb-kb-ao
Данные по банку https://www.asv.org.ru/banks/reb-kb-ao добавлены в общий DataFrame

Парсинг банка: https://www.asv.org.ru/banks/s-bank-zao
Данные по банку https://www.asv.org.ru/banks/s-bank-zao добавлены в общий DataFrame

Парсинг банка: https://www.asv.org.ru/banks/sb-bank-ooo
Данные по банку https://www.asv.org.ru/banks/sb-bank-ooo добавлены в общий DataFrame

Парсинг банка: https://www.asv.org.ru/banks/sbrr-bank-ooo
Данные по банку https://www.asv.org.ru/banks/sbrr-bank-ooo добавлены в общий DataFrame

Парсинг банка: https://www.asv.org.ru/banks/sakhadaymondbank-kb-ooo
Ошибка парсинга страховых выплат: Mes

In [123]:
combined_data_4

,URL,Дата парсинга,Полное фирменное наименование,Сокращенное фирменное наименование,Место нахождения,Почтовый адрес,ОГРН,Номер в Книге государственной регистрации кредитных организаций,Номер по реестру,Дата включения в реестр,...,Конкурсная масса - Имущество,Конкурсная масса - Сформированная масса,Конкурсная масса - Реализация имущества,Расходы - Общая сумма,Расходы - Дата обновления,Расходы - Динамика,Юридическая работа - Дата обновления,Юридическая работа - Процессуальные действия,Выплаты кредиторам - Данные,Выплаты кредиторам - Итого
0,https://www.asv.org.ru/banks/russkiy-trastovyy...,2025-06-20 02:04:35,"Акционерный Коммерческий Банк ""Русский Трастов...","АКБ ""Русский Трастовый Банк"" (АО)","125009, г. Москва, улица Большая Никитская, до...","109992, г. Москва, ГСП-2, ГК «АСВ» – конкурсны...",1022600000158,1139,70,14.10.2004,...,Денежные средства и драгоценные металлы: 0\nКо...,Денежные средства на дату открытия конкурсного...,Стоимость реализованного имущества: 0\nПоступи...,141 313 тыс. руб.,Обновлено 01.05.2025,янв: 0 \nфев: 0 \nмар: 0 \nапр: 0 \n2016: 33 7...,Обновлено 01.05.2025,"Предъявлено исков (Кол-во: 137, Сумма: 23 044 ...",1-я очередь (в реестре): Установлено 3 024 918...,"Установлено: 3 209 575, Удовлетворено: 212, От..."
1,https://www.asv.org.ru/banks/russtroybank-ao,2025-06-20 02:05:42,"Акционерное общество ""РУССКИЙ СТРОИТЕЛЬНЫЙ БАНК""","АО ""РУССТРОЙБАНК""","109004, г. Москва, Большой Дровяной переулок, ...","109992, г. Москва, ГСП-2, ГК «АСВ» – конкурсны...",1027739327880,3205,607,10.02.2005,...,Денежные средства и драгоценные металлы: 0\nКо...,Денежные средства на дату открытия конкурсного...,Стоимость реализованного имущества: 34 405\nПо...,1 109 968 тыс. руб.,Обновлено 01.05.2025,янв: 1126 \nфев: 3346 \nмар: 3087 \nапр: 2889 ...,Обновлено 01.05.2025,"Предъявлено исков (Кол-во: 856, Сумма: 134 728...",1-я очередь (в реестре): Установлено 22 416 97...,"Установлено: 28 524 091, Удовлетворено: 6 336 ..."
2,https://www.asv.org.ru/banks/reb-kb-ao,2025-06-20 02:06:43,"Коммерческий банк ""РОСЭНЕРГОБАНК"" (акционерное...","КБ ""РЭБ"" (АО)","105062, г. Москва, пер. Подсосенский, д. 30, с...","109992, г. Москва, ГСП-2, ГК «АСВ» – конкурсны...",1027739136622,2211,932,29.09.2005,...,Денежные средства и драгоценные металлы: 0\nКо...,Денежные средства на дату открытия конкурсного...,Стоимость реализованного имущества: 109\nПосту...,1 519 032 тыс. руб.,Обновлено 01.05.2025,янв: 2261 \nфев: 5550 \nмар: 4025 \nапр: 6411 ...,Обновлено 01.05.2025,"Предъявлено исков (Кол-во: 1 650, Сумма: 239 9...",1-я очередь (в реестре): Установлено 35 223 55...,"Установлено: 41 444 437, Удовлетворено: 8 255 ..."
3,https://www.asv.org.ru/banks/s-bank-zao,2025-06-20 02:07:40,Акционерный Коммерческий Банк Сбережений и Кре...,"ЗАО ""С банк""","115054, г. Москва, ул. Новокузнецкая, д. 36/2,...","109992, г. Москва, ГСП-2, ГК «АСВ» – конкурсны...",1037739299685,2581,723,03.03.2005,...,Денежные средства и драгоценные металлы: 0\nКо...,Денежные средства на дату открытия конкурсного...,Стоимость реализованного имущества: 0\nПоступи...,456 582 тыс. руб.,Обновлено 01.05.2025,янв: 423 \nфев: 257 \nмар: 981 \nапр: 535 \n20...,Обновлено 01.05.2025,"Предъявлено исков (Кол-во: 1 969, Сумма: 41 51...",1-я очередь (в реестре): Установлено 6 799 856...,"Установлено: 11 520 091, Удовлетворено: 435 60..."
4,https://www.asv.org.ru/banks/sb-bank-ooo,2025-06-20 02:08:37,"Коммерческий банк ""Судостроительный банк"" (общ...",СБ Банк (ООО),"115035, г. Москва, Раушская наб., д.4/5, стр.1","109992, г. Москва, ГСП-2, ГК «АСВ» – конкурсны...",1027739177091,2999,920,22.09.2005,...,Денежные средства и драгоценные металлы: 0\nКо...,Денежные средства на дату открытия конкурсного...,Стоимость реализованного имущества: 0\nПоступи...,1 346 820 тыс. руб.,Обновлено 01.05.2025,янв: 1316 \nфев: 2162 \nмар: 2485 \nапр: 1577 ...,Обновлено 01.05.2025,"Предъявлено исков (Кол-во: 1 720, Сумма: 200 2...",1-я очередь (в реестре): Установлено 21 918 02...,"Установлено: 35 935 118, Удо

In [124]:
combined_data_4.to_csv('combined_data_4', encoding= ' utf-8')

In [125]:
combined_data = pd.concat([combined_data_1, combined_data_2, combined_data_3, combined_data_4], axis=0)
combined_data = combined_data.reset_index(drop=True)
combined_data

,URL,Дата парсинга,Полное фирменное наименование,Сокращенное фирменное наименование,Место нахождения,Почтовый адрес,ОГРН,Номер в Книге государственной регистрации кредитных организаций,Номер по реестру,Дата включения в реестр,...,Конкурсная масса - Сформированная масса,Конкурсная масса - Реализация имущества,Расходы - Общая сумма,Расходы - Дата обновления,Расходы - Динамика,Юридическая работа - Дата обновления,Юридическая работа - Процессуальные действия,Выплаты кредиторам - Данные,Выплаты кредиторам - Итого,Примечание
0,https://www.asv.org.ru/banks/1bank-akb-pao,2025-06-19 01:30:04,"Акционерный коммерческий банк ""1Банк"" (публичн...","ПАО АКБ ""1Банк""","362040, Республика Северная Осетия-Алания, г.В...","362001, г. Владикавказ, ул. Гагарина, д. 30",1021500000147,2896,985,20.10.2010,...,Денежные средства на дату открытия конкурсного...,Стоимость реализованного имущества: 9\nПоступи...,182 872 тыс. руб.,Обновлено 01.05.2025,янв: 442 \nфев: 343 \nмар: 436 \nапр: 409 \n20...,Обновлено 01.05.2025,"Предъявлено исков (Кол-во: 1 192, Сумма: 7 257...",1-я очередь (в реестре): Установлено 1 244 844...,"Установлено: 1 301 874, Удовлетворено: 82 050,...",NaN
1,https://www.asv.org.ru/banks/ao-avtogradbank,2025-06-19 01:31:04,Акционерное общество «Автоградбанк»,АО «Автоградбанк»,"423831, Республика Татарстан (Татарстан), г. Н...","109992, Москва, «Москва ГСП-2» – ликвидатор АО...",1021600000806,1455,5,21.09.2004,...,Денежные средства на дату открытия конкурсного...,Стоимость реализованного имущества: 196 075\nП...,221 975 тыс. руб.,Обновлено 01.05.2025,янв: 17 439 \nфев: 34 835 \nмар: 6431 \nапр: 2...,Обновлено 01.05.2025,"Предъявлено исков (Кол-во: 303, Сумма: 3 425 2...",1-я очередь (в реестре): Установлено 2 208 780...,"Установлено: 6 222 267, Удовлетворено: 5 623 2...",NaN
2,https://www.asv.org.ru/banks/agroinkombank-pao,2025-06-19 01:32:03,Публичное акционерное общество Агроинвестицион...,ПАО АГРОИНКОМБАНК,"414000, город Астрахань, пл. Ленина/ул. Бурова...","109992, г. Москва, ГСП-2, ГК «АСВ» – конкурсны...",1023000927520,1946,22,21.09.2004,...,Денежные средства на дату открытия конкурсного...,Стоимость реализованного имущества: 0\nПоступи...,68 176 тыс. руб.,Обновлено 01.05.2025,янв: 0 \nфев: 689 \nмар: 79 \nапр: 76 \n2016: ...,Обновлено 01.05.2025,"Предъявлено исков (Кол-во: 51, Сумма: 5 273 78...","1-я очередь (в реестре): Установлено 622 049, ...","Установлено: 991 747, Удовлетворено: 12 939, О...",NaN
3,https://www.asv.org.ru/banks/agrosoyuz-kb-ooo,2025-06-19 01:33:08,Общество с ограниченной ответственностью Комме...,"ООО КБ ""Агросоюз""","101000, г. Москва, Уланский пер., д. 13, стр. 1","109992, г. Москва, ГСП-2, ГК «АСВ» – конкурсны...",1025600001130,1459,649,17.02.2005,...,Денежные средства на дату открытия конкурсного...,Стоимость реализованного имущества: 1 180\nПос...,924 795 тыс. руб.,Обновлено 01.05.2025,янв: 1749 \nфев: 7670 \nмар: 4870 \nапр: 5576 ...,Обновлено 01.05.2025,"Предъявлено исков (Кол-во: 1 975, Сумма: 63 71...",1-я очередь (в реестре): Установлено 8 303 165...,"Установлено: 9 169 142, Удовлетворено: 2 204 0...",NaN
4,https://www.asv.org.ru/banks/admiralteyskiy-kb...,2025-06-19 01:34:08,"Коммерческий Банк ""Адмиралтейский"" (общество с...","ООО КБ ""Адмиралтейский""","119034, г. Москва, Лопухинский пер., д.6, стр.1","109992, г. Москва, ГСП-2, ГК «АСВ» – конкурсны...",1037739763753,3054,791,21.03.2005,...,Денежные средства на дату открытия конкурсного...,Стоимость реализованного имущества: 116\nПосту...,725 408 тыс. руб.,Обновлено 01.05.2025,янв: 683 \nфев: 1491 \nмар: 2802 \nапр: 2607 \...,Обновлено 01.05.2025,"Предъявлено исков (Кол-во: 260, Сумма: 22 091 ...",1-я очередь (в реестре): Установлено 3 867 592...,"Установлено: 7 089 011, Удовлетворено: 2 856 3...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280,https://www.asv.org.ru/banks/ao-orbank,2025-06-20 03:26:15,"Акционерное общество ""Объединенный резервный б...","АО ""ОРБАНК""","115280, г. Москва, ул. Ле

In [143]:
combined_data.to_csv('combined_data', encoding= ' utf-8')

## Парсинг проблемных банков

Вручную отбираем банки, у которых высветилась "Ошибка при обработке банка: 'URL'". У проблемных банков есть 3 категории: "problem_links" - банки, у которых есть только общая информация о банке. "problem_links_insur" - банки, у которых есть общая информация и информация про страховые выплаты, но нет другой, "parsing_problem_link" - банк, у которого есть вся информация, кроме страховых выплат.

In [134]:
problem_links = ['https://www.asv.org.ru/banks/sakhadaymondbank-kb-ooo', 'https://www.asv.org.ru/banks/peresvet-akb-pao-prezhnee-naimenovanie-peresvet-akb-ao-', 
                 'https://www.asv.org.ru/banks/mgb-rnko-ooo', 'https://www.asv.org.ru/banks/mgmb-ooo', 'https://www.asv.org.ru/banks/mega-bank-zao', 'https://www.asv.org.ru/banks/medinvestbank-ooo', 
                 'https://www.asv.org.ru/banks/ooo-kb-evroaziatskiy-investitsionnyy-bank']

In [162]:
problem_links_insur = ['https://www.asv.org.ru/banks/bank-ptb-ooo','https://www.asv.org.ru/banks/ao-runetbank', 'https://www.asv.org.ru/banks/ooo-kb-alba-alyans']

In [166]:
"""Эта программа работает примерно 25 минут"""

chrome_options = Options()
driver = webdriver.Chrome(options=chrome_options)

try:
    bank_links = problem_links
    problem_bank_data = [] # Парсим данные с каждой страницы банка
    for link in bank_links:
        print(f"Переходим по ссылке: {link}")
        driver.get(link)
        time.sleep(5)

        bank_info = {}  # Создаем словарь для хранения информации о банке
        try:
            try:
                ul_element = WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, 'div.foldable__body > div > app-company-block-requisites-table > ul')) # В коде страницы этот элемент располагается в ветви с названием 'ul', поэтому нам надо найти все 'ul', чтобы найти все элементы
                )
            except Exception as e:
                ul_element = None
                print()

            if ul_element:
                li_elements = ul_element.find_elements(By.TAG_NAME, "li") # Находим все элементы <li> внутри <ul>

                for li in li_elements:
                    try:
                        
                        title_element = li.find_element(By.CSS_SELECTOR, 'div.requisite-item__title > div') # Извлекаем название поля
                        title = title_element.text.strip()
                        value_element = li.find_element(By.CSS_SELECTOR, 'div.requisite-item__value > div') # Извлекаем значение поля
                        value = value_element.text.strip()

                        bank_info[title] = value

                    except Exception as e:
                        print(f"Ошибка при парсинге элемента <li>")
            else:
                print("Не удалось найти элемент ul, пропуск парсинга информации о банке")
                continue  # Перейти к следующей ссылке, если ul не найден

        except Exception as e:
            print(f"Ошибка при парсинге страницы {link}")

        problem_bank_data.append(bank_info)  # Сохраняем информацию о банке в список

    df_problem = pd.DataFrame(problem_bank_data)
    df_problem['URL'] = bank_links[:len(problem_bank_data)]
    cols = df_problem.columns.tolist()
    cols = ['URL'] + [col for col in cols if col != 'URL']
    df_problem = df_problem[cols]

except Exception as e:
    print(f"Произошла общая ошибка")

finally:
    driver.quit()

Переходим по ссылке: https://www.asv.org.ru/banks/sakhadaymondbank-kb-ooo
Переходим по ссылке: https://www.asv.org.ru/banks/peresvet-akb-pao-prezhnee-naimenovanie-peresvet-akb-ao-
Переходим по ссылке: https://www.asv.org.ru/banks/mgb-rnko-ooo
Переходим по ссылке: https://www.asv.org.ru/banks/mgmb-ooo
Переходим по ссылке: https://www.asv.org.ru/banks/mega-bank-zao
Переходим по ссылке: https://www.asv.org.ru/banks/medinvestbank-ooo
Переходим по ссылке: https://www.asv.org.ru/banks/ooo-kb-evroaziatskiy-investitsionnyy-bank


In [167]:
df_problem

,URL,Полное фирменное наименование,Сокращенное фирменное наименование,Место нахождения,ОГРН,Номер в Книге государственной регистрации кредитных организаций,Номер по реестру,Дата включения в реестр,Статусы,Адрес официального сайта в сети Интернет,Инвестор,Дата исключения из реестра,Основание для исключения из реестра
0,https://www.asv.org.ru/banks/sakhadaymondbank-...,"Коммерческий банк ""СахаДаймондБанк"" (Общество ...","Коммерческий банк ""СДБ"" (ООО)","101000, г. Москва, Колпачный пер., д.9, стр.1",1027739200884,3272,686,24.02.2005,Лицензия отозвана 30 сентября 2010\nУчастник ССВ,NaN,NaN,NaN,NaN
1,https://www.asv.org.ru/banks/peresvet-akb-pao-...,"Акционерный коммерческий банк ""ПЕРЕСВЕТ"" (Публ...","АКБ ""ПЕРЕСВЕТ"" (ПАО)","119049, г. Москва, ул. Шаболовка, д. 10, корп. 2",1027739250285,2110,304,16.12.2004,Процедура оздоровления введена 3 февраля 2017\...,www.bank-peresvet.ru/,Банк «ВБРР» (АО),NaN,NaN
2,https://www.asv.org.ru/banks/mgb-rnko-ooo,"Расчетная небанковская кредитная организация ""...","РНКО ""МГБ"" ООО","367000, Республика Дагестан, г. Махачкала, ул....",1020500001488,2866,612,10.02.2005,Лицензия отозвана 21 августа 2015,NaN,NaN,09.01.2017,Отзыв (аннулирование) лицензии и завершение пр...
3,https://www.asv.org.ru/banks/mgmb-ooo,"Общество с ограниченной ответственностью ""Маха...","ООО ""МГМБ""","367000, Республика Дагестан, г. Махачкала, ул....",1020500001180,3192,163,11.11.2004,Лицензия отозвана 8 июля 2013,NaN,NaN,07.06.2022,Отзыв (аннулирование) лицензии и завершение пр...
4,https://www.asv.org.ru/banks/mega-bank-zao,"Закрытое акционерное общество ""МЕГА БАНК""","ЗАО ""МЕГА БАНК""","625000, Тюменская область, г. Тюмень, ул. Уриц...",1027200000190,2220,966,23.07.2008,Лицензия отозвана 24 декабря 2009,NaN,NaN,30.11.2016,Отзыв (аннулирование) лицензии и завершение пр...
5,https://www.asv.org.ru/banks/medinvestbank-ooo,Общество с ограниченной ответственностью Инвес...,Инвестиционно-коммерческий банк развития медиц...,"113105, г. Москва, ул. Нагатинская, д.1",1037739766272,2058,702,24.02.2005,Лицензия отозвана 12 мая 2008,NaN,NaN,03.11.2010,Отзыв (аннулирование) лицензии и завершение пр...
6,https://www.asv.org.ru/banks/ooo-kb-evroaziats...,"Коммерческий банк ""Евроазиатский Инвестиционны...","ООО КБ ""Евроазиатский Инвестиционный Банк""","119021, г. Москва, Зубовский бульвар, д. 22/39",1027739915796,2897,670,17.02.2005,Лицензия отозвана 4 декабря 2020,NaN,Отзыв (аннулирование) лицензии и завершение пр...,18.10.2022,NaN


Программа, которая парсит только общую информацию и страховые выплаты

In [168]:
chrome_options = Options()
driver = webdriver.Chrome(options=chrome_options)

def format_amount(amount_str):
    if amount_str == "Информация отсутствует":
        return "Нет данных"
    try:
        amount = float(amount_str.replace(" ", "").replace("₽", "").strip())
        return amount
    except ValueError:
        return amount_str

try:
    bank_links = problem_links_insur
    problem_insur_bank_data = []
    
    for link in bank_links:
        print(f"Переходим по ссылке: {link}")
        driver.get(link)
        time.sleep(5)

        bank_info = {"URL": link}  # Добавляем URL банка в словарь

        try:
            ul_element = driver.find_element(By.CSS_SELECTOR, 'div.foldable__body > div > app-company-block-requisites-table > ul')

            li_elements = ul_element.find_elements(By.TAG_NAME, "li")
            for li in li_elements:
                try:
                    title_element = li.find_element(By.CSS_SELECTOR, 'div.requisite-item__title > div')
                    title = title_element.text.strip()
                    value_element = li.find_element(By.CSS_SELECTOR, 'div.requisite-item__value > div')
                    value = value_element.text.strip()
                    bank_info[title] = value
                except Exception as e:
                    print(f"Ошибка при парсинге элемента <li>: {e}")

        except Exception as e:
            print(f"Ошибка при парсинге информации о банке: {e}")
            bank_info["Информация о банке"] = "Информация отсутствует"

        # Блок страховых выплат
        bank_info.update({
            "Страховые выплаты - Выплачено": "Информация отсутствует",
            "Страховые выплаты - Осталось выплатить": "Информация отсутствует",
            "Страховые выплаты - Процент выплаченного": "Информация отсутствует"
        })

        try:
            insurance_payments_element = driver.find_element(By.CSS_SELECTOR, 'app-insurance-payments')

            paid_element = insurance_payments_element.find_element(By.XPATH, ".//div[@class='col-24 col-md-auto'][1]//span/nobr")
            bank_info["Страховые выплаты - Выплачено"] = paid_element.text.strip().replace(" ", "").replace("₽", "").strip()

            remaining_element = insurance_payments_element.find_element(By.XPATH, ".//div[@class='col-24 col-md-auto mt-4 mt-md-0'][1]//span/nobr")
            bank_info["Страховые выплаты - Осталось выплатить"] = remaining_element.text.strip().replace(" ", "").replace("₽", "").strip()

            percentage_element = insurance_payments_element.find_element(By.CSS_SELECTOR, 'div.progress-performed-number')
            bank_info["Страховые выплаты - Процент выплаченного"] = percentage_element.text.strip()

        except Exception as e:
            print(f"Ошибка при парсинге страховых выплат: {e}")

        problem_insur_bank_data.append(bank_info)

    # Создаем DataFrame
    df_problem_insur = pd.DataFrame(problem_insur_bank_data)
    
    # Форматируем числовые столбцы
    for col in ["Страховые выплаты - Выплачено", "Страховые выплаты - Осталось выплатить"]:
        df_problem_insur[col] = df_problem_insur[col].apply(format_amount)

    # Переносим столбец URL в начало DataFrame
    cols = ['URL'] + [col for col in df_problem_insur.columns if col != 'URL']
    df_problem_insur = df_problem_insur[cols]

except Exception as e:
    print(f"Произошла общая ошибка: {e}")
    df_problem_insur = pd.DataFrame()  # Создаем пустой DataFrame в случае ошибки

finally:
    driver.quit()

Переходим по ссылке: https://www.asv.org.ru/banks/bank-ptb-ooo
Переходим по ссылке: https://www.asv.org.ru/banks/ao-runetbank
Переходим по ссылке: https://www.asv.org.ru/banks/ooo-kb-alba-alyans


In [169]:
df_problem_insur

,URL,Полное фирменное наименование,Сокращенное фирменное наименование,Место нахождения,ОГРН,Номер в Книге государственной регистрации кредитных организаций,Номер по реестру,Дата включения в реестр,Временная администрация,Статусы,Страховые выплаты - Выплачено,Страховые выплаты - Осталось выплатить,Страховые выплаты - Процент выплаченного
0,https://www.asv.org.ru/banks/bank-ptb-ooo,ПромТрансБанк (Общество с ограниченной ответст...,Банк ПТБ (ООО),"450008 , Республика Башкортостан, г. Уфа, ул.Л...",1020200000083,2638,293,09.12.2004,Агентство по страхованию вкладов\nПриказ об от...,Лицензия отозвана 15 апреля 2025\nУчастник ССВ,"5030465070,59","607042392,95",
1,https://www.asv.org.ru/banks/ao-runetbank,"Акционерное общество ""РУНЭТБАНК""","АО ""РУНЭТБАНК""","123056, г. Москва, ул.Гашека, д.9, стр.1, корп. 4",1037711002581,2829,684,24.02.2005,NaN,Лицензия аннулирована 14 декабря 2018\nУчастни...,0.0,"53,32",
2,https://www.asv.org.ru/banks/ooo-kb-alba-alyans,"Коммерческий Банк ""Альба Альянс"" (Общество с о...","ООО КБ ""Альба Альянс""","119019, г. Москва, Кремлевская набережная, д. ...",1025000004787,2593,822,21.03.2005,NaN,Лицензия аннулирована 12 марта 2021\nУчастник ССВ,"3000578,05","3580978,08",


In [140]:
parsing_problem_link = ['https://www.asv.org.ru/banks/um-bank-ooo']

In [141]:
class UnifiedBankDataScraper:
    def __init__(self):
        self.driver = self._setup_driver()
        self.combined_df = pd.DataFrame()
        
    def _setup_driver(self):
        chrome_options = Options()
        chrome_options.add_argument('--disable-gpu')
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-dev-shm-usage')
        driver = webdriver.Chrome(options=chrome_options)
        driver.implicitly_wait(5)
        return driver
    
    def _scroll_to_element(self, element):
        actions = ActionChains(self.driver)
        actions.move_to_element(element).perform()
        time.sleep(5)
    
    def _expand_section(self, section):
        if 'card-foldable_open' not in section.get_attribute('class'):
            header = section.find_element(By.CSS_SELECTOR, 'div.card-foldable__header')
            header.click()
            time.sleep(5)
    
    def _parse_bank_info(self, url):
        """Парсинг основной информации о банке"""
        bank_info = {"URL": url, "Дата парсинга": datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
        
        try:
            ul_element = WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div.foldable__body > div > app-company-block-requisites-table > ul'))
            )
            
            for li in ul_element.find_elements(By.TAG_NAME, "li"):
                try:
                    title = li.find_element(By.CSS_SELECTOR, 'div.requisite-item__title > div').text.strip()
                    value = li.find_element(By.CSS_SELECTOR, 'div.requisite-item__value > div').text.strip()
                    bank_info[title] = value
                except:
                    continue
        except Exception as e:
            print(f"Ошибка парсинга информации о банке: {str(e)}")
        
        return bank_info
    
    def _parse_insurance_payments(self):
        """Парсинг страховых выплат"""
        payments = {}
        
        try:
            insurance_element = self.driver.find_element(By.CSS_SELECTOR, 'app-insurance-payments')
            
            paid = insurance_element.find_element(By.XPATH, ".//div[@class='col-24 col-md-auto'][1]//span/nobr").text.strip()
            remaining = insurance_element.find_element(By.XPATH, ".//div[@class='col-24 col-md-auto mt-4 mt-md-0'][1]//span/nobr").text.strip()
            percentage = insurance_element.find_element(By.CSS_SELECTOR, 'div.progress-performed-number').text.strip()
            
            payments.update({
                "Страховые выплаты - Выплачено": paid,
                "Страховые выплаты - Осталось выплатить": remaining,
                "Страховые выплаты - Процент выплаченного": percentage
            })
        except Exception as e:
            print(f"Ошибка парсинга страховых выплат: {str(e)}")
        
        return payments
    
    def _parse_competitive_mass(self, url):
        """Парсинг конкурсной массы и добавление в общий DataFrame"""
        try:
            sections = WebDriverWait(self.driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.card-foldable")))
            
            target_section = None
            for section in sections:
                try:
                    title = section.find_element(By.CSS_SELECTOR, "div.card-foldable-title__main-text").text
                    if "формировании конкурсной массы" in title.lower():
                        target_section = section
                        break
                except:
                    continue
            
            if not target_section:
                return pd.DataFrame()
            
            self._expand_section(target_section)
            self._scroll_to_element(target_section)
            time.sleep(2)
            
            temp_df = pd.DataFrame()

            try:
                total = target_section.find_element(By.CSS_SELECTOR, "div.card-foldable-title__secondary-text").text.strip()
                temp_df["Конкурсная масса - Общая сумма"] = [total]
            except:
                pass
            
            try:
                diagrams = target_section.find_elements(By.CSS_SELECTOR, "app-block-page-kp-diagram")
                dynamics = []
                for diagram in diagrams:
                    bars = diagram.find_elements(By.CSS_SELECTOR, "div.diagram__bar-wrapper")
                    for bar in bars:
                        try:
                            period = bar.find_element(By.CSS_SELECTOR, "div.diagram__month").text.strip()
                            amount = bar.find_element(By.CSS_SELECTOR, "div.diagram__percent span").text.strip()
                            currency = bar.find_elements(By.CSS_SELECTOR, "div.diagram__percent span")[1].text.strip()
                            dynamics.append(f"{period}: {amount} {currency}")
                        except:
                            continue
                temp_df["Конкурсная масса - Динамика"] = [json.dumps(dynamics, ensure_ascii=False)]
            except:
                pass
            
            try:
                tables = target_section.find_elements(By.CSS_SELECTOR, "div.card-body_foldable-card")
                for table in tables:
                    try:
                        header = table.find_element(By.CSS_SELECTOR, "span.body-bold").text
                        rows = table.find_elements(By.CSS_SELECTOR, "div.table-list__row_depth-0")
                        
                        table_data = []
                        for row in rows:
                            try:
                                if 'table-list__row_header' in row.get_attribute('class') or 'table-list__row_footer' in row.get_attribute('class'):
                                    continue
                                
                                title = row.find_element(By.CSS_SELECTOR, "span.table-list__cell-title-text").text.strip()
                                value = row.find_element(By.CSS_SELECTOR, "span.value").text.strip()
                                table_data.append(f"{title}: {value}")
                            except:
                                continue
                        
                        if "Имущество (активы)" in header:
                            temp_df["Конкурсная масса - Имущество"] = [json.dumps(table_data, ensure_ascii=False)]
                        elif "Сформированная конкурсная масса" in header:
                            temp_df["Конкурсная масса - Сформированная масса"] = [json.dumps(table_data, ensure_ascii=False)]
                        elif "Реализация имущества" in header:
                            temp_df["Конкурсная масса - Реализация имущества"] = [json.dumps(table_data, ensure_ascii=False)]
                    except:
                        continue
            except:
                pass
            
            temp_df["URL"] = url
            
            return temp_df
            
        except Exception as e:
            print(f"Ошибка парсинга конкурсной массы: {str(e)}")
            return pd.DataFrame()
    
    def _parse_expenses(self, url):
        """Парсинг расходов и добавление в общий DataFrame"""
        try:
            sections = WebDriverWait(self.driver, 20).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.card-foldable")))
            
            target_section = None
            for section in sections:
                try:
                    title = section.find_element(By.CSS_SELECTOR, "div.card-foldable-title__main-text").text
                    if "Расходы" in title:
                        target_section = section
                        break
                except:
                    continue
            
            if not target_section:
                return pd.DataFrame()
            
            self._expand_section(target_section)
            self._scroll_to_element(target_section)
            time.sleep(2)
            
            temp_df = pd.DataFrame()
            
            try:
                total = target_section.find_element(By.CSS_SELECTOR, "div.card-foldable-title__secondary-text").text.strip()
                temp_df["Расходы - Общая сумма"] = [total]
            except:
                pass
            
            try:
                date = target_section.find_element(By.CSS_SELECTOR, "div.card-foldable__desc").text.strip()
                temp_df["Расходы - Дата обновления"] = [date]
            except:
                pass
            
            try:
                diagrams = target_section.find_elements(By.CSS_SELECTOR, "app-block-page-kp-diagram")
                dynamics = []
                for diagram in diagrams:
                    bars = diagram.find_elements(By.CSS_SELECTOR, "div.diagram__bar-wrapper")
                    for bar in bars:
                        try:
                            period = bar.find_element(By.CSS_SELECTOR, "div.diagram__month").text.strip()
                            amount = bar.find_element(By.CSS_SELECTOR, "div.diagram__percent span").text.strip()
                            currency = bar.find_elements(By.CSS_SELECTOR, "div.diagram__percent span")[1].text.strip()
                            dynamics.append(f"{period}: {amount} {currency}")
                        except:
                            continue
                temp_df["Расходы - Динамика"] = [json.dumps(dynamics, ensure_ascii=False)]
            except:
                pass
            
            try:
                expenses_header = target_section.find_element(By.XPATH, ".//span[contains(text(), 'Наименования расходов')]")
                expenses_table = expenses_header.find_element(By.XPATH, "./following-sibling::div[contains(@class, 'table-expenses-list')]")
                
                expenses_data = []
                rows = expenses_table.find_elements(By.CSS_SELECTOR, "app-table-statistic-row")
                for row in rows:
                    try:
                        title = row.find_element(By.CSS_SELECTOR, "span.table-list__cell-title-text").text.strip()
                        value = row.find_element(By.CSS_SELECTOR, "span.value").text.strip()
                        expenses_data.append(f"{title}: {value}")
                    except:
                        continue
                
                temp_df["Расходы - Статьи"] = [json.dumps(expenses_data, ensure_ascii=False)]
            except:
                pass
            
            temp_df["URL"] = url
            
            return temp_df
            
        except Exception as e:
            print(f"Ошибка парсинга расходов: {str(e)}")
            return pd.DataFrame()
    
    def _parse_legal_work(self, url):
        """Парсинг юридической работы и добавление в общий DataFrame"""
        try:
            sections = WebDriverWait(self.driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.card-foldable")))
            
            target_section = None
            for section in sections:
                try:
                    title = section.find_element(By.CSS_SELECTOR, "div.card-foldable-title__main-text").text
                    if "Юридическая работа" in title:
                        target_section = section
                        break
                except:
                    continue
            
            if not target_section:
                return pd.DataFrame()
            
            self._expand_section(target_section)
            self._scroll_to_element(target_section)
            time.sleep(2)
            
            temp_df = pd.DataFrame()
            
            try:
                date = target_section.find_element(By.CSS_SELECTOR, "div.card-foldable__desc").text.strip()
                temp_df["Юридическая работа - Дата обновления"] = [date]
            except:
                pass
            
            try:
                table = target_section.find_element(By.CSS_SELECTOR, "div.table-list")
                actions_data = []
                rows = table.find_elements(By.CSS_SELECTOR, "div.table-list__row.ng-star-inserted")
                
                for row in rows:
                    try:
                        action = row.find_element(By.CSS_SELECTOR, "span.tooltip").text.strip()
                        count = row.find_element(By.CSS_SELECTOR, "div.table-list__sub-cell.second-value").text.strip()
                        amount = row.find_element(By.CSS_SELECTOR, "div.table-list__cell_number nobr").text.strip()
                        actions_data.append(f"{action} (Кол-во: {count}, Сумма: {amount})")
                    except:
                        continue
                
                temp_df["Юридическая работа - Процессуальные действия"] = [json.dumps(actions_data, ensure_ascii=False)]
            except:
                pass
            

            temp_df["URL"] = url
            
            return temp_df
            
        except Exception as e:
            print(f"Ошибка парсинга юридической работы: {str(e)}")
            return pd.DataFrame()
    
    def _parse_payments_to_creditors(self, url):
        """Парсинг выплат кредиторам и добавление в общий DataFrame"""
        try:
            payments_section = self.driver.find_element(By.XPATH, "//div[contains(., 'Выплачено кредиторам')]")
            self.driver.execute_script("arguments[0].scrollIntoView(true);", payments_section)
            time.sleep(2)
            
            temp_df = pd.DataFrame()
            
            try:
                rows = self.driver.find_elements(By.CSS_SELECTOR, "div.table-list__row_depth-0")
                payments_data = []
                
                for row in rows:
                    try:
                        if ('table-list__row_header' in row.get_attribute('class') or 
                            'table-list__row_footer' in row.get_attribute('class')):
                            continue
                        
                        title = row.find_element(By.CSS_SELECTOR, "span.table-list__cell-title-text").text.strip()
                        cells = row.find_elements(By.CSS_SELECTOR, "div.table-list__cell-value")
                        
                        if len(cells) >= 3:
                            payments_data.append(
                                f"{title}: Установлено {cells[0].text.strip()}, "
                                f"Удовлетворено {cells[1].text.strip()}, "
                                f"Отступное {cells[2].text.strip()}"
                            )
                    except:
                        continue
                
                temp_df["Выплаты кредиторам - Данные"] = [json.dumps(payments_data, ensure_ascii=False)]
            except:
                pass
            
            try:
                total_row = self.driver.find_element(By.CSS_SELECTOR, "div.table-list__row_footer")
                cells = total_row.find_elements(By.CSS_SELECTOR, "div.table-list__cell-value")
                
                if len(cells) >= 3:
                    temp_df["Выплаты кредиторам - Итого"] = [
                        f"Установлено: {cells[0].text.strip()}, "
                        f"Удовлетворено: {cells[1].text.strip()}, "
                        f"Отступное: {cells[2].text.strip()}"
                    ]
            except:
                pass
            
            temp_df["URL"] = url
            
            return temp_df
            
        except Exception as e:
            print(f"Ошибка при парсинге выплат кредиторам: {str(e)}")
            return pd.DataFrame()
    
    def scrape_bank(self, url):
        """Сбор всех данных по одному банку и объединение в один DataFrame"""
        print(f"\nПарсинг банка: {url}")
        self.driver.get(url)
        time.sleep(5)
        
        self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
        self.driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(5)
        
        bank_info = self._parse_bank_info(url)
        insurance = self._parse_insurance_payments()
        competitive_mass = self._parse_competitive_mass(url)
        expenses = self._parse_expenses(url)
        legal_work = self._parse_legal_work(url)
        payments = self._parse_payments_to_creditors(url)
        
        bank_df = pd.DataFrame([bank_info])
        
        bank_df = bank_df.merge(pd.DataFrame([insurance]), how='left', left_index=True, right_index=True)
        bank_df = bank_df.merge(competitive_mass, how='left', on='URL')
        bank_df = bank_df.merge(expenses, how='left', on='URL')
        bank_df = bank_df.merge(legal_work, how='left', on='URL')
        bank_df = bank_df.merge(payments, how='left', on='URL')
        
        self.combined_df = pd.concat([self.combined_df, bank_df], ignore_index=True)
        
        print(f"Данные по банку {url} добавлены в общий DataFrame")
        return bank_df
    
    def scrape_all_banks(self, bank_urls):
        """Сбор данных по всем банкам"""
        for url in bank_urls:
            try:
                self.scrape_bank(url)
                time.sleep(3)
            except Exception as e:
                print(f"Ошибка при обработке банка {url}: {str(e)}")
                continue
        
        self.combined_df.fillna("Нет данных", inplace=True)
        
        json_columns = [
            "Конкурсная масса - Динамика",
            "Конкурсная масса - Имущество",
            "Конкурсная масса - Сформированная масса",
            "Конкурсная масса - Реализация имущества",
            "Расходы - Динамика",
            "Расходы - Статьи",
            "Юридическая работа - Процессуальные действия",
            "Выплаты кредиторам - Данные"
        ]
        
        for col in json_columns:
            if col in self.combined_df.columns:
                self.combined_df[col] = self.combined_df[col].apply(
                    lambda x: "\n".join(json.loads(x)) if x != "Нет данных" else x
                )
        
        return self.combined_df
    
    def save_to_excel(self, filename="all_banks_combined.xlsx"):
        """Сохранение объединенного DataFrame в Excel"""
        if not self.combined_df.empty:
            self.combined_df.to_excel(filename, index=False, encoding='utf-8-sig')
            print(f"\nВсе данные сохранены в файл {filename}")
        else:
            print("\nНет данных для сохранения")
    
    def close(self):
        """Закрытие драйвера"""
        self.driver.quit()

if __name__ == "__main__":
    bank_urls = parsing_problem_link
    
    scraper = UnifiedBankDataScraper()
    try:
        parsing_problem_df = scraper.scrape_all_banks(bank_urls)
        
        print("\nСоздан объединенный DataFrame со следующими столбцами:")
        print(parsing_problem_df.columns.tolist())
        print(f"\nВсего записей: {len(parsing_problem_df)}")
        
        scraper.save_to_excel("bank_data.xlsx")
        
        print("\nПервые строки данных:")
        print(parsing_problem_df.head())
        
    except Exception as e:
        print(f"Произошла ошибка: {str(e)}")
    finally:
        scraper.close()


Парсинг банка: https://www.asv.org.ru/banks/um-bank-ooo
Ошибка парсинга страховых выплат: Message: no such element: Unable to locate element: {"method":"css selector","selector":"app-insurance-payments"}
  (Session info: chrome=137.0.7151.120); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x0x7ff7eb6ccda5+78885]
	GetHandleVerifier [0x0x7ff7eb6cce00+78976]
	(No symbol) [0x0x7ff7eb489bca]
	(No symbol) [0x0x7ff7eb4e0766]
	(No symbol) [0x0x7ff7eb4e0a1c]
	(No symbol) [0x0x7ff7eb534467]
	(No symbol) [0x0x7ff7eb508bcf]
	(No symbol) [0x0x7ff7eb53122f]
	(No symbol) [0x0x7ff7eb508963]
	(No symbol) [0x0x7ff7eb4d16b1]
	(No symbol) [0x0x7ff7eb4d2443]
	GetHandleVerifier [0x0x7ff7eb9a4eed+3061101]
	GetHandleVerifier [0x0x7ff7eb99f33d+3037629]
	GetHandleVerifier [0x0x7ff7eb9be592+3165202]
	GetHandleVerifier [0x0x7ff7eb6e730e+186766]
	GetHandleVerifier [0x0x7ff7eb6eeb3f+2

In [142]:
parsing_problem_df

,URL,Дата парсинга,Полное фирменное наименование,Сокращенное фирменное наименование,Место нахождения,Почтовый адрес,ОГРН,Номер в Книге государственной регистрации кредитных организаций,Номер по реестру,Дата включения в реестр,...,Конкурсная масса - Имущество,Конкурсная масса - Сформированная масса,Конкурсная масса - Реализация имущества,Расходы - Общая сумма,Расходы - Дата обновления,Расходы - Динамика,Юридическая работа - Дата обновления,Юридическая работа - Процессуальные действия,Выплаты кредиторам - Данные,Выплаты кредиторам - Итого
0,https://www.asv.org.ru/banks/um-bank-ooo,2025-06-20 08:44:20,"""УМ-Банк"" Общество с ограниченной ответственно...","""УМ-Банк"" ООО","115184, г. Москва, ул. Ордынка Б, д. 59, стр. 2","127994, г. Москва, ГСП-4, ГК «АСВ» – конкурсны...",1026600000063,1441,151,11.11.2004,...,Денежные средства и драгоценные металлы: 0\nКо...,Денежные средства на дату открытия конкурсного...,Стоимость реализованного имущества: 0\nПоступи...,209 235 тыс. руб.,Обновлено 01.05.2025,янв: 238 \nфев: 52 \nмар: 11 \nапр: 88 \n2019:...,Обновлено 01.05.2025,"Предъявлено исков (Кол-во: 114, Сумма: 9 060 2...","1-я очередь (в реестре): Установлено 0, Удовле...","Установлено: 355 449, Удовлетворено: 355 449, ..."


In [170]:
merged_step1 = pd.merge(
    parsing_problem_df,
    df_problem_insur,
    on='URL',
    how='outer',
    suffixes=('', '_insur')
)

merged_problem = pd.merge(
    merged_step1,
    df_problem,
    on='URL',
    how='outer',
    suffixes=('', '_prob')
)

duplicate_cols = [col for col in df_problem_insur.columns if col in parsing_problem_df and col != 'URL']
duplicate_cols += [col for col in df_problem.columns if col in merged_step1 and col != 'URL']

for col in duplicate_cols:
    if f'{col}_insur' in merged_problem.columns:
        merged_problem[col] = merged_problem[col].fillna(merged_problem[f'{col}_insur'])
        merged_problem.drop(f'{col}_insur', axis=1, inplace=True)
    if f'{col}_prob' in merged_problem.columns:
        merged_problem[col] = merged_problem[col].fillna(merged_problem[f'{col}_prob'])
        merged_problem.drop(f'{col}_prob', axis=1, inplace=True)

merged_problem = merged_problem.fillna('Нет данных')
merged_problem = merged_problem.drop_duplicates(subset='URL')
print(f"Итоговое количество строк: {len(merged_problem)}")

Итоговое количество строк: 11


In [171]:
merged_problem

,URL,Дата парсинга,Полное фирменное наименование,Сокращенное фирменное наименование,Место нахождения,Почтовый адрес,ОГРН,Номер в Книге государственной регистрации кредитных организаций,Номер по реестру,Дата включения в реестр,...,Выплаты кредиторам - Данные,Выплаты кредиторам - Итого,Временная администрация,Страховые выплаты - Выплачено,Страховые выплаты - Осталось выплатить,Страховые выплаты - Процент выплаченного,Адрес официального сайта в сети Интернет,Инвестор,Дата исключения из реестра,Основание для исключения из реестра
0,https://www.asv.org.ru/banks/ao-runetbank,Нет данных,"Акционерное общество ""РУНЭТБАНК""","АО ""РУНЭТБАНК""","123056, г. Москва, ул.Гашека, д.9, стр.1, корп. 4",Нет данных,1037711002581,2829,684,24.02.2005,...,Нет данных,Нет данных,Нет данных,0.0,"53,32",,Нет данных,Нет данных,Нет данных,Нет данных
1,https://www.asv.org.ru/banks/bank-ptb-ooo,Нет данных,ПромТрансБанк (Общество с ограниченной ответст...,Банк ПТБ (ООО),"450008 , Республика Башкортостан, г. Уфа, ул.Л...",Нет данных,1020200000083,2638,293,09.12.2004,...,Нет данных,Нет данных,Агентство по страхованию вкладов\nПриказ об от...,"5030465070,59","607042392,95",,Нет данных,Нет данных,Нет данных,Нет данных
2,https://www.asv.org.ru/banks/medinvestbank-ooo,Нет данных,Общество с ограниченной ответственностью Инвес...,Инвестиционно-коммерческий банк развития медиц...,"113105, г. Москва, ул. Нагатинская, д.1",Нет данных,1037739766272,2058,702,24.02.2005,...,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,03.11.2010,Отзыв (аннулирование) лицензии и завершение пр...
3,https://www.asv.org.ru/banks/mega-bank-zao,Нет данных,"Закрытое акционерное общество ""МЕГА БАНК""","ЗАО ""МЕГА БАНК""","625000, Тюменская область, г. Тюмень, ул. Уриц...",Нет данных,1027200000190,2220,966,23.07.2008,...,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,30.11.2016,Отзыв (аннулирование) лицензии и завершение пр...
4,https://www.asv.org.ru/banks/mgb-rnko-ooo,Нет данных,"Расчетная небанковская кредитная организация ""...","РНКО ""МГБ"" ООО","367000, Республика Дагестан, г. Махачкала, ул....",Нет данных,1020500001488,2866,612,10.02.2005,...,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,09.01.2017,Отзыв (аннулирование) лицензии и завершение пр...
5,https://www.asv.org.ru/banks/mgmb-ooo,Нет данных,"Общество с ограниченной ответственностью ""Маха...","ООО ""МГМБ""","367000, Республика Дагестан, г. Махачкала, ул....",Нет данных,1020500001180,3192,163,11.11.2004,...,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,07.06.2022,Отзыв (аннулирование) лицензии и завершение пр...
6,https://www.asv.org.ru/banks/ooo-kb-alba-alyans,Нет данных,"Коммерческий Банк ""Альба Альянс"" (Общество с о...","ООО КБ ""Альба Альянс""","119019, г. Москва, Кремлевская набережная, д. ...",Нет данных,1025000004787,2593,822,21.03.2005,...,Нет данных,Нет данных,Нет данных,"3000578,05","3580978,08",,Нет данных,Нет данных,Нет данных,Нет данных
7,https://www.asv.org.ru/banks/ooo-kb-evroaziats...,Нет данных,"Коммерческий банк ""Евроазиатский Инвестиционны...","ООО КБ ""Евроазиатский Инвестиционный Банк""","119021, г. Москва, Зубовский бульвар, д. 22/39",Нет данных,1027739915796,2897,670,17.02.2005,...,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Отзыв (аннулирование) лицензии и завершение пр...,18.10.2022,Нет данных
8,https://www.asv.org.ru/banks/peresvet-akb-pao-...,Нет данных,"Акционерный коммерческий банк ""ПЕРЕСВЕТ"" (Публ...","АКБ ""ПЕРЕСВЕТ"" (ПАО)","119049, г. Москва, ул. Шаболовка, д. 10, корп. 2",Нет данных,1027739250285,2110,304,16.12.2004,...,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,www.bank-peresvet.ru/,Банк «ВБРР» (АО),Нет данных,Нет данных
9,https://www.asv.org.ru/banks/sakhadaymondbank-...,Нет данных,"Коммерческий банк ""СахаДаймондБанк"" (Общество ...","Коммерческий банк ""СДБ"" (ООО)","10

## Итоговое объединение всех датафреймов

In [ ]:
if 'Ссылка на банк' in combined_df.columns:
    combined_df = combined_df.rename(columns={'Ссылка на банк': 'URL'})
if 'URL' not in combined_data.columns or 'URL' not in combined_df.columns:
    common_columns = set(combined_data.columns) & set(combined_df.columns)
    if not common_columns:
        raise ValueError("Нет общих столбцов для объединения")
    
    merge_key = common_columns.pop()
else:
    merge_key = 'URL'

combined_data = combined_data.reset_index(drop=True)
combined_df = combined_df.reset_index(drop=True)

common_cols = list(set(combined_data.columns) & set(combined_df.columns) - {merge_key})

merged = pd.merge(
    combined_data.drop(columns=common_cols),
    combined_df,
    on=merge_key,
    how='outer',
    suffixes=('', '_y')
)

for col in common_cols:
    if f'{col}_y' in merged.columns:
        # Берем данные из combined_data, если они не пустые, иначе из combined_df
        merged[col] = merged[col].where(merged[col].notna(), merged[f'{col}_y'])
        merged.drop(f'{col}_y', axis=1, inplace=True)

merged = merged.fillna('Нет данных')
merged = merged.drop_duplicates(subset=merge_key)
merged.to_excel('merged_banks.xlsx', index=False)
print(f"Итоговое количество строк: {len(merged)}")

Итоговое количество строк: 553
